In [2]:
from falcomchain.helper import load_pickle, save_pickle, DataHandler
from falcomchain.tree import SpanningTree, capacitated_recursive_tree, bipartition_tree, find_edge_cuts, find_superedge_cuts, one_sided_cut, two_sided_cut

In [3]:
handler = DataHandler()
graph = handler.load_graphhh()
gg = handler.load_graph()
chicago = handler.load_chicagooo()

In [4]:
chicago.head()

,id,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,...,AWATER20,INTPTLAT20,INTPTLON20,geometry,constant,population,phc,area,color,centroid_geo
8514,127589,17,031,842400,1027,170318424001027,Block 1027,G5040,R,,...,0,+41.7389993,-087.6252955,"POLYGON ((-87.62558 41.74182, -87.62557 41.741...",2,0.0,0,22750.846549,0,POINT (-87.62530 41.73900)
9593,130205,17,031,842400,2049,170318424002049,Block 2049,G5040,R,,...,0,+41.7424462,-087.6285590,"POLYGON ((-87.62910 41.74312, -87.62909 41.742...",2,44.0,0,13466.809500,1,POINT (-87.62856 41.74245)
26893,186912,17,031,842400,2048,170318424002048,Block 2048,G5040,R,,...,0,+41.7424633,-087.6277213,"POLYGON ((-87.62804 41.74314, -87.62802 41.742...",2,30.0,0,7647.023154,1,POINT (-87.62772 41.74246)
17558,154120,17,031,842400,2050,170318424002050,Block 2050,G5040,R,,...,0,+41.7424739,-087.6271135,"POLYGON ((-87.62744 41.74315, -87.62740 41.741...",2,32.0,0,7639.960569,1,POINT (-87.62711 41.74247)
21143,169435,17,031,842400,2046,170318424002046,Block 2046,G5040,R,,...,0,+41.7424866,-087.6265068,"POLYGON ((-87.62683 41.74316, -87.62680 41.742...",2,24.0,0,7570.357731,1,POINT (-87.62651 41.74249)


In [6]:
# --- Make a copy ---
del chicago['centroid_geo']
chicago_fixed = chicago.copy()

# --- Replace (or add) the 'id' column with the index values ---
chicago_fixed["id"] = chicago_fixed.index.astype(str)  # ensure string ids

# --- Save to JSON ---
chicago_fixed.to_file("blocks.json", driver="GeoJSON")

In [28]:
gg.nodes[37453]


{'boundary_node': False,
 'area': 33006.70523853937,
 'GEOID20': '170310321004000',
 'C_X': -87.65128238444233,
 'C_Y': 41.95785778242464,
 'population': 1479,
 'candidate': 0,
 'real_candidate': 0,
 'artificial_candidate': 0}

In [32]:
1533*(0.75)

1149.75

__slots__ = (
        "graph",
        "root",
        "pop_col",
        "area_col",
        "facility_col",
        "density_col",
        "ideal_pop",
        "n_teams",
        "capacity_level",
        "epsilon",
        "successors",
        "tot_pop",
        "supertree",
        "two_sided",
        "tot_candidates",
    )

In [3]:
path = "/Users/kirtisoglu/Documents/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/falcomchain/stree.pkl"
tree = load_pickle(path)

In [4]:
tree.epsilon

0.07111156982144701

In [ ]:
import json
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

specials = set(tree.graph.nodes)

# Make a copy and add a color column
geo_data = chicago.copy()
geojson_data = json.loads(geo_data.to_json())
geo_data["fill_color"] = geo_data.index.map(
    lambda idx: "yellow" if idx in specials else "lightgray"
)

fig = px.choropleth_mapbox(
    geo_data,
    geojson=geojson_data,
    locations=geo_data.index,
    color="fill_color",  # use custom color column
    color_discrete_map="identity",  # tells Plotly to use literal color names
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": -87.6298},
    zoom=9,
    opacity=0.6,
    width=1200,
    height=900,
)

fig.update_layout(
    title="Chicago Census Blocks (highlighted specials in yellow)",
    margin={"r":0,"t":40,"l":0,"b":0}
)

fig.show()

In [ ]:
import json
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# --- Data prep ---
# Ensure lon/lat CRS for plotting and centroid extraction
del chicago['centroid_geo']
geo_ll = chicago.to_crs(epsg=4326).copy()

# ids to highlight (assumes they match chicago/geo_ll index)
specials = set(tree.graph.nodes)

# Base fill colors: yellow for specials, gray otherwise
geo_ll["fill_color"] = geo_ll.index.map(lambda idx: "yellow" if idx in specials else "lightgray")

# GeoJSON for the polygon layer
geojson_data = json.loads(geo_ll.to_json())

# --- Base choropleth ---
fig = px.choropleth_mapbox(
    geo_ll,
    geojson=geojson_data,
    locations=geo_ll.index,
    color="fill_color",
    color_discrete_map="identity",
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": -87.6298},
    zoom=9,
    opacity=0.6,
    width=1800,
    height=900,
)

# --- Centroid points for specials (red markers) ---
centroids = geo_ll.geometry.centroid
special_idxs = [i for i in specials if i in centroids.index]
special_pts = centroids.loc[special_idxs]

# Build hover text for each node
hover_texts = []
for node in special_idxs:
    has_fac = tree.has_facility(node)
    comp_has_fac = tree.complement_has_facility(node)
    pop = tree.graph.nodes[node].get("population", "N/A")

    hover_texts.append(
        f"<b>Node:</b> {node}<br>"
        f"has_facility: {has_fac}<br>"
        f"complement_has_facility: {comp_has_fac}<br>"
        f"population: {pop}"
    )

fig.add_trace(go.Scattermapbox(
    lon=[p.x for p in special_pts],
    lat=[p.y for p in special_pts],
    mode="markers",
    marker=dict(size=5, color="red"),
    name="Special centroids",
    text=hover_texts,
    hoverinfo="text"
))

# --- Tree edges between specials (red lines) ---
# Build a single multi-segment trace with None separators
edge_lons, edge_lats = [], []
for u, v in tree.graph.edges():
    if u in centroids.index and v in centroids.index:
        pu, pv = centroids[u], centroids[v]
        edge_lons += [pu.x, pv.x, None]
        edge_lats += [pu.y, pv.y, None]

fig.add_trace(go.Scattermapbox(
    lon=edge_lons,
    lat=edge_lats,
    mode="lines",
    line=dict(width=1, color="red"),
    name="Tree edges"
))

# --- Layout ---
fig.update_layout(
    title="Chicago Census Blocks with Special Centroids and Tree Edges",
    margin={"r":0,"t":40,"l":0,"b":0}
)
fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        bearing=90,          # rotation in degrees, 0 = north-up, 90 = east-up
        pitch=0              # optional: camera tilt (0 = straight down)
    )
)
#fig.show()


In [6]:
import os, webbrowser
from plotly.io import write_html

filename = "chicago_tree_map.html"
write_html(fig, file=filename, full_html=True, include_plotlyjs=True)
webbrowser.open("file://" + os.path.abspath(filename))

True

In [17]:
root = tree.root
tree.graph.nodes[root]['population']

72604

In [ ]:
def has_ideal_pop(self, assign_team, pop):
        return (
            abs(pop - assign_team * self.ideal_pop)
            <= self.ideal_pop * assign_team * self.epsilon
        )

    def complement_has_the_ideal_pop(self, assign_team, pop):
        return (
            abs((self.tot_pop - pop) - assign_team * self.ideal_pop)
            <= self.ideal_pop * assign_team * self.epsilon
        )

    def complement_has_ideal_pop_too(self, assign_team, pop):
        return (
            abs((self.tot_pop - pop) - (self.n_teams - assign_team) * self.ideal_pop)
            <= self.ideal_pop * (self.n_teams - assign_team) * self.epsilon
        )



tree.has_facility(node)
tree.complement_has_facility(node)
tree.graph.nodes[node]['population']
[tree.ideal_pop - tree.epsilon, tree]

In [ ]:
from falcomchain.tree import export_tree

# usage
export_tree(tree, "tree.json")


In [25]:
tree.graph.nodes[35470]

{'boundary_node': False,
 'area': 10847.316786183628,
 'GEOID20': '170313301021003',
 'population': 56,
 'C_X': -87.73480077531124,
 'C_Y': 41.78307754537012,
 'candidate': 0,
 'real_candidate': 0,
 'artificial_candidate': 0,
 'has_facility': False,
 'compl_facility': True,
 'x': -87.73480077531124,
 'y': 41.78307754537012}

In [20]:
print(chicago.crs)
print("Bounds:", chicago.total_bounds)

EPSG:4326
Bounds: [-87.74351   41.730383 -87.528879  41.921598]


In [9]:
import random
node = random.choice(list(G.nodes))
G.nodes[node]

{'boundary_node': False,
 'area': 4122898.4597857897,
 'GEOID20': '170313301023005',
 'population': 9443,
 'C_X': -87.66430966996171,
 'C_Y': 41.87623986225397,
 'candidate': 16,
 'real_candidate': 0,
 'artificial_candidate': 1,
 'has_facility': True,
 'compl_facility': True,
 'x': -87.66430966996171,
 'y': 41.87623986225397}

In [2]:
from dataclasses import dataclass, field, replace
from typing import Any, Dict, Set
from falcomchain.tree import Flip
# (Assume your Flip class is defined above)

# 1. Create an original Flip instance
original_flip = Flip(
    new_ids={'A', 'B'},
)

print(f"Original Object ID: {id(original_flip)}")
print(f"Original new_ids: {original_flip.new_ids}")
print(f"Original super_cut_object: {original_flip.super_cut_object}")
print(f"Original merged_ids: {original_flip.merged_ids}")

# 2. Call the method to create a NEW instance
new_merged_set = {'C', 'D'}
new_flip_instance = original_flip.add_merged_ids(new_merged_set)

print("-" * 20)
print(f"New Object ID: {id(new_flip_instance)}") # ID is different
print(f"New new_ids: {new_flip_instance.new_ids}") # KEPT from original
print(f"New super_cut_object: {new_flip_instance.super_cut_object}") # KEPT from original
print(f"New merged_ids: {new_flip_instance.merged_ids}") # UPDATED to the new set







Original Object ID: 4387582896
Original new_ids: {'B', 'A'}
Original super_cut_object: None
Original merged_ids: set()
--------------------
New Object ID: 5877837568
New new_ids: {'B', 'A'}
New super_cut_object: None
New merged_ids: {'D', 'C'}


In [1]:
import numpy as np
import geopandas as gpd
 
import pickle 
import random
import json

from functools import partial
from types import MappingProxyType
#from tqdm import tqdm


from functools import partial

from falcomchain.partition import Partition, Assignment

from falcomchain.helper import DataHandler #add_to_data, add_to_graph, save_pickle, load_pickle


"Load data"

handler = DataHandler()
#graph = handler.load_graph()
graphhh = handler.load_graphhh()
#chicago = handler.load_chicago()
chicagooo = handler.load_chicagooo()
#candidates = handler.load_candidates()  # set of nodes
#travel_times_walking = handler.load_travel_walking()
#travel_times = handler.load_travel_times() # travel time between blocks is 10 minutes
#newtravel = handler.load_newtravel()
real_travel_times = handler.load_real_travel_times()


In [1]:
parts = {1: frozenset({29145, 9554, 31287}), 2: frozenset({2981, 2534, 20044, 15155, 6421, 13430, 13431, 37176, 3256}), 3: frozenset({36453, 5446, 11577, 21323, 7884, 19117, 5331, 5332, 4053, 11574, 21816, 25081, 11578}), 4: frozenset({12033, 20747, 20496, 33233, 4637, 4638, 37663, 37664, 7713, 7711, 16225, 17060, 16230, 37674, 37675, 37676, 16301, 16302, 37677, 4974, 16752, 2675, 37685, 37686, 16309, 14522, 16831}), 5: frozenset({22400, 22401, 21892, 21893, 21894, 19852, 38417, 13586, 13587, 13588, 19861, 38418, 7193, 19867, 16540, 20894, 39071, 20895, 12068, 22396, 25388, 23476, 950, 2232, 3384, 3386, 3387, 15676, 2233, 3385, 2623, 38336, 13635, 11851, 3535, 2640, 5719, 5720, 5721, 5722, 17499, 5724, 5723, 26463, 15205, 5738, 17516, 15214, 15215, 15216, 23920, 16754, 22395, 16508, 22397, 22398}), 6: frozenset({19458, 20492, 19216, 20516, 9253, 9258, 16432, 30008, 18491, 2366, 30016, 30017, 30022, 30023, 30027, 5453, 30035, 30036, 30039, 26711, 30041, 30042, 26717, 30045, 20831, 37476, 26213, 26214, 7281, 24690, 24694, 24695, 24696, 24697, 24699, 24700, 7293, 2685, 1918, 7296, 7295, 3722, 3723, 3724, 5516, 5515, 20625, 13971, 28052, 27291, 27292, 27293, 3236, 20648, 18606, 9153, 2770, 5076, 12244, 5078, 3031, 5079, 12248, 5080, 12245, 19414, 19415, 19416, 19417, 9440, 25077, 12277}), 7: frozenset({27272, 12169, 5003, 8972, 18453, 7319, 16799, 27556, 4134, 27558, 27560, 13996, 3628, 21422, 21295, 27568, 21424, 10544, 27569, 27572, 27571, 27570, 21567, 575, 27595, 26070, 26071, 4839, 25322, 25323, 4331, 11886, 13305, 13308, 13310}), 8: frozenset({30986, 30987, 33168, 4561, 9491, 3411, 14548, 33174, 85, 29781, 87, 14549, 89, 88, 3413, 92, 29788, 14173, 1956, 35556, 37608, 1067, 23980, 39086, 39087, 39091, 39092, 30581, 14137, 15483, 11775}), 9: frozenset({6720, 20164, 11332, 20176, 11346, 11347, 26839, 38810, 38811, 5212, 17884, 21354, 14769, 1972, 4408, 29433, 29434, 23547, 29435}), 10: frozenset({10560, 19779, 27271, 27273, 27274, 27219, 1305, 5023, 12001, 8676, 8677, 34790, 12391, 25321, 3626, 21548, 25324, 20271, 10545, 13301, 28663, 6969, 21433, 36478}), 11: frozenset({5377, 23042, 23046, 6279, 776, 6278, 777, 14872, 14873, 23323, 24604, 14878, 19118, 30767, 30768, 12214, 18119, 17736, 10447, 10064, 25936, 10066, 10067, 10068, 10065, 10070, 10071, 10069, 33763, 33764, 16997, 33768, 9194, 33515, 20717, 17518, 2798, 13298, 14456, 5115, 5116, 5118}), 12: frozenset({30080, 30081, 3335, 13181, 12175, 21904, 21905, 13182, 1300, 13183, 19226, 7335, 4140, 4141, 6960, 6961, 12465, 12464, 20660, 1461, 1462, 1463, 12466, 12467, 11764, 16320, 16323, 1607, 1608, 11768, 586, 25293, 11769, 14675, 5467, 2018, 2020, 11748, 20712, 13163, 23787, 23789, 13166, 23790, 23788, 23793, 23794, 30067, 30068, 30069, 8307, 11767, 35576, 12409, 3322, 4095, 30076, 11766, 4094, 30079}), 13: frozenset({9219, 12804, 12809, 3210, 908, 30988, 9106, 30994, 23199, 23200, 31652, 31653, 4903, 31661, 31662, 31663, 944, 31667, 37044, 39093, 37302, 37301, 7994, 14281, 4562, 4949, 21334, 4570, 24029, 24030, 24031, 37472, 37474, 16758, 23673, 16890, 23419}), 14: frozenset({12032, 37633, 37635, 4740, 3206, 12041, 20746, 7693, 1555, 18325, 533, 21783, 7706, 35741, 35742, 1951, 6430, 35743, 35746, 9124, 38437, 7716, 38442, 38444, 35759, 3504, 35760, 17714, 3379, 17716, 17715, 37690, 37691, 38331, 37692, 13118, 1985, 19140, 11083, 6477, 2126, 26193, 26194, 23659, 23660, 23661, 15596, 37103, 37104, 15597, 31615}), 15: frozenset({20193, 36130, 36131, 12293, 20070, 24658, 21618, 19474, 10716}), 16: frozenset({19073, 22658, 22659, 19075, 22660, 34824, 34825, 12298, 34826, 12301, 12302, 12303, 34831, 27025, 19476, 19478, 16412, 33183, 33184, 33185, 33186, 33187, 33188, 1575, 33193, 33194, 1577, 33197, 33199, 33200, 8497, 8502, 33207, 14263, 8503, 23995, 10684, 10685, 25149, 7883, 24527, 17745, 10718, 3043, 21619, 25083, 18940, 19071}), 17: frozenset({2629, 12124, 28112, 859, 15260, 15261, 15262, 16543, 15264, 15265, 25762, 25763, 15266, 25764, 23458, 15270, 15271, 25128, 15267, 13482, 15268, 15276, 25389, 8423, 13416, 13620, 13621, 12092}), 18: frozenset({18146, 3203, 103, 24903, 31593, 33258, 10221, 17694, 1807, 6035, 17525, 26935, 36222, 36223}), 19: frozenset({22530, 22726, 31943, 31944, 31945, 31946, 34956, 17903, 7798, 28791}), 20: frozenset({21761, 21762, 17352, 17355, 17357, 17360, 11857, 11859, 31574, 32343, 31575, 37530, 15645, 6945, 6946, 6947, 6948, 31588, 31589, 37543, 20778, 17835, 20779, 39151, 39152, 2164, 21878, 21881, 6460, 6463}), 21: frozenset({16526, 35025, 10130, 26966, 6046, 20134, 25767, 25768, 22699, 12085, 5750, 5751, 5753, 34490, 5755, 33276, 12095}), 22: frozenset({16384, 24968, 36108, 36109, 16398, 19471, 16400, 36113, 16399, 25103, 36111, 36112, 35478, 35479, 35480, 18713, 36118, 36119, 668, 669, 674, 19470, 17064, 19390, 25922, 3528, 10569, 4810, 4811, 8908, 20308, 8791, 35812, 20069, 35813, 35815, 10599, 20068, 10859, 1644, 5103, 5104, 20082, 16382, 16383}), 23: frozenset({26625, 29955, 26501, 6550, 6551, 7840, 37027, 37028, 37030, 13695, 6444, 2995, 24373, 24374, 24378, 35643, 27068, 7104, 13508, 13509, 13510, 13511, 13513, 25434, 35941, 35942, 24935, 30952, 30953, 30954, 30955, 30956, 30957, 23536, 1906, 12146, 25971, 3068, 13694, 26623}), 24: frozenset({11392, 33666, 18885, 30474, 8143, 25423, 27665, 10325, 11414, 726, 10326, 10331, 34973, 161, 427, 6194, 6195, 17907, 19708}), 25: frozenset({24129, 19971, 24133, 24134, 24135, 24136, 13193, 24137, 1355, 25932, 1356, 23823, 23824, 23830, 23831, 23832, 23833, 8857, 1368, 16345, 16346, 16610, 8314, 8822, 5626, 4283}), 26: frozenset({23107, 23109, 30472, 34376, 10321, 24210, 22550, 34966, 8683, 6191, 6192, 7154, 3130, 33661, 33662, 31871}), 27: frozenset({16394, 19468, 8215, 17440, 18722, 17446, 17447, 17452, 17453, 17454, 1332, 35893, 17462, 35894, 35896, 17463, 17464, 17467, 17468, 17469, 17466, 20287, 17465, 17472, 20289, 17474, 17475, 1335, 17480, 17481, 20298, 17483, 17482, 17484, 1615, 1616, 1617, 11872, 2360, 10858, 3196, 8322, 8324, 8325, 8326, 24967, 648, 649, 650, 4492, 4749, 8876, 8877, 8879, 28079, 8882, 8883, 8884, 3525, 4806, 4807, 4805, 5061, 19402, 2000, 16355, 16357, 16358}), 28: frozenset({34305, 34306, 33530, 18120, 30792, 25994, 27530, 27531, 17933, 2379, 23371, 22990, 2001, 6228, 6229, 23326, 17123, 17125, 15398, 21298, 25138, 10098, 38131, 10099, 29816, 34298}), 29: frozenset({8828, 30083, 30084, 35567, 3336, 18443, 1041, 30617, 28442, 8221, 5032, 26921, 8618, 23092, 12468, 17207, 17208, 17209, 11765, 14789, 27206, 19399, 9170, 14677, 14678, 14679, 14680, 11737, 11739, 11740, 11741, 11742, 11743, 13171, 11747, 20711, 2025, 2026, 20713, 11754, 2029, 11755, 8303, 11756, 11761, 11762, 11760, 11757, 2028, 11763, 13175, 13167, 8305, 10865, 13179, 13180, 18421, 30070}), 30: frozenset({26883, 16010, 19854, 19855, 19856, 31002, 31003, 7195, 12061, 13598, 13471, 31008, 31009, 13597, 25123, 13603, 23461, 12062, 31007, 23460, 13600, 23466, 7858, 5684, 5685, 5686, 5687, 5688, 5689, 5690, 26557, 26558, 5695, 5697, 5698, 5699, 3396, 3397, 2630, 3398, 3401, 31004, 30550, 4952, 13407, 14176, 13409, 37344, 3298, 31006, 13413, 13414, 13415, 16503, 16504, 16505}), 31: frozenset({26113, 26114, 26115, 38531, 1679, 14735, 16017, 14736, 6546, 9490, 21009, 21010, 9086, 37272, 24345, 37273, 37274, 24347, 13606, 13607, 16552, 8755, 10806, 6398, 32328, 17865, 14792, 23883, 7248, 19964, 17246, 32609, 37097, 31732, 14199, 17148, 37757, 26110}), 32: frozenset({18947, 6598, 19208, 4682, 203, 8014, 18295, 3539, 3540, 3541, 18296, 21788, 3165, 20768, 18529, 18530, 18531, 870, 19431, 2473, 3442, 4984, 9397, 23734, 18294, 23736, 2171}), 33: frozenset({6530, 34435, 16388, 24965, 19462, 19463, 8328, 16389, 17797, 34436, 19084, 653, 19085, 655, 657, 26386, 2710, 26395, 26396, 26397, 26408, 26410, 26411, 26412, 20664, 3518, 8894, 3520, 8897, 34437, 10703, 14160, 34438, 1622, 3039, 8801, 2662, 25066, 25068, 12280, 12281}), 34: frozenset({16975, 34706, 34707, 2901, 34711, 23321, 13339, 30621, 9565, 17121, 29604, 23334, 5862, 30763, 15405, 15406, 33711, 33712, 2815}), 35: frozenset({33664, 33665, 28802, 28803, 28804, 33157, 33158, 25991, 30465, 24201, 24200, 31874, 11397, 11398, 11411, 11412, 11413, 14103, 34971, 412, 34972, 34974, 25254, 10872, 15924, 3124, 28090, 15550, 15551, 24640, 11972, 15556, 21702, 21707, 8268, 25422, 19707, 22737, 22738, 22739, 22740, 20437, 22741, 19709, 11390, 23259, 16476, 28254, 23271, 11396, 9199, 31875, 9336, 10873, 33147, 11388, 11389, 19710, 11391}), 36: frozenset({24608, 3266, 32739, 32740, 5122, 9694, 12843, 12844, 22993, 20244, 21621, 33528, 24606, 9695}), 37: frozenset({15170, 11845, 2631, 9099, 3409, 13412, 31013, 37286, 31014, 2536, 23465, 5673, 2537, 19948, 8751, 3257, 954, 15164, 15165}), 38: frozenset({3842, 13961, 17806, 19087, 19088, 34449, 5138, 34451, 34452, 34837, 34455, 34456, 34457, 19105, 19753, 16313, 16314, 16315, 23614, 23615, 1224, 5451, 21837, 21838, 17999, 10706, 10707, 10708, 24533, 10729, 16238, 27638, 27639, 3836}), 39: frozenset({28097, 17299, 13718, 13530, 38300, 38301, 19937, 14181, 23400, 7593, 25898, 13801, 4396, 19944, 38387, 38388, 38389, 23222, 24950}), 40: frozenset({24577, 9222, 10255, 10256, 24336, 24335, 8406, 24343, 6042, 15783, 23976, 18859, 7472, 12530, 10234, 10235, 6397}), 41: frozenset({28096, 16894, 24330, 13580, 13581, 28109, 4558, 1912, 7191, 32289, 36898, 18849, 36901, 13543, 36587, 7852, 13548, 13549, 36589, 32304, 12782, 36590, 36592, 36593, 36591, 24946, 36596, 25976, 18161, 17982}), 42: frozenset({21568, 27265, 8899, 20295, 16776, 5069, 18702, 18703, 11871, 5088, 36082, 36083, 26420, 26421, 36088, 36089, 26426, 26427, 26428}), 43: frozenset({7879, 16200, 5447, 9420, 22423, 22424, 19873, 19874, 32934, 3879, 22440, 32935, 22438, 22439, 13945, 20478}), 44: frozenset({6919, 16269, 34445, 34446, 24978, 2139, 19099, 34469, 34470, 9578, 298, 16750, 6895, 6896, 6894, 27630, 27641}), 45: frozenset({31872, 11808, 14082, 17508, 22566, 22567, 5179, 426, 29167, 10323, 25235, 2582, 10361, 9563}), 46: frozenset({7174, 21767, 23564, 23565, 2319, 2320, 7963, 7964, 4381, 4382, 26910, 26920, 24617, 17196, 17197, 7229, 25406, 8515, 38470, 12374, 12376, 12379, 7271, 7272, 13939, 4469, 36993, 36994, 13445, 13446, 14241, 14242, 14244, 14245, 19373, 17326, 17327, 17328, 17329, 17330, 17331, 17844, 17333, 37557, 17846, 17332, 17338, 17339, 17340, 37563, 11450, 37564, 9147, 17341, 37570, 37571, 37572, 37573, 30915, 17342, 16851, 16852, 16853, 11493}), 47: frozenset({26752, 26753, 4100, 4101, 4102, 19975, 4103, 26761, 4105, 4104, 26768, 19987, 12436, 12438, 19999, 20275, 19386, 2005, 25563, 25565, 5607, 27242, 27243, 27244, 14445, 5614, 27247, 28016, 28017, 16624, 18033, 28015, 16373, 5621, 18034, 25725, 26750, 26751}), 48: frozenset({2119, 18248, 5770, 5771, 5772, 5773, 5774, 5775, 5776, 10827, 9504, 8170, 20907, 25772, 8430, 37167, 37168, 7600, 1202, 33010, 30900, 30901, 22461}), 49: frozenset({1864, 20173, 18959, 29392, 29393, 2848, 29413, 21350, 29799, 29415, 29417, 29418, 29416, 29421, 23159, 6718}), 50: frozenset({832, 4675, 843, 909, 17174, 17175, 36567, 4568, 4569, 24028, 93, 36576, 1504, 23202, 2094, 26480, 26481, 18175, 37046, 39096, 39097, 39098, 23676, 37437, 11455}), 51: frozenset({25792, 20995, 11919, 8336, 10006, 854, 24795, 1180, 1181, 1182, 8352, 13732, 29860, 37157, 19238, 26024, 29865, 12135, 37164, 18862, 25779, 25396, 18872, 8761, 22458, 22460}), 52: frozenset({3844, 6917, 6797, 16270, 3855, 6800, 6801, 6805, 34844, 8605, 34462, 34468, 19750, 34857, 34858, 34859, 34860, 3889, 4018, 3890, 18621, 18622, 28742, 28743, 28744, 10711, 2650, 13147, 13149, 8030, 27615, 27616, 27617, 27618, 27619, 10720, 26472, 36457, 18282, 18283, 18284, 2159, 6898}), 53: frozenset({16321, 16322, 12426, 25485, 18642, 18643, 25492, 30075, 15519, 20000, 3645, 2019, 28199, 8299, 13164, 2283, 4142, 4092, 9201, 8306, 5491, 4093, 12413, 3643, 12412, 30077}), 54: frozenset({11393, 31873, 17149, 33162, 31757, 26003, 17151, 2583, 11415, 11417, 11418, 11416, 2085, 2740, 2741, 20538, 31802, 36798, 36799, 28356, 15557, 18886, 18891, 2381, 79, 80, 83, 17235, 16982, 10327, 10328, 2532, 26220, 23151, 30832, 29169, 29170, 29171, 29172, 29173, 10364, 10365, 10367}), 55: frozenset({21120, 37506, 25861, 38547, 6547, 38548, 7190, 38549, 11288, 8730, 11290, 32282, 32284, 32283, 32287, 36895, 36897, 9503, 24355, 24356, 24357, 24358, 24359, 24360, 5280, 36896, 4399, 4911, 10817, 16582, 16583, 17864, 4553, 17866, 17867, 17747, 24919, 2775, 21494, 1911}), 56: frozenset({26627, 10511, 2832, 2831, 21017, 7201, 35623, 2864, 2865, 35636, 35637, 35638, 27070, 27071, 38465, 38466, 20037, 8773, 38348, 38350, 13783, 13784, 1114, 23388, 14300, 35939, 35940, 23398, 35943, 35944, 18793, 14314, 21364, 23932, 13693}), 57: frozenset({25474, 7301, 24709, 4754, 2707, 4501, 7333, 683, 684, 35886, 35887, 687, 1326, 2354, 2355, 2359, 11704, 11708, 11715, 1988, 35783, 35784, 35787, 35788, 35789, 1358, 35791, 3668, 3669, 11989, 3671, 3672, 3670, 1370, 16347, 1371, 7005, 7006, 3678, 3679, 26209, 1372, 8800, 18020, 10592, 35809, 14439, 18024, 5482, 2286, 6767, 16377, 3194}), 58: frozenset({10368, 10369, 11778, 31876, 25351, 17159, 5256, 4876, 1676, 15502, 27663, 27666, 33172, 33684, 2584, 1820, 162, 26019, 2086, 14377, 26027, 26028, 23855, 23857, 5046, 6198, 23866, 14462, 10366, 20414, 36800, 23492, 1486, 1487, 26449, 1490, 26450, 84, 10329, 10330, 18138, 10332, 6369, 20456, 23273, 34410, 34411, 14824, 16109, 34926, 23272, 34928, 14459, 24572, 17150, 19711}), 59: frozenset({29317, 29318, 1677, 21005, 25873, 7581, 7582, 25890, 15781, 16037, 37799, 38571, 38572, 15149, 26030, 23471, 15541, 15542, 15543, 15544, 15545, 15925, 14010, 12220, 13757, 12222, 13755, 37184, 8769, 37186, 37187, 37188, 13756, 2122, 77, 78, 22742, 37078, 37080, 37081, 37082, 37083, 20457, 38516, 3957, 38517, 38518, 6392, 18938, 14843}), 60: frozenset({26048, 35147, 8716, 35148, 35150, 14928, 38613, 22235, 22236, 8097, 20322, 28965, 10150, 10151, 28966, 28524, 28525, 36399, 24752, 19249, 1715}), 61: frozenset({11072, 20484, 13959, 18375, 18376, 19098, 19164, 19101, 27613, 27614, 13150, 10719, 4705, 4706, 34852, 34853, 34854, 34855, 34856, 10723, 4966, 18285, 21809, 34866, 8445, 7615}), 62: frozenset({10121, 10122, 10123, 6351, 22805, 12694, 7576, 3928, 5789, 25759, 22815, 38496, 9953, 38498, 38499, 7650, 13350, 13925, 29165, 29293, 12209, 35058, 29298, 26038, 10742, 19958, 11515}), 63: frozenset({10240, 10242, 10243, 35588, 6662, 17658, 6027, 6028, 20109, 6669, 22031, 17147, 20114, 12574, 12575, 7463, 4907, 4909, 26937, 7482, 22716, 7489, 2755, 26948, 26181, 2756, 24650, 35582, 24543, 10218, 10219, 10220, 12527, 17656, 37626, 17659, 3198, 37631}), 64: frozenset({16704, 14917, 14919, 8073, 7192, 7130, 23969, 17701, 36904, 32297, 32298, 13550, 13552, 12150, 5048, 9212}), 65: frozenset({4800, 25473, 4801, 26435, 26436, 26437, 8900, 8905, 8907, 5068, 18704, 1686, 25306, 1628, 28073, 28074, 26413, 36081, 28082, 26422, 16823, 26425, 15803, 16061}), 66: frozenset({25636, 11781, 8678, 25639, 2823, 24873, 22534, 28267, 23341, 27726, 28883, 33715, 34998}), 67: frozenset({21889, 8260, 33094, 9416, 15433, 15432, 21516, 6413, 1941, 21279, 14568, 14569, 13295, 8690, 15283, 15285, 25661}), 68: frozenset({27497, 20941, 10126, 35024, 35026, 1810, 21268, 31259, 23132, 31261}), 69: frozenset({1290, 3594, 7694, 7695, 3598, 25552, 25553, 19223, 23066, 16224, 16226, 35751, 35752, 35753, 37673, 4010, 15591, 35758, 36464, 19377, 36664, 36665, 36666, 36667, 17086}), 70: frozenset({5130, 3217, 8601, 8602, 8008, 202, 8010, 8011, 24267, 24398, 24399, 20815, 5455, 26702, 8019, 8018, 24405, 24407, 19424, 24418, 24419, 24420, 9442, 12260, 12259, 9577, 19434, 24427, 24429, 24430, 18293, 13950}), 71: frozenset({24386, 24388, 24389, 12231, 1096, 37451, 13453, 13454, 10836, 3416, 38429, 549, 18855, 15466, 111, 112, 2100, 38581, 38583, 3768, 2621, 24383}), 72: frozenset({11394, 11395, 10759, 11399, 30473, 33163, 11020, 33164, 24592, 35474, 17914, 33685, 160, 23849, 23851, 25648, 6193, 5177, 36797, 27713, 15558, 10324, 33622, 17239, 9566, 6368, 9569, 5348, 34924, 34925, 34927, 29168, 16502, 10362, 10363, 17790}), 73: frozenset({1920, 4035, 2372, 11091, 21853, 11494, 2153, 6827, 26286, 9135, 4980, 18613, 3575, 3576, 3577, 4026, 26299}), 74: frozenset({16129, 31751, 3349, 16150, 16151, 19993, 18202, 36124, 36125, 19232, 8234, 35885, 37693, 37694, 37695, 1345, 1346, 19793, 5468, 1632, 1377, 1378, 1379, 1380, 1381, 1638, 35431, 35432, 18026, 3691, 3692, 20075, 35436, 1390, 6768, 3700, 3701, 10612, 28023, 36728, 11897, 20090, 36738, 25475, 21125, 21908, 21909, 21910, 4759, 4760, 4761, 25503, 4517, 24997, 679, 37801, 21948, 8640, 7365, 7367, 21963, 31707, 31708, 31711, 31712, 31713, 31714, 23011, 23012, 23013, 23014, 31716, 31717, 2290, 16628, 16630, 18680}), 75: frozenset({24832, 33282, 2570, 29836, 29837, 11537, 1816, 6049, 30461, 22573, 31278, 33071, 18733, 33072, 25650, 22574, 18100, 16962, 5954, 5956, 5957, 5958, 26054, 34893, 26962, 22613, 22614, 22616, 34271, 34272, 23135, 20590, 34287, 32378, 32379, 33277, 33278, 18815}), 76: frozenset({32288, 37129, 24362, 24363, 17868, 24364, 4554, 4913, 38550, 17977, 23418, 26875, 32286}), 77: frozenset({27073, 38467, 27076, 13701, 37509, 38469, 8774, 38468, 970, 13516, 2829, 10510, 38352, 13785, 21018, 13786, 14313, 2862, 35631, 2863, 35633, 35634, 35635, 27062, 13686, 23929, 14202, 7100, 7101, 13687}), 78: frozenset({19072, 25476, 2692, 22661, 4870, 28168, 25482, 16781, 34833, 1688, 33181, 4771, 4773, 1449, 1450, 1451, 2348, 23991, 23996, 21572, 7366, 37708, 37713, 37714, 19796, 37718, 4056, 4057, 6750, 6769, 16121, 7035}), 79: frozenset({7171, 26438, 1800, 26025, 33289, 24653, 17485, 17487, 34899, 36148, 17110, 33241, 1788, 33246}), 80: frozenset({9412, 3205, 1157, 6406, 32331, 32332, 32333, 1934, 957, 1104, 17634, 4387, 21859, 17638, 17639, 25960, 21865, 16556, 38449, 17650, 17651, 17652, 17653, 13494, 8178, 6459, 2109, 38334}), 81: frozenset({34497, 34498, 31299, 31300, 31301, 34499, 22792, 20937, 2576, 14811, 23966, 29155, 29156, 23149, 37618, 5043, 37621, 31733, 6205}), 82: frozenset({10177, 14149, 27145, 19151, 3984, 3985, 21778, 5139, 2167, 19352, 28703, 28704, 28707, 26083, 8997, 21799, 18472, 18471, 14572, 14205, 2172, 7155, 23735, 1207, 11068, 10173, 6527}), 83: frozenset({1409, 9603, 18956, 27277, 13968, 5524, 14490, 14491, 8986, 1405, 18209, 808, 27565, 27566, 27567, 21425, 25009, 27573, 27574, 31543, 31544, 31545, 31546, 27575, 21441, 21058, 10178, 10180, 10179, 10181, 6482, 6483, 8793, 2402, 6499, 34788, 7146, 20222, 28669, 5492, 5493, 7156, 5495, 28664, 28666, 28667, 28668, 13309, 1406}), 84: frozenset({15236, 29818, 27015, 21511, 4617, 4616, 29819, 27135, 22418, 11094, 2647, 20504, 18271, 12321, 2157, 10110, 9139, 12341, 15349, 15353, 15354, 187, 29820, 29821, 2811, 15679}), 85: frozenset({8979, 8212, 5148, 21534, 1313, 1314, 1315, 804, 1316, 10534, 3883, 21566, 2367, 10563, 21325, 21326, 8015, 11598, 8017, 11600, 11603, 8016, 11602, 11604, 5457, 11605, 34393, 34394, 3163, 34396, 34397, 34395, 11606, 11607, 11612, 14690, 2659, 14691, 5479, 3702, 1411, 27021, 27022, 5522, 5523, 23191, 5529, 28057, 27298, 9126, 14274, 14275, 24262, 24263, 24264, 7629, 27598, 27599, 27601, 27602, 27603, 27604, 27605, 5336, 5337, 3544, 3545, 26075, 20189, 3036, 12258, 999, 4840, 19432, 10474, 19433, 19435, 12268, 12267, 12270, 7148, 7149, 15851, 11599, 8441, 19450}), 86: frozenset({37440, 17859, 838, 24327, 24328, 24329, 20039, 17748, 27092, 17622, 38295, 17303, 13719, 25431, 13531, 13532, 17630, 38303, 38304, 38305, 38306, 21028, 7082, 4915, 18996, 18997, 12148, 23223, 38390}), 87: frozenset({11841, 30978, 5700, 5701, 1092, 3400, 14545, 539, 37276, 16096, 39076, 39078, 39079, 16506, 5691, 5692, 5693, 5694}), 88: frozenset({36928, 36929, 36930, 36931, 26244, 26245, 26246, 36932, 6725, 21320, 2315, 14411, 29388, 26254, 8079, 29389, 29390, 18900, 22037, 22038, 20184, 1882, 1883, 38877, 26855, 26856, 23226, 6715, 36927}), 89: frozenset({26754, 8291, 26755, 19974, 26759, 26760, 5608, 3656, 26765, 12435, 14421, 14426, 14427, 8636, 25567}), 90: frozenset({31432, 10249, 26446, 26447, 1489, 6036, 6037, 25755, 13922, 21734, 5678, 7471, 8753, 10229, 29302, 36149, 9982}), 91: frozenset({8065, 8066, 3721, 10898, 8984, 1304, 8985, 18206, 7330, 12451, 12452, 12453, 807, 3625, 20268, 3629, 6967, 6968, 6971, 574, 20675, 20676, 6983, 24008, 9159, 16329, 25931, 972, 24010, 16330, 8655, 27218, 6995, 6996, 27221, 26078, 21983, 14431, 5604, 34789, 12390, 3303, 4329, 5616, 27250, 28659, 28660, 28661, 28662, 27251, 5624, 28665, 8827}), 92: frozenset({22403, 22404, 13636, 9415, 6411, 22413, 22414, 15183, 20756, 16024, 1177, 1176, 5725, 32350, 15647, 20896, 32351, 26462, 14561, 12069, 25962, 8427, 52, 53, 54, 55, 16509, 16510, 22399}), 93: frozenset({9226, 9227, 9229, 3215, 3216, 3093, 21526, 20511, 26913, 18299, 18300, 35498, 9132, 18301, 21045, 4024, 4025, 26682, 14143, 8009, 3022, 20816, 20817, 20818, 20820, 24406, 24408, 24409, 24410, 24411, 3547, 24413, 24414, 24415, 24416, 24417, 24421, 24422, 18535, 18536, 24425, 24426, 3179, 24428, 24424, 3567, 3439, 4977, 25072, 3440, 14580, 3568, 14579, 18292, 3448, 3449, 3450, 20212, 14581, 14582}), 94: frozenset({30976, 30977, 16893, 27139, 27140, 27141, 27142, 27143, 4619, 21259, 21260, 5137, 9492, 12052, 12057, 1064, 1065, 1066, 7980, 7981, 7982, 7984, 6452, 6453, 7223, 12344, 12345, 11063, 7231, 15680, 21827, 21831, 23369, 5449, 16246, 2130, 2131, 15186, 4946, 15192, 18012, 15196, 15197, 15198, 15199, 26465, 18273, 22370, 15202, 10855, 26223, 3186, 27506, 27507, 27508, 27510, 16756, 2680, 27509, 27511, 27512, 16247, 16248, 27515, 27513, 27514, 27516, 28296, 1928, 19849, 15499, 19857, 18343, 9140, 188, 189, 3009, 7620, 3781, 7622, 14537, 14542, 14543, 13264, 6866, 14546, 19157, 36578, 36584, 20203, 7670, 30971, 6909}), 95: frozenset({21763, 6412, 8474, 8477, 15649, 8481, 8483, 8482, 553, 23349, 33085, 33086, 12094, 6465, 6466, 15431, 1097, 17737, 13644, 13645, 13647, 13648, 2642, 11861, 32354, 32358, 32360, 32361, 25963, 21890, 21891, 30603, 8337, 8338, 11943, 15284, 25781, 25782, 25783, 25784, 32954, 32957, 32958, 3006, 26047, 32960, 32961, 32962, 32959, 22470, 22471, 6855, 22472, 32970, 32968, 24777, 24781, 24782, 32974, 24783, 24778, 24779, 14565, 14570, 14571, 13296, 39165, 39167}), 96: frozenset({19089, 34450, 10709, 10710, 34458, 34459, 8604, 34460, 34461, 34463, 23076, 3176, 3837, 16237, 12334, 17071, 23983, 12335, 4017, 23982, 6897, 21617, 16118, 23613}), 97: frozenset({30595, 23944, 23945, 23946, 28301, 1943, 6813, 20772, 6821, 19366, 19365, 29747, 29748, 18612, 4022, 4029, 4030, 11459, 3909, 26697, 11594, 3021, 11597, 26701, 21847, 5335, 2653, 3170, 18533, 3564, 3565, 16240, 14578, 115, 116, 2678}), 98: frozenset({24034, 24035, 23204, 26179, 23203, 37305, 525, 4563, 9107, 4565, 37303, 21336, 39095, 35706, 39131}), 99: frozenset({16259, 16260, 16262, 3851, 18317, 27661, 18322, 18323, 5144, 5146, 28708, 18350, 18351, 1584, 3886, 9408, 9409, 21834, 21835, 21452, 21453, 21454, 16208, 3157, 5988, 5989, 10857, 5993, 5994, 5996, 22637, 5997, 3823, 22639, 22638, 5998, 22644, 22652, 22653, 22654, 21759}), 100: frozenset({3584, 11942, 4041, 781, 9136, 15376, 15377, 22548, 10102, 4982, 10104, 30777, 2810}), 101: frozenset({7553, 10502, 36487, 17290, 17291, 17292, 17293, 23182, 26260, 26262, 9496, 14747, 14748, 14749, 2846, 14750, 14752, 14753, 6559, 2847, 14751, 1960, 35250, 26675, 35252, 23220, 35253, 7095, 35255, 35257, 7096, 35254, 35251, 35256, 23230, 11327, 11328, 6719, 26674, 11331, 11333, 11335, 11336, 1865, 1866, 1867, 11338, 1869, 20174, 20175, 26840, 14306, 21346, 29422, 29424, 29425, 31858, 31859, 29426, 29429, 29430, 29431, 23161}), 102: frozenset({26503, 29771, 26637, 26638, 1168, 27096, 6553, 27098, 27099, 21023, 38559, 38371, 38373, 38374, 15527, 35561, 8426, 25965, 754, 18870, 13502}), 103: frozenset({13192, 12168, 12170, 651, 652, 26380, 26379, 25487, 26378, 28049, 28050, 8211, 9236, 28051, 3221, 8855, 8856, 1683, 1306, 1307, 23836, 1308, 23838, 23839, 23840, 23841, 1311, 23837, 28068, 4135, 19381, 8886, 8887, 8888, 20286, 21569, 17476, 17477, 17478, 5063, 5064, 5065, 3658, 3659, 16334, 14422, 26712, 26713, 17244, 11870, 9185, 9186, 2039, 3312, 27252, 2037, 8311, 8312}), 104: frozenset({14468, 19879, 12077, 19760, 27124, 17237, 27125, 17495, 32920, 32921, 32923, 32924, 32925, 32926}), 105: frozenset({6850, 15171, 15172, 20741, 15173, 15174, 15175, 6851, 32910, 21071, 18576, 32913, 32914, 9428, 1047, 37339, 19876, 25125, 4645, 4647, 14564, 23919, 2166, 27193, 26555, 3004}), 106: frozenset({9538, 7500, 35853, 7244, 22677, 22678, 11422, 11423, 11424, 16479, 7076, 7077, 14695, 4910, 34480, 37619, 37620, 37622, 6010, 6011}), 107: frozenset({14529, 3593, 1291, 33234, 21781, 25113, 16223, 4009, 15594, 15595, 16300, 35754, 35755, 37682, 21746, 37684, 14134, 20474}), 108: frozenset({19970, 19980, 19988, 8213, 28187, 8224, 1312, 21296, 28209, 6970, 25147, 24641, 1351, 6984, 6985, 25929, 3660, 21076, 6997, 6754, 14438, 8807, 14447, 13303, 8057, 3195, 3197, 25471, 25733, 4487, 2699, 28055, 28056, 12442, 12443, 27553, 25002, 21420, 21421, 28349, 20674, 24009, 24014, 16335, 16336, 8656, 26068, 26069, 16343, 21984, 21985, 25314, 34791, 34792, 34793, 34794, 3051, 34796, 1005, 34797, 34798, 34795, 5618, 34804, 13302, 34807, 34808, 34806, 13306}), 109: frozenset({23558, 23559, 17295, 17296, 17297, 7571, 13716, 13717, 17302, 17300, 13720, 17817, 38298, 25884, 3745, 21027, 4390, 15662, 16561, 16562, 5426, 2998, 19003, 19004, 25404, 14846, 3271, 31560, 3273, 31562, 20811, 31563, 31565, 31566, 37583, 31564, 31561, 31573, 27094, 13529, 37595, 13533, 1117, 38369, 38370, 21347, 39146, 13803, 18924, 13932, 25970, 14836, 38391, 10493, 8190}), 110: frozenset({9472, 35601, 9489, 18965, 31765, 18967, 21016, 21013, 35607, 25880, 35608, 35609, 35611, 35610, 7200, 35612, 20253, 35614, 26911, 35613, 35619, 6436, 35620, 35621, 35622, 2855, 35624, 35625, 2857, 25894, 2860, 2861, 6443, 35642, 7227, 38461, 38462, 38463, 38464, 13390, 861, 1890, 24936, 12141, 23927, 36473, 36986, 36987, 36988, 36989, 36990, 13690, 13691, 13692, 13442, 13696, 7565, 17815, 7587, 7588, 16041, 16042, 2985, 4533, 27065, 27066, 27067, 27069, 13507, 198, 13512, 2761, 38344, 38345, 38346, 38347, 38349, 13780, 13781, 6879, 14310, 14312, 750, 8181, 8186}), 111: frozenset({18308, 30598, 19847, 17807, 12053, 12316, 8094, 12322, 15395, 15396, 15652, 5158, 1058, 18341, 18339, 18340, 17068, 12338, 12342, 12343, 12216, 26556, 26559, 3008, 21826, 28360, 23625, 25420, 25421, 9421, 6863, 6865, 15185, 11095, 37341, 37342, 18272, 15201, 2401, 15203, 15200, 15204, 22379, 22380, 22381, 22382, 27129, 20479}), 112: frozenset({3200, 17665, 17666, 17667, 17668, 17671, 17672, 17673, 17674, 38801, 38802, 38803, 36908, 36909, 13553, 13554, 28277, 20471, 21496, 17655}), 113: frozenset({24961, 8005, 10699, 21451, 34424, 30606, 23060, 21333, 34399, 34400, 5985, 17059, 3496, 3497, 15659, 18352, 23665, 23670, 13111, 13112, 34425, 34426}), 114: frozenset({6532, 5126, 8589, 8591, 8592, 3222, 9238, 35356, 35357, 35358, 35359, 3233, 3234, 27171, 27172, 9254, 9256, 3884, 18607, 18608, 18609, 21041, 30009, 30010, 30011, 30012, 30013, 30014, 30015, 30024, 30025, 7631, 5459, 8023, 14170, 19418, 7294, 26718, 3555, 26720, 26721, 26722, 26723, 3552, 3553, 3554, 20834, 26212, 26725, 26726, 26727, 8551, 26724, 20833, 20210, 24691, 2419, 24692, 12275, 12276, 25078, 7292, 4990}), 115: frozenset({30085, 25477, 30086, 30088, 30089, 30090, 12427, 30092, 30093, 30094, 30095, 30096, 30097, 12428, 12429, 12430, 25493, 30098, 30103, 4891, 8609, 301, 21942, 3766, 19384, 19771, 19774, 3646, 3647, 3648, 3650, 3651, 16324, 16071, 30087, 18644, 18645, 18646, 18647, 27236, 2021, 16614, 2022, 18408, 16613, 11882, 14699, 2284, 30091, 8301, 13168, 13169, 13170, 16625, 18420, 11892, 18035, 30071, 30072, 30073, 25724}), 116: frozenset({18177, 9223, 20488, 19210, 17167, 21519, 529, 541, 26915, 26917, 1069, 1583, 2097, 7987, 7988, 7989, 7990, 56, 2617, 11065, 31807, 24387, 1093, 35668, 35669, 2132, 35672, 3184, 2681, 1914, 16250, 16251, 16252, 903, 18314, 9103, 919, 37289, 37291, 23981, 3763, 39094, 37306, 37052, 3010, 7623, 37326, 4564, 4566, 4567, 36568, 14550, 36570, 36571, 36572, 36573, 14551, 36569, 39132, 39133, 39134, 1503, 24032, 36574, 4581, 4582, 4583, 37609, 37610, 37611, 20204, 35566, 16884, 30454, 16891, 14552}), 117: frozenset({8192, 3204, 22023, 39192, 8731, 11294, 36899, 17702, 17705, 17706, 29355, 17978, 20028, 16701, 16702, 16703, 28095, 28101, 28102, 4555, 4556, 3288, 13544, 36585, 36586, 106, 31595, 13545, 26479, 36597, 21495, 16895}), 118: frozenset({24195, 39044, 39046, 16584, 17742, 10831, 14034, 1491, 5208, 7577, 2979, 3748, 13667, 13539, 13540, 38502, 13738, 13739, 13740, 11052, 3949, 35504, 13424, 4465, 13425, 25975, 8191}), 119: frozenset({25026, 12680, 12681, 29834, 30280, 29196, 29197, 36365, 32016, 9042, 18898, 25687, 30554, 10145, 10147, 31339, 19314, 39347, 8121}), 120: frozenset({26052, 39045, 39047, 25887, 13669, 13541, 13542, 12136, 19882, 13422, 13423, 12080, 8431, 6450, 33139, 37172, 12149, 5939}), 121: frozenset({19200, 39170, 39171, 39172, 6407, 23560, 23561, 23563, 26647, 20249, 20508, 15644, 6943, 6944, 37436, 19005, 6462, 19006, 31559, 31567, 32335, 29777, 31569, 31571, 31572, 32341, 31568, 31570, 11858, 31577, 12130, 31587, 25961, 21869, 21872, 21873, 21874, 21876, 21877, 26486, 26487, 26506, 26507, 1169, 1170, 1171, 16020, 38560, 20657, 20658, 1974, 13499, 17853, 958, 17854, 17858, 3272, 7119, 17362, 16856, 30424, 27097, 39148, 39149, 23544}), 122: frozenset({33220, 10692, 25285, 19079, 33223, 33224, 33225, 33222, 33221, 18523, 23068, 18524, 34464, 3435, 16236, 16235, 3437, 4975, 3436, 21747, 2677}), 123: frozenset({16512, 16513, 2112, 6464, 22407, 22408, 30602, 22410, 22412, 15178, 12055, 1179, 19869, 15648, 20898, 20899, 32355, 14563, 12070, 8746, 39158, 39160, 39161, 21885, 21886}), 124: frozenset({18624, 18625, 3908, 15816, 28300, 25101, 29646, 29647, 29648, 3601, 3602, 29649, 29650, 29651, 8606, 8607, 34466, 34467, 24684, 24685, 3438, 9588, 3830, 16573}), 125: frozenset({12481, 258, 15819, 8788, 10587, 11997, 3689, 3690, 35448, 10542, 11695, 2289, 35443, 18678, 8824, 21946, 35771, 37500, 37501, 37502}), 126: frozenset({1412, 2310, 5521, 18458, 27297, 21161, 21289, 10548, 2369, 19396, 24265, 8781, 24269, 24270, 24272, 24273, 24274, 24275, 24276, 24271, 20822, 34390, 24280, 24281, 34392, 34391, 11998, 995, 3306, 10475, 15852, 15853, 15854, 15856, 19441, 24435, 6517, 3703, 2297, 19452}), 127: frozenset({25478, 25479, 12176, 25494, 3105, 8611, 3107, 8869, 28201, 8619, 8620, 8621, 6957, 4144, 21555, 20277, 20278, 27196, 27197, 27198, 19773, 27200, 27201, 27202, 21575, 16073, 27210, 589, 11731, 11732, 5462, 18648, 18655, 25312, 11744, 11745, 11751, 11752, 11753, 2024, 18411, 18412, 11883, 2027, 18415, 10866, 13173, 11894, 3193}), 128: frozenset({6786, 30982, 9929, 39081, 9930, 5676, 5677, 12206, 3407, 8112, 7985, 5675, 8117, 8119, 3258}), 129: frozenset({1152, 18180, 3270, 37641, 12042, 37642, 37643, 37645, 37644, 38601, 18249, 12049, 38475, 38478, 25114, 7708, 13795, 25385, 15599, 15600, 38329, 38330, 1149, 1151}), 130: frozenset({13962, 20748, 13905, 20498, 21842, 13907, 11090, 13909, 2773, 1944, 7129, 2652, 7645, 1567, 6817, 6818, 2147, 24676, 24037, 24036, 24423, 7677, 3569, 18545, 3892, 21748, 119, 16317}), 131: frozenset({27078, 27079, 38474, 7309, 12048, 25556, 25557, 25558, 38359, 38358, 25440, 13794, 7718, 7591, 15598, 5423, 2098, 17717, 17719, 17720, 31677}), 132: frozenset({19712, 19713, 19714, 36226, 9028, 28805, 28806, 37452, 37453, 23886, 22743, 17240, 23260, 34975, 18784, 15520, 30628, 38511}), 133: frozenset({15041, 30277, 28358, 30279, 23505, 32018, 32019, 12950, 2905, 37851, 30190, 2545, 30194, 2546, 39348, 39349, 1523, 4535, 6139, 9343}), 134: frozenset({34688, 34689, 34690, 34691, 34692, 34693, 34950, 34694, 34695, 25228, 32012, 15244, 15245, 27669, 27671, 27672, 33022, 33056, 33057, 34341, 30758, 30759, 30760, 18729, 18727, 11950, 34356, 23355, 8380, 34365, 12611, 18116, 25674, 11979, 9940, 2908, 5863, 9322, 30187, 30188, 6135, 6136, 6137, 33019, 34685, 34686, 34687}), 135: frozenset({11716, 20292, 24711, 10887, 24712, 1357, 19791, 11728, 19217, 2706, 11727, 3667, 1367, 16354, 24995, 3241, 3243, 5627, 11709, 11710}), 136: frozenset({3841, 8804, 22662, 24970, 34828, 34829, 4494, 24528, 24529, 18194, 4824, 59, 2715}), 137: frozenset({10432, 34948, 21580, 22807, 6680, 2074, 7773, 9950, 9951, 29216, 15905, 9952, 35048, 9963, 9965, 12081, 11955, 12084, 10743, 24504, 12606}), 138: frozenset({1795, 12547, 10250, 12554, 26954, 34488, 6038, 26968, 33240, 18137, 22694, 22695, 16177, 33267, 33140, 29301, 10230, 34487, 26936, 33141, 6013, 6014}), 139: frozenset({7457, 7460, 13958, 18281, 6795, 19116, 21742, 17809, 28754, 28755, 28759, 28760, 2649, 24539, 24540}), 140: frozenset({8973, 8975, 8976, 18454, 18455, 18456, 18457, 5147, 21533, 18214, 18470, 8999, 10537, 20270, 20272, 20273, 8780, 8013, 5456, 18003, 3162, 2657, 2658, 26993, 30583, 1407, 1408, 1410, 17795, 5004, 17813, 5526, 5527, 5528, 5525, 28060, 20638, 27559, 27561, 27562, 27563, 27564, 21423, 21426, 27576, 27580, 27581, 27582, 27583, 27584, 27585, 27587, 27588, 19395, 26828, 27597, 27596, 7628, 26067, 980, 984, 26072, 26074, 3035, 19427, 19428, 19429, 12262, 31729, 19443, 19444, 19445, 19446, 8440, 2298}), 141: frozenset({2945, 21249, 29828, 23048, 18827, 38796, 780, 5904, 9364, 22936, 795, 22940, 5917, 5918, 5919, 5920, 10148, 10149, 25638, 936, 22953, 22954, 20147, 18103, 30776, 32069, 15953, 38611, 11227, 4445, 20958, 34288, 34289, 34290, 2803, 38132, 38133, 25461, 34292, 10101, 29817, 2813}), 142: frozenset({13824, 8836, 6535, 13831, 7946, 10001, 10002, 10003, 10004, 10005, 10007, 8349, 16031, 37156, 37159, 34217, 34219, 34220, 1196, 1197, 25902, 7599, 34222, 34225, 25778, 34221, 1204, 34224, 30895, 34223, 22456, 22457, 22459, 25793, 964, 25802, 34257, 16727, 34782, 24810, 16107, 24811, 14194, 37244}), 143: frozenset({24384, 23875, 35653, 35654, 35655, 12232, 35656, 2124, 2125, 1103, 23961, 6428, 38430, 38431, 38432, 38433, 37793, 38434, 37093, 37096, 37098, 24381, 37099, 37100, 25399, 26105, 10813}), 144: frozenset({39173, 39174, 39175, 25864, 6409, 23562, 39176, 39179, 6415, 20755, 26648, 20250, 18719, 21024, 21025, 37152, 4391, 18984, 4392, 15665, 5428, 8759, 19007, 10820, 10821, 20040, 2636, 2638, 11862, 32342, 32344, 32345, 32346, 31576, 31578, 31579, 31581, 31582, 31583, 31584, 31585, 7267, 31586, 31591, 21879, 37521, 37522, 37523, 37524, 1172, 37525, 37532, 37533, 37534, 13727, 38561, 38562, 38563, 38564, 17833, 17834, 17836, 17839, 20659, 17855, 17856, 17857, 7120, 7121, 7122, 27100, 38372, 38376, 38377, 39150, 25070, 39153, 39154, 14837, 18937}), 145: frozenset({11456, 11457, 11458, 7233, 4676, 20489, 7626, 30989, 30990, 18894, 30996, 24022, 17177, 17178, 17179, 17180, 14553, 14554, 37036, 1072, 25525, 30582, 17529, 11454}), 146: frozenset({12608, 10433, 10434, 34378, 10124, 10125, 23571, 23572, 27732, 27733, 5656, 22808, 22809, 24859, 22810, 22811, 31260, 31262, 13277, 9958, 9959, 35049, 35050, 9962, 9964, 9961, 19887, 16178, 34873, 19324, 8255}), 147: frozenset({16257, 8002, 13955, 27656, 16201, 7627, 20749, 18319, 16207, 19108, 5995, 1073, 18355, 6454, 16255, 7609, 14138, 7998, 7999}), 148: frozenset({18697, 18698, 18711, 19992, 8216, 18712, 4136, 1325, 1328, 2356, 2357, 1340, 20294, 20309, 20310, 1373, 10591, 10593, 10595, 10596, 6757, 10597, 1639, 10598, 20079, 20081, 3190, 25719, 4483, 16777, 12172, 4752, 4753, 25489, 5013, 665, 4509, 28069, 28070, 28071, 28072, 685, 688, 28083, 28084, 28085, 16063, 19393, 35785, 35786, 5067, 35792, 35793, 35795, 35801, 25307, 35803, 35804, 35805, 35806, 35807, 16348, 35808, 35810, 35811, 35814, 9190, 16360, 16361, 24042, 16362, 5100, 5101, 5102, 24041, 16378, 36095}), 149: frozenset({18433, 9592, 1352, 1353, 25544, 7307, 3662, 3663, 17553, 15825, 4500, 6998, 6999, 7000, 8668, 8669, 36191, 18018, 7332, 23845, 11433, 25006, 24111, 24112, 3250, 2040, 20284}), 150: frozenset({33216, 10690, 5443, 33214, 19077, 33215, 14480, 14481, 24530, 16120, 19483, 33213, 23998, 1578, 33195, 33196, 33198, 23988, 23989, 14262, 18366, 13942, 14267, 33212, 24637, 10686, 18367}), 151: frozenset({33313, 24194, 33318, 27725, 36366, 5967, 33328, 5968, 16497, 33332, 33333, 1717, 32734}), 152: frozenset({37124, 37125, 13576, 13577, 37130, 13578, 37128, 4552, 20753, 38297, 24352, 24353, 18848, 24354, 13796, 13799, 7592, 24361, 7851, 24365, 18159, 4912, 38385}), 153: frozenset({6676, 5908, 5909, 5911, 5910, 13341, 29601, 33705, 29610, 29609, 19890, 15932, 20413, 29765, 21705, 32716, 32717, 23502, 20558, 28881, 28882, 20441, 37849, 37850, 6108, 6107, 32735, 9696, 22241, 9697, 611, 22243, 22242, 20857, 10351, 32754, 9332, 32757, 32756, 10359, 14713, 37371}), 154: frozenset({11329, 11330, 1859, 1860, 9479, 20171, 1868, 4368, 26259, 29396, 26261, 29397, 14294, 29399, 29403, 29404, 29405, 6712, 29432, 29427, 29428, 23157, 4918, 23160, 6716, 6717}), 155: frozenset({12568, 31427, 31428, 7493, 20987, 7466, 6667, 2539, 20458, 10254, 1678, 33143, 33144, 22363, 22364}), 156: frozenset({15371, 18828, 15374, 15378, 20503, 15384, 15385, 15386, 23327, 15392, 23331, 23332, 20773, 19764, 18626, 18627, 30789, 4039, 4040, 18260, 18261, 18262, 2645, 15459, 15460, 27113, 2155, 15348, 2805, 8565, 10103, 4604, 10109, 3582, 3583}), 157: frozenset({961, 2115, 13638, 10824, 26442, 13646, 39183, 17105, 8345, 30874, 30875, 24794, 27105, 16548, 34213, 29861, 34215, 34216, 34212, 34218, 34214, 12132, 25967, 18927, 15666, 32955, 32956}), 158: frozenset({23809, 23811, 13188, 23813, 23812, 23815, 23816, 23817, 13189, 23814, 25480, 13190, 1294, 21903, 1297, 1298, 3114, 12459, 6959, 1459, 1460, 16053, 8630, 5050, 5052, 5054, 21574, 1610, 1611, 17243, 6748, 2034, 2035, 8310, 23801, 23802, 23803, 25469, 23806}), 159: frozenset({16261, 21456, 22647, 36505, 7066, 22649, 18724, 2683, 3824, 1585, 1586, 22641, 22640, 22643, 22642, 6455, 6456, 22645, 22651}), 160: frozenset({27137, 27138, 32899, 18310, 18311, 16520, 17165, 24973, 18829, 19858, 18331, 18333, 12317, 19742, 15394, 15651, 25124, 18342, 32937, 32938, 32945, 27195, 21821, 9407, 23108, 21828, 21829, 5702, 7880, 5704, 5706, 5705, 17229, 18009, 18011, 17515, 3182, 13946, 27131}), 161: frozenset({16065, 36099, 16131, 36100, 36101, 37703, 36104, 36744, 20106, 37704, 25104, 18715, 36123, 1629, 23006, 1442, 31715, 1444, 35429, 10860, 23088, 7024, 28088, 1341, 1342}), 162: frozenset({12034, 12038, 1287, 1288, 1286, 28299, 21523, 25108, 23065, 4635, 21792, 4005, 4006, 4007, 4008, 16298, 16299, 3498, 36653, 36656, 37683, 565, 11061, 14520, 14521, 25150, 25284, 14279, 10696, 15817, 24522, 24523, 25549, 33237, 18518, 7639, 5980, 5981, 16222, 15587, 3428, 4973, 37489, 23666, 23667, 37492, 37493, 37491, 37490, 3832, 2426, 30588}), 163: frozenset({4992, 25547, 25548, 8594, 7701, 8600, 36642, 36643, 36645, 36646, 36647, 36648, 36649, 36650, 27179, 36652, 36651, 18291, 9402}), 164: frozenset({13714, 13715, 6549, 38294, 25883, 21022, 7841, 4385, 6448, 4400, 4914, 24371, 24370, 2997, 18998, 20041, 17616, 27089, 27088, 38483, 17620, 17621, 27093, 13527, 17623, 13528, 17629, 38365, 23390, 3053, 38384, 25973}), 165: frozenset({15488, 20227, 18948, 14152, 5070, 13904, 13906, 6931, 13908, 1558, 13910, 3227, 9244, 24675, 4388, 3172, 2406, 1642, 4652, 6831, 27184, 37487, 18544, 2161, 18546, 27003, 16318}), 166: frozenset({33280, 34886, 26969, 26044, 21275, 24860, 27737, 22491, 22492, 27738, 20446, 34339, 18084, 22499, 34876, 32370, 8439, 33272, 8697, 8698, 10172}), 167: frozenset({544, 16545, 14177, 10880, 32321, 32322, 72, 2634, 8847, 13488, 25392, 13489, 2108}), 168: frozenset({35776, 36098, 36102, 8910, 35794, 35800, 666, 35802, 1374, 20066, 8233, 682, 1641, 5098, 5099, 686, 16364, 16363, 5486, 19382}), 169: frozenset({2822, 6281, 6282, 30347, 9867, 38797, 5903, 36380, 21405, 22941, 25634, 21411, 8098, 11173, 23339, 20401, 2375, 13000, 13001, 28488, 12999, 37837, 24654, 6226, 38610, 38612, 6227, 11222, 22231, 22232, 7513, 22234, 6619, 4444, 9693, 6232, 21601, 32738, 14051, 4452, 32741, 4453, 33767, 20972, 623, 16113, 38130, 20723, 7668, 20853, 17014, 25586}), 170: frozenset({27077, 13703, 39178, 38355, 4380, 27101, 27102, 27103, 7269, 7270, 37545, 37546, 37547, 37548, 18989, 38378, 37551, 17840, 17841, 37552, 37555, 38379, 5429, 25401}), 171: frozenset({36449, 2757, 296, 7209, 22032, 31603, 12536, 12537, 17660, 17661, 5406}), 172: frozenset({29826, 32067, 30533, 10061, 10062, 33935, 10063, 10449, 33936, 469, 10072, 22937, 30558, 21602, 22950, 2797, 30772}), 173: frozenset({39104, 1029, 36614, 38407, 19211, 37260, 36624, 36625, 913, 37594, 26916, 37221, 23401, 3755, 3756, 37232, 9203, 13812, 32309, 32310, 9205, 4475, 13821, 36607}), 174: frozenset({23993, 12304, 12306, 19479, 4825, 16413, 16414, 4511, 13984, 16415, 18656, 33189, 33190, 17066, 21102, 10862, 33201, 33202, 2417, 33204, 3826, 3828, 23992, 185, 23994, 7036, 7037, 10687}), 175: frozenset({4739, 13960, 17803, 12045, 12046, 37647, 12047, 19347, 21779, 4639, 4640, 7712, 37666, 7715, 35748, 35747, 37670, 37671, 35750, 37667, 35749, 4011, 4012, 15661, 37678, 37679, 16303, 2352, 3501, 3503, 16304, 16305, 2231, 37688, 37689, 3002, 37687, 14525, 14526, 14527, 7876, 14278, 21331, 37669, 7641, 10846, 7264, 2145, 299, 16227, 37672, 20200, 15593, 23657, 23658, 16753, 3185, 2676, 27002}), 176: frozenset({27133, 6908, 16517, 24974, 783, 15383, 15447, 22425, 18010, 12319, 12320, 18336, 23328, 9509, 19368, 12339, 12340, 15351, 15350, 12215, 15352, 2806, 6907, 30780, 15357}), 177: frozenset({5824, 17089, 17090, 17091, 17092, 17093, 17094, 17095, 24840, 22733, 31954, 24211, 31955, 33663, 34968, 34969, 23258, 18075, 16104, 16815, 17906, 5814, 5815, 5822, 5823}), 178: frozenset({11712, 11713, 11714, 20291, 7302, 24710, 4744, 10889, 4748, 19790, 1359, 11729, 19990, 8214, 5083, 16349, 16350, 7007, 5089, 6756, 1327, 1331, 11705, 11706, 11707, 16060, 11711}), 179: frozenset({10114, 10120, 15241, 5777, 24789, 18136, 23578, 9114, 32995, 35044, 32997, 32998, 14819, 16680, 29297, 11505, 29299, 11316, 11317, 11318, 29300, 20981, 29370, 25788}), 180: frozenset({26562, 39050, 39051, 39052, 15151, 15152, 23856, 15154, 15153, 8116, 37268, 14458, 5946}), 181: frozenset({26240, 35841, 36936, 17289, 20170, 26253, 23167, 29394, 29395, 29398, 29400, 29401, 29402, 21353, 23156, 26235, 26237, 26238, 26239}), 182: frozenset({33101, 30548, 4951, 17500, 13408, 36322, 7205, 23462, 26469, 7206, 23463, 11948, 25390, 13623, 18936, 13625, 13626}), 183: frozenset({16834, 30470, 17226, 17227, 10316, 13325, 34958, 34959, 34960, 37902, 37906, 10317, 9683, 724, 23256, 6175, 7138, 33448, 33449, 33450, 424, 27883, 27884, 15345, 15346, 5683, 15347, 5181, 5182}), 184: frozenset({16708, 939, 35565, 37873, 12786, 26867, 37875, 12787, 37874, 39194, 10875, 39196, 24222, 17983}), 185: frozenset({28160, 26050, 12869, 34310, 34311, 12871, 778, 34315, 17932, 33612, 33614, 23313, 30745, 21469, 14114, 5924, 5927, 5928, 34665, 33386, 12848, 18107, 33531}), 186: frozenset({22529, 33285, 33286, 1799, 20119, 34717, 6050, 6054, 6055, 6056, 6057, 20136, 33067, 33068, 18088, 18087, 7496, 7497, 34894, 34896, 34898, 26963, 26971, 20449, 22508, 22509, 22510, 22511, 22512, 22513, 22515, 18804, 22519, 22520, 22521, 32380}), 187: frozenset({31424, 31425, 31426, 24644, 32644, 3924, 20884, 853, 32599, 22365, 13921, 38525, 15782, 18856, 10803, 6394, 34493}), 188: frozenset({31682, 31683, 36676, 36677, 36675, 31684, 13704, 13705, 27080, 36674, 7310, 7567, 7569, 25882, 21020, 38494, 7719, 6446, 16558, 5424, 15601, 15602, 31674, 31675, 31676}), 189: frozenset({22016, 1861, 13371, 35963, 2834, 2835, 8729, 17755, 14301, 29406, 29407, 29792, 29793, 29794, 29791, 1957, 35948, 35949, 25518, 35950, 35952, 35955, 35956, 35957, 32566, 32567, 32568, 21366, 35962, 16699}), 190: frozenset({14720, 4106, 2314, 18957, 3323, 18448, 18201, 19227, 16796, 8861, 8862, 8863, 4515, 17202, 10550, 20283, 24142, 24143, 24144, 24145, 8786, 21587, 24146, 24147, 24150, 24151, 21080, 3673, 3674, 988, 3676, 3677, 7004, 3684, 14696, 14697, 2282, 14698, 37868, 8315, 2043, 2044}), 191: frozenset({25863, 6408, 21021, 4389, 6440, 6441, 5425, 8758, 19002, 31555, 31556, 31557, 36166, 31558, 20812, 2637, 1105, 32338, 38482, 38484, 32340, 38485, 32337, 32339, 95, 96, 12127, 13931, 14190, 12145, 1164, 1165, 1166, 13713, 16021, 917, 37785, 38554, 38555, 38556, 38557, 38558, 13500, 13501, 14014, 9414, 17617, 17618, 17619, 27091, 27090, 38363, 17115, 37597, 38364, 38367, 38368, 37599, 38366, 39147, 753, 18929, 14835, 17654, 9209, 8189}), 192: frozenset({39040, 5766, 5767, 39048, 15239, 3275, 26444, 17740, 30542, 26445, 26448, 17099, 17100, 2978, 2980, 25770, 13421, 3056, 5937, 5938, 13426, 13427}), 193: frozenset({5442, 17799, 34823, 7240, 12299, 34827, 3532, 21775, 34832, 16403, 19477, 4826, 21539, 36452, 8805, 23987, 14261, 823, 28346, 9531}), 194: frozenset({31616, 31617, 31618, 31619, 31620, 31621, 31622, 6921, 6922, 1931, 18326, 23062, 20506, 35744, 35745, 4386, 25896, 16555, 38443, 38448, 3378, 3380, 3381, 15675, 38332, 38333, 13119, 13120, 20031, 9410, 17631, 17632, 25441, 17633, 12140, 17645, 23662, 17647, 23663, 17646, 36466, 9213}), 195: frozenset({28161, 29829, 29830, 2952, 2824, 30348, 8717, 5902, 33933, 33932, 33934, 33938, 5916, 30766, 30774, 12862, 32064, 32065, 32066, 30534, 12103, 35149, 28366, 28367, 14929, 14930, 14931, 14932, 21460, 470, 14935, 14934, 34777, 10073, 10074, 10076, 11220, 10075, 10450, 16099, 17124, 16101, 30566, 33769, 17005, 10094, 10095, 2802, 34291, 10100, 11511, 23288, 14933}), 196: frozenset({32320, 25672, 11855, 3419, 11420, 3420, 23589, 23590, 37177, 13671, 24491, 24492, 5037, 13487, 14191, 15156, 23478, 14009, 32319}), 197: frozenset({7559, 19337, 38538, 20752, 2836, 17560, 5401, 26265, 26267, 26268, 18717, 26269, 26270, 26272, 26273, 26274, 26271, 26282, 26283, 26264, 7087, 32561, 32562, 32564, 32565, 18230, 26266, 13367, 13368, 38841, 23225, 13365, 13366, 38838, 13369, 15802, 23232, 23233, 23234, 35905, 23236, 36933, 21317, 38599, 24916, 24917, 14295, 23385, 39006, 21351, 21355, 21356, 21358, 21359, 21360, 35958, 9078}), 198: frozenset({33092, 33093, 32902, 32903, 32904, 6854, 33095, 33098, 1048, 1049, 1051, 16095, 19875, 14567, 11946, 3187, 8691, 21052, 3005}), 199: frozenset({20677, 3020, 1942, 2168, 3561, 3562, 19499, 9131, 3563, 14575, 14576, 20787, 21044, 3443, 3444, 3445, 26488, 21819, 7870}), 200: frozenset({7459, 7461, 6920, 28747, 6796, 6802, 6803, 28756, 28757, 6804, 28758, 28762, 24541, 6911}), 201: frozenset({6528, 20491, 8974, 8977, 8978, 803, 21427, 21428, 27577, 27578, 27579, 27586, 27589, 14022, 27590, 27591, 11608, 11609, 26073, 2136, 19430, 12263, 12264, 7144, 12265, 12266, 7143, 12269, 19447, 19448, 19449, 13951}), 202: frozenset({21504, 15624, 21848, 16857, 19165, 2655, 26016, 1569, 6822, 6823, 6829, 6830, 25069, 26289, 26290, 26291, 4979, 26294, 26295, 26296, 26297, 2429}), 203: frozenset({27659, 31755, 27660, 3852, 24975, 24976, 1560, 21787, 12323, 12324, 19109, 19110, 12327, 3880, 12328, 28714, 18347, 18348, 28717, 28715, 17070, 28716, 28718, 19369, 11059, 23604, 23605, 18349, 24613, 28709, 12325, 12349, 1216, 12326, 11074, 24003, 1219, 1221, 28710, 28711, 1218, 21833, 5450, 19145, 21836, 1220, 17998, 14165, 9437, 16870, 36583, 6888, 6889, 4713, 4715, 5610, 1217, 2158, 13947}), 204: frozenset({8841, 37267, 7578, 8096, 16036, 8741, 21032, 18858, 13741, 10798, 26031, 13743, 34354, 37171, 37173, 5941, 5942, 5944, 32951, 5943, 10811, 13624, 32317, 32952, 32575, 20033, 36418, 18244, 36428, 9934, 30545, 30546, 17107, 5207, 8418, 13668, 38501, 13670, 26216, 13419, 3948, 3950, 3951, 18926, 4466, 14841, 19963}), 205: frozenset({9210, 4869, 1162, 13708, 27085, 27086, 13711, 13712, 27087, 7570, 38480, 18769, 13522, 25427, 13525, 13526, 38362, 7707, 37598, 37601, 21026, 37602, 37604, 23975, 16560, 752, 25969, 8185, 25402, 39102}), 206: frozenset({21121, 16710, 24904, 24905, 11340, 11341, 23183, 38808, 38809, 17883, 35548, 9384, 14766, 14767, 14768, 16688, 29363, 4916, 26677, 26678, 20663, 26676, 17531, 1023}), 207: frozenset({8768, 13665, 13666, 599, 13485, 17486, 36431, 3952, 32946, 32947, 1111, 855, 3417}), 208: frozenset({28928, 28794, 22727, 15563, 15566, 15568, 9334, 36250, 24221}), 209: frozenset({20944, 22818, 34882, 30457, 32363}), 210: frozenset({4738, 37634, 37639, 37640, 12043, 35852, 37646, 37648, 37649, 37650, 21012, 6429, 37662, 19361, 7714, 37665, 37668, 38438, 38439, 38440, 38441, 16553, 3502, 37680, 37681, 2230, 14524, 17990, 36683, 36684, 36685, 20056, 37102, 18928, 8177, 36465, 1150}), 211: frozenset({22433, 22434, 22435, 24548, 10251, 12051, 23863, 10232, 10233}), 212: frozenset({12697, 37978, 10428}), 213: frozenset({8197, 8198, 11915, 9996, 9997, 9998, 9999, 10000, 14871, 8350, 14881, 29863, 15791, 39216, 39217, 25904, 38319, 32312, 29368, 17984, 39107, 39108, 25799, 25800, 25801, 848, 34256, 34258, 34259, 34260, 34261, 34262, 34263, 12763, 12764, 24797, 12765, 25951, 34270, 18148, 24805, 24806, 24807, 1897, 21486, 37235, 26868, 12796, 25980}), 214: frozenset({1344, 35777, 35778, 36103, 20304, 3345, 20306, 10581, 4758, 10582, 23007, 35426, 35427, 35428, 677, 35430, 21929, 35434, 35435, 16366, 4783, 23087, 16367, 5487, 1343}), 215: frozenset({29824, 33033, 33034, 33035, 33036, 30281, 34703, 32017, 37905, 9043, 32021, 32022, 8279, 18782, 24482, 21603, 30756, 21612, 18094, 30192, 33330, 1525, 15416, 15418, 6140}), 216: frozenset({22496, 25665, 18085, 22504, 22664, 22700, 20117, 8375, 33273, 33274}), 217: frozenset({16390, 8904, 4874, 18700, 18190, 18705, 662, 20311, 1627, 13980, 35421, 20064, 23206, 5095, 9191, 21096, 28075, 36078, 36079, 36080, 3313, 27263}), 218: frozenset({34949, 8455, 9358, 9937, 9939, 27670, 9944, 9945, 9946, 9947, 9948, 9949, 29218, 19885, 19886, 12082, 12083, 19897, 12607}), 219: frozenset({8323, 2308, 646, 647, 5514, 3725, 3726, 5519, 5520, 17428, 28053, 28054, 17812, 17435, 19228, 17436, 27296, 18721, 5027, 10533, 20647, 17448, 17449, 17450, 17451, 8878, 21551, 16430, 17457, 17458, 16435, 17459, 17461, 17460, 25015, 16436, 8881, 18490, 3515, 17479, 15818, 1613, 5074, 979, 3030, 30040, 19167, 2656, 14689, 12256, 2661, 26215, 6504, 20206, 19439, 19440, 25076, 6520, 23933}), 220: frozenset({13633, 32323, 37316, 37317, 904, 16013, 9110, 29786, 21596, 8416, 39142, 39143, 25959, 39144, 11435, 37612, 26098, 19955, 13492, 2619, 12221, 2750}), 221: frozenset({38530, 31433, 35664, 35665, 37269, 37270, 37271, 23962, 23963, 23964, 32607, 16736, 13923, 35686, 37094, 37095, 23977, 18923, 37616, 8754, 10805, 26108, 26109, 26111}), 222: frozenset({13634, 15812, 26885, 968, 35657, 35658, 35659, 35660, 35661, 35662, 35663, 16016, 21008, 37200, 6426, 37092, 37607, 8744, 12139, 109, 8175, 16881, 16882, 25400, 26106, 26107, 24382}), 223: frozenset({9474, 1026, 3460, 16772, 14725, 2180, 26117, 14726, 3461, 15626, 24723, 1945, 26490, 8995, 4648, 4649, 2474, 19498, 556, 20269, 19501, 3247, 19500, 23733, 1208, 21820, 21439, 17088, 23871, 26307, 20678, 6599, 21065, 9417, 17995, 17996, 11089, 23763, 23764, 23765, 23766, 10837, 26082, 35557, 19175, 11496, 11497, 1257, 4206, 7152, 4208, 14577, 24052, 24053, 24054, 24055, 17528, 26489, 17146, 36477}), 224: frozenset({28801, 11905, 22789, 22790, 29319, 33160, 38519, 21006, 38521, 38522, 11806, 16038, 15142, 12205, 10801, 37429, 15546, 15547, 15548, 6204, 31422, 13759, 13760, 31423, 15553, 15554, 13761, 8770, 18246, 13762, 37192, 2123, 32591, 32592, 32593, 32594, 32595, 3960, 17112, 3961, 1112, 18140, 37085, 37086, 18783, 13920, 23395, 8420, 21736, 20986, 3958, 3959, 38520, 6393, 19962, 14844, 11517}), 225: frozenset({10320, 6180, 6181, 6182, 5180}), 226: frozenset({11520, 4866, 24841, 4363, 11021, 27664, 17172, 15143, 15144, 15145, 15146, 10799, 10800, 6196, 6197, 18245, 32587, 32588, 32589, 32590, 75, 36434, 23125, 7254, 3927, 20055, 601, 26459, 13918, 13919, 26466, 17509, 17510, 19302, 17511, 17512, 17513, 17514, 38508, 25450, 38510, 20984, 38512, 38513, 38514, 29309, 29310, 11907, 29315, 33667, 31877, 29316, 25992, 36230, 11404, 15780, 5035, 5036, 13751, 13754, 18890, 1485, 1488, 30419, 28116, 28117, 18139, 16097, 21735, 37863, 6391, 19960, 20985}), 227: frozenset({13832, 18313, 11403, 18316, 30993, 4896, 23201, 4898, 30627, 4900, 18344, 37037, 1070, 7991, 7992, 7993, 7995, 25157, 14790, 21830, 7624, 14280, 24023, 24024, 24025, 24026, 24027, 14556, 21335, 36575, 3296, 37473, 16759, 14453, 23671, 16249, 23674, 23675, 16892, 23677, 21758}), 228: frozenset({8391, 22986, 34316, 34317, 6222, 15951, 6223, 32976, 32975, 32979, 24784, 23324, 10079, 10080, 7520, 10081, 38124, 2800, 30770, 30771}), 229: frozenset({30469, 33446, 33447, 34957, 21134, 38607, 38608, 17137, 38609, 1714, 18808, 38619}), 230: frozenset({36900, 22022, 11292, 911, 36594, 36595, 28276, 35541, 24920, 11291, 5404, 11293, 17886, 11295}), 231: frozenset({15040, 12616, 12618, 2059, 6350, 23440, 38615, 37979, 37980, 27678, 20394, 5356, 6141, 25587, 35060, 6142, 19898, 15037, 15038, 15039}), 232: frozenset({38656, 38657, 38658, 38659, 4356, 38661, 38660, 4354, 30738, 13332, 11803, 2077, 24865, 31275, 31276, 31277, 22830, 31279, 22831, 31282, 34890, 34891, 22607, 22608, 22610, 22611, 22612, 9047, 22615, 16483, 31345, 31346, 31347, 32376, 32377, 18810, 19326, 29838, 12946, 12947, 21663, 38652, 20396, 8122, 8386, 33479, 26058, 15050, 38606, 29139, 6612, 20948, 20448, 34273, 34274, 34275, 21227, 34286, 25591, 7163, 30460, 38653, 38654, 38655}), 233: frozenset({26693, 26694, 26695, 26696, 26698, 9233, 3219, 20826, 6814, 6815, 6816, 5153, 20771, 16241, 113, 4021, 14136, 2428}), 234: frozenset({32896, 32898, 16516, 1927, 22421, 22422, 7960, 12315, 18332, 12318, 19871, 18334, 18335, 20900, 22437, 32933, 24612, 19880, 12078, 7222, 19767, 6906, 30781, 21822, 15678, 6472, 15356, 19156, 17114, 863, 27128, 27130, 15355, 27132}), 235: frozenset({17154, 39049, 33170, 28435, 28436, 28434, 12054, 20886, 11419, 548, 21284, 21033, 24490, 5040, 38578, 23477, 37174, 25398, 37175, 28345, 26561, 12227, 73, 3276, 37197, 37198, 10835, 16478, 20063, 22369, 2533, 3057, 23923, 13428, 16501, 13429, 33149}), 236: frozenset({23557, 17804, 1933, 6929, 6932, 17304, 18328, 17305, 18329, 20507, 18330, 6431, 7083, 38445, 13493, 18999, 6457, 19001, 2622, 13122, 13126, 32330, 14290, 12126, 17635, 21348, 17637, 21862, 21863, 21864, 17636, 17648, 2160}), 237: frozenset({20999, 11918, 12752, 11920, 1205, 1206, 22455, 34779, 24796}), 238: frozenset({6785, 30787, 30788, 5157, 30790, 4615, 4614, 4618, 13141, 18269, 18270}), 239: frozenset({19081, 19082, 3083, 12044, 6925, 1554, 37652, 23061, 1556, 25112, 37660, 1950, 20767, 9123, 3499, 15660, 3500, 2225, 2226, 13114, 14523, 13115, 13117, 25537, 24002, 17989, 24520, 10697, 24521, 14158, 33235, 33236, 33238, 33239, 7640, 2412, 34419, 34420, 34422, 34423, 3833, 16122, 4735}), 240: frozenset({20736, 9476, 15627, 20769, 20785, 14144, 4033, 11595, 36172, 11596, 11086, 36173, 21327, 11601, 21843, 8020, 11610, 11611, 2654, 3171, 18532, 18534, 4968, 18537, 18538, 14704, 3446, 3447, 886, 2169, 18297, 18298, 3451, 3452, 3453, 3454, 30591, 4995, 4996, 9605, 9604, 30604, 17811, 3220, 23192, 19363, 17829, 21159, 9127, 6825, 9133, 20656, 9393, 9394, 18614, 25275, 26300, 9405, 26301, 26302, 26304, 9404, 26306, 4032, 11460, 26308, 26309, 26310, 24260, 24261, 11461, 27592, 204, 205, 24268, 27593, 27594, 3542, 3543, 3546, 16860, 3548, 7145, 3570, 3571, 3572, 20213, 14583, 14584, 14585, 21756}), 241: frozenset({16769, 6915, 27652, 27653, 27654, 30599, 27655, 7305, 1930, 1929, 3850, 23053, 6798, 6799, 27019, 28182, 19097, 34842, 34841, 34843, 34845, 34846, 20505, 19749, 19751, 19114, 11060, 23606, 28728, 37433, 17081, 28729, 12347, 29756, 12348, 9533, 28735, 28736, 28732, 28738, 24638, 28737, 28745, 28746, 18377, 13146, 28763, 28764, 28765, 24542, 13148, 13151, 4703, 4704, 5611, 5612, 18286, 6918}), 242: frozenset({14081, 22532, 37898, 37899, 13324, 37900, 37901, 5265, 28438, 28439, 6174, 19240, 6186, 6187, 6188, 6189, 6190, 36783, 36784, 36786, 8378, 28353, 14788, 20423, 10314, 10315, 21706, 26066, 35540, 16980, 8154, 38618, 6367, 7137, 16488, 30444, 33653, 33654, 14712}), 243: frozenset({21376, 6337, 35080, 3361, 36772, 22244, 32742, 17126, 32744, 32745, 32746, 32748, 32046, 32751, 32755, 19254, 20855, 20856, 21375}), 244: frozenset({15489, 32031, 15565, 33488, 15569, 31285, 29142, 29143, 28927}), 245: frozenset({21632, 1792, 8448, 1803, 19212, 20111, 25999, 33298, 22675, 24340, 22676, 6009, 12570, 12571, 12572, 23965, 12573, 11421, 22686, 291, 20131, 7078, 34476, 34477, 34478, 3119, 34479, 34482, 22712, 22713, 22714, 22715, 34494, 34495, 34496, 31298, 35524, 26952, 35542, 33251, 33252, 33253, 33254, 24551, 33255, 24555, 33260, 35577}), 246: frozenset({36096, 36097, 16130, 36105, 36106, 16778, 36110, 18192, 25105, 16402, 2713, 7321, 18714, 667, 670, 19230, 13982, 20652, 20653, 28086, 28087, 1339, 16064, 37698, 37699, 37700, 25923, 37701, 21570, 10567, 37705, 37706, 3531, 4809, 8909, 4814, 8911, 4816, 4812, 4813, 16843, 11873, 20067, 35816, 21098, 7026, 3314, 10614, 10615, 37702, 20092}), 247: frozenset({19074, 19076, 24580, 4871, 25993, 24971, 1549, 4495, 12305, 12307, 12308, 14484, 12310, 12311, 19480, 19481, 2716, 19484, 33191, 1576, 33192, 36015, 33203, 33205, 33206, 14264, 33209, 33208, 33211, 33210, 10688, 10689, 25925, 21322, 18254, 18657, 18658, 18659, 3044, 3045, 10727, 2666, 3827, 3829}), 248: frozenset({23299, 9351, 9352, 27534, 27535, 17936, 27537, 5906, 5907, 5905, 2581, 27536, 25241, 5914, 5915, 11933, 25635, 25637, 36774, 36773, 29096, 15403, 15404, 36781, 36780, 23340, 32048, 32047, 20532, 32053, 19253, 33600, 33609, 33617, 16469, 13270, 6231, 30557, 6109, 23265, 34406, 32743, 32747, 32749, 25454, 7791, 10352, 2931, 2933, 20854, 21374}), 249: frozenset({24579, 16132, 22663, 12300, 34830, 19094, 16411, 33182, 21919, 21920, 4772, 35879, 1448, 10617, 21804, 1453, 20654, 23090, 7032, 23990, 23997, 25924, 37709, 37710, 37711, 37712, 1997, 4820, 4821, 4822, 4823, 31709, 31710, 5471, 11875, 31718, 31719, 21099, 21101, 7025, 25082, 10616, 7033, 7034, 7038}), 250: frozenset({20426, 3787, 6347, 715, 20430, 25232, 17234, 29140, 29141, 9687, 32027, 32028, 11804, 11807, 30821, 22828, 22829, 10414, 10415, 10416, 10413, 31283, 7162}), 251: frozenset({37920, 14816, 6179, 7139, 10757, 20935, 14376, 8366, 10382, 10319, 10383, 23029, 36791}), 252: frozenset({37632, 37629, 1804, 6029, 6030, 37630, 23508, 22680, 22681, 20122, 22682, 23967, 12576, 33256, 24552, 24556, 5999, 37624, 37625, 24570, 37627, 37628, 22717, 22718}), 253: frozenset({32576, 5768, 34511, 29305, 4890, 5788, 13917, 38497, 21283, 38503, 29303, 20905, 12780, 29304, 13742, 25774, 20911, 11313, 11314, 11315, 5940, 32949, 32950, 11312, 24559, 6389, 29294, 20982, 16958, 29295}), 254: frozenset({16961, 8450, 34881, 34883, 23877, 34892, 11536, 20945, 15444, 15445, 24471, 5847, 30430, 20447, 5856, 5855, 31270, 31271, 32364, 32366, 16180, 18101, 6779, 19325}), 255: frozenset({23555, 3075, 23556, 37126, 13575, 3078, 37127, 14916, 17870, 7186, 851, 6548, 38293, 38296, 26909, 32285, 13797, 13798, 24366, 13808, 24369, 24945, 18930, 24372, 1973, 25403, 18239}), 256: frozenset({18950, 14611, 14612, 14613, 8219, 20002, 6955, 6956, 21805, 23093, 17206, 27199, 3649, 27207, 27208, 27209, 5201, 5206, 5463, 5465, 6746, 6747, 8292, 14437, 8300, 6765, 27245, 27246, 8304, 11893, 36470, 3191, 11895, 13176, 13178, 25727, 26756, 26757, 26758, 4743, 26762, 26763, 26764, 21899, 26766, 12431, 12432, 12433, 12434, 26767, 26772, 21900, 26774, 25495, 25500, 8610, 8633, 8637, 20671, 16072, 25294, 25295, 2006, 2007, 18650, 25564, 18653, 18654, 25566, 25568, 11746, 36323, 11750, 3048, 18409, 18410, 18413, 11758, 11759, 18414}), 257: frozenset({19328, 22788, 37892, 15494, 6021, 28935, 1802, 32395, 32396, 2575, 33296, 22673, 22674, 33297, 35472, 12567, 20127, 2080, 15141, 26022, 20138, 23470, 8114, 3254, 25658, 33083, 33084, 3772, 31293, 31294, 31295, 31296, 31297, 31292, 9027, 708, 19908, 20936, 26956, 17230, 32596, 32597, 32598, 14810, 33248, 23137, 33250, 29154, 6372, 30052, 20450, 7655, 6203, 30442, 30827, 38523, 15484}), 258: frozenset({33153, 23394, 33154, 33156, 37894, 3750, 37895, 32582, 38509, 7251, 38515, 19961, 22076, 13758}), 259: frozenset({4998, 2694, 7318, 678, 681, 4784, 21947, 35772, 14652, 35774, 35780, 20305, 11991, 10583, 10584, 10586, 14428, 1375, 1376, 1637, 35433, 20076, 20077, 35437, 35439, 35440, 35441, 35442, 6515, 35438, 3699, 28021, 35449, 35450}), 260: frozenset({18696, 16077, 27220, 25304, 3099, 16798, 6498, 28067, 4330, 3627, 34799, 34800, 34801, 34802, 34803, 34805, 13304, 16057, 3324, 21565}), 261: frozenset({21760, 20481, 4737, 4736, 16266, 16267, 1932, 16268, 18324, 37653, 37654, 37655, 37656, 5145, 37658, 37659, 37657, 37661, 23198, 7710, 6432, 19743, 16557, 18353, 18354, 2227, 2228, 2229, 25524, 7608, 15674, 13116, 16574, 9411, 8004, 4678, 4680, 21578, 10698, 16205, 16206, 21332, 21860, 26470, 23664, 34417, 34418, 23668, 23669, 34421, 4734}), 262: frozenset({7616, 14272, 16770, 2758, 18380, 3854, 28752, 3857, 28753, 28761, 24538, 19355, 2651, 13152, 7458, 4707, 36454, 36455, 36456, 297, 19115, 28733, 28734}), 263: frozenset({19456, 19457, 24964, 1989, 30032, 30033, 5075, 2708, 5077, 30038, 5018, 30043, 17820, 30044, 20258, 24698, 9257, 9259, 12274, 3511, 3512, 3513, 3514, 16381, 23934, 25919}), 264: frozenset({26560, 26884, 26053, 37196, 1102, 16015, 37266, 37908, 23393, 37090, 37091, 34412, 15150, 2738, 26103, 26104}), 265: frozenset({11776, 6784, 16515, 32900, 14469, 32901, 32906, 21772, 18573, 32908, 18574, 18575, 32909, 18577, 18578, 32915, 32916, 32917, 32918, 32919, 1940, 1050, 4646, 11947, 12076, 11054, 7218, 23350, 24633, 27194, 4669, 6852, 6853, 33096, 33097, 33099, 33100, 14030, 32911, 17492, 32912, 17494, 3158, 20218, 10851, 16741, 16742, 14566, 8435, 8692, 27126, 39162, 20477}), 266: frozenset({16385, 16386, 4802, 16392, 18699, 4491, 10578, 10579, 2709, 35414, 35415, 35416, 35481, 11993, 5019, 35418, 675, 5093, 4137, 4138, 4139, 20073, 35892, 6519}), 267: frozenset({10113, 7877, 27014, 18567, 18568, 6857, 16523, 14029, 8334, 19154, 7957, 25758, 3167, 9510, 20902, 22444, 3885, 20476, 22449, 12088, 4668, 14463}), 268: frozenset({23489, 11523, 24645, 29768, 29769, 9355, 24845, 15503, 15504, 33682, 35475, 33683, 32407, 728, 25823, 19305, 7805}), 269: frozenset({2571, 36364, 11532, 12945, 1813, 22298, 22299, 24861, 21662, 36384, 24866, 31268, 36405, 36406, 16182, 15419, 34879, 34880, 16960, 15042, 35017, 35018, 34506, 35019, 34507, 22609, 15446, 27739, 34524, 2527, 29161, 38644, 25588, 38645, 38647, 38648, 38646, 32381, 32382}), 270: frozenset({6916, 3848, 19086, 34447, 34448, 24977, 19346, 34839, 34840, 1562, 34847, 19104, 34848, 23074, 23075, 12329, 12330, 12331, 12332, 12333, 9532, 23616, 1222, 1223, 10705, 7633, 24531, 24532, 10728, 36458, 6890, 16748, 16621, 10730, 6891, 6892, 16749, 6893, 27631, 27632, 27633, 27634, 27635, 27640, 27636, 27637, 27005}), 271: frozenset({12800, 12801, 11910, 9545, 24907, 30540, 275, 8342, 23576, 14874, 2906, 13275, 25051, 39211, 21613, 15790, 6131, 29369, 701}), 272: frozenset({13570, 38597, 16581, 23237, 35913, 24915, 20058, 23208, 24043, 24047, 7088, 23535, 38836, 18868, 17973, 18229, 1908, 18170, 4540}), 273: frozenset({17550, 25488, 28058, 28059, 27554, 27555, 27557, 1321, 1322, 21803, 16058, 16331, 16332, 3661, 16333, 18897, 16338, 16339, 16340, 24286, 24287, 24288, 16609, 11489, 11490, 24290, 24293, 18790, 18791, 24289, 9460, 13307}), 274: frozenset({23737, 4751, 25490, 25502, 16351, 20257, 5090, 3242, 5483, 5485, 26926, 1329, 24178, 24179, 24180, 24182, 1337, 1338, 36093, 36094}), 275: frozenset({27264, 8901, 3527, 4808, 8903, 8330, 16396, 18701, 18191, 18708, 661, 18710, 663, 2712, 18709, 1624, 26415, 26416, 26418, 36084, 36087, 16827, 26429}), 276: frozenset({18944, 20738, 6924, 21776, 6930, 35349, 35350, 35351, 35352, 9243, 19752, 11062, 14135, 24632, 36668, 3900, 13892, 13893, 13894, 17991, 13897, 16221, 21855, 24672, 24673, 10848, 16739, 16228, 16229, 21349, 23653, 23655, 23656, 26226, 26227, 26228, 26230, 9591, 2427, 30589, 2430, 37505, 24962, 17543, 37513, 37514, 6806, 6807, 6808, 3225, 18327, 17306, 8603, 4004, 7085, 4015, 16306, 16307, 16308, 23224, 14528, 15822, 15823, 2776, 7642, 9446, 27111, 27112, 9447, 1015}), 277: frozenset({26888, 39177, 6410, 39180, 2317, 2318, 4378, 4379, 15646, 18985, 18986, 18987, 5422, 2111, 19008, 2639, 1106, 16468, 32347, 32348, 32349, 31580, 7268, 31590, 25968, 21880, 21882, 21883, 36997, 1173, 1174, 16023, 1175, 37529, 7834, 37531, 37536, 37537, 37538, 37539, 37540, 37541, 37542, 38566, 37544, 38567, 38565, 16546, 17837, 17838, 37553, 37554, 37556, 17343, 959, 38354, 13789, 14560, 11492, 751, 39155, 39156, 39157, 14838, 755}), 278: frozenset({9218, 9220, 9221, 12808, 12810, 3211, 19213, 30992, 30429, 31647, 31648, 31649, 31650, 31654, 1900, 37039, 31664, 37041, 37042, 37043, 31666, 37045, 31665, 31669, 31668, 23412, 18165}), 279: frozenset({12548, 9975, 1806, 10127, 10128, 9976, 22802, 1811, 24598, 5655, 30489, 29211, 20123, 29213, 29214, 31263, 22689, 22690, 22691, 20140, 26039, 19896, 16959, 8256, 12609, 12610, 10431, 10436, 10435, 8133, 30538, 20427, 34508, 35021, 20942, 7503, 7502, 35020, 28238, 25812, 27734, 27735, 13278, 9955, 21608, 29162, 24554, 35051, 35052, 29163, 8684, 5354, 23150, 9966, 9968, 17772, 9970, 33268, 33271, 9967}), 280: frozenset({15340, 24207, 33457, 16981, 8245, 5816, 5817, 31870, 159}), 281: frozenset({274, 39197, 6558, 38815, 39200, 6560, 39202, 39203, 39201, 26666, 26668, 15789, 26670, 26671, 26672, 16689, 26673, 29362, 4917, 26679, 4407, 26681, 26680, 16709, 20165, 11339, 17882, 5210, 35550, 21485, 20464, 12791, 12152, 36220}), 282: frozenset({512, 35842, 17282, 17028, 17029, 17286, 17287, 511, 26257, 18718, 23216, 11441, 25523, 23229, 36926, 1857, 20169, 6735, 20183, 29784, 29785, 38881, 38882, 38884, 38885, 38886, 38887, 38888, 38889, 38890, 29803, 38892, 38893, 38894, 38891, 19194, 510, 17279}), 283: frozenset({28162, 29827, 18822, 18824, 18825, 18826, 779, 6283, 33937, 22938, 22939, 22951, 22952, 30769, 30773, 30775, 32068, 11224, 2395, 20957, 6373, 10086, 10087, 21477, 20729, 10090, 38122, 10092, 10093, 10091, 38127, 10096, 10097, 38134, 34296, 34297, 34300}), 284: frozenset({22024, 22026, 22027, 22028, 910, 38799, 38800, 6034, 17695, 11296, 17697, 11297, 11298, 17700, 11301, 11299, 36903, 17704, 36905, 36906, 17703, 11300, 9382, 24900, 17696, 5281, 14410, 17698, 24921, 24545, 105, 26477, 26478, 13551, 24569}), 285: frozenset({19845, 19846, 20487, 19848, 538, 13599, 39072, 39073, 39074, 1059, 1060, 1061, 25122, 39075, 13602, 7977, 7978, 7979, 942, 5696, 11840, 15228, 16841, 14538, 14539, 15188, 15189, 15190, 15191, 30549, 15193, 15194, 15195, 858, 3294, 22371, 20220, 22373, 22372, 22375, 22376, 22377, 22374, 22378, 16755, 16507, 30972, 15229}), 286: frozenset({25472, 3332, 18694, 18695, 17547, 17548, 17549, 1685, 1309, 1310, 26912, 28065, 28066, 4133, 1320, 21163, 1964, 21164, 16431, 16056, 25532, 21564, 9165, 15824, 26709, 26710, 16342, 16341, 25305, 26716, 11868, 991, 12002, 24291, 24292, 9189, 5478, 5480, 3050, 10477, 25073, 15858, 15859, 9459, 9461, 6518, 23935}), 287: frozenset({37312, 18241, 12807, 31624, 31625, 31623, 31628, 1901, 31629, 37308, 9214, 8415}), 288: frozenset({26063, 27600, 20626, 1684, 24277, 24278, 20637, 28061, 9187, 9188, 4838, 4841, 21162, 7147, 9457, 21429}), 289: frozenset({30981, 30983, 30984, 9101, 36750, 7196, 15138, 15139, 15140, 37284, 37283, 5668, 5667, 39080, 39082, 39083, 2092, 39084, 5670, 943, 8113, 7986, 7859, 15158, 3255, 16675, 37285, 9931, 9932, 9933, 3406, 9935, 9936, 3408, 14547, 4947, 3295, 2535, 16757}), 290: frozenset({20996, 26504, 26890, 26635, 26636, 6414, 26639, 7573, 37526, 37527, 37528, 4377, 26650, 6555, 26649, 25885, 37150, 37151, 13728, 37153, 13725, 31779, 37149, 15525, 15526, 13726, 30888, 37154, 13730, 30892, 27052, 1580, 15663, 15664, 30894, 30893, 4403, 37549, 18871, 14015, 963, 2117, 15815, 13729, 17877, 26908, 11866, 23391, 13537, 12134, 38375, 37482, 37483, 37484, 13933, 37485, 25071, 37488, 25974, 23545}), 291: frozenset({8196, 28100, 26186, 26187, 11406, 25432, 39204, 39205, 39206, 39207, 15528, 39208, 1896, 24939, 4908, 21741, 31790, 29358, 12792, 2737, 23410, 12793, 29364, 29365, 29366, 4919, 5044, 24438, 3770, 31864, 39103}), 292: frozenset({14466, 14467, 9989, 27013, 3591, 22536, 22537, 9991, 3592, 18571, 9992, 9990, 21258, 22538, 4884, 32922, 32927, 32928, 32929, 32930, 32931, 32932, 19878, 25640, 7217, 9396, 27190, 27191, 12087, 7878, 14154, 4044, 4046, 4047, 4048, 17490, 19155, 17493, 17238, 1926, 27500, 27501, 27502, 27119, 27120, 27505, 27122, 27123, 27121, 27503, 6902, 6904, 2431}), 293: frozenset({8320, 12288, 642, 13954, 8007, 17429, 17430, 17433, 17434, 3033, 24187, 17437, 17439, 12255, 19423, 21794, 12257, 17445, 8871, 8872, 8873, 3882, 30002, 30003, 18359, 24184, 24185, 24186, 7867, 16830, 12287}), 294: frozenset({18692, 18693, 9606, 9607, 13979, 28062, 28063, 28064, 4131, 4132, 1317, 1318, 1319, 2368, 16337, 24279, 26714, 26715, 24283, 24282, 20830, 24284, 11487, 11488, 24285, 11869, 21989, 10476, 15855, 9456, 24433, 24434, 25074, 15857, 9458, 3704, 16379, 16380}), 295: frozenset({30784, 26049, 30786, 3587, 15365, 18630, 15368, 15369, 11093, 18263, 18264, 19160, 15449, 15452, 15389, 15453, 15457, 25643, 6899, 19765, 2421, 17079, 2809, 25145, 3898, 2812, 4606}), 296: frozenset({22720, 8449, 7501, 11405, 14737, 22679, 22683, 17692, 1787, 33257, 20139, 6000, 6001, 34481, 37623, 7483, 22719}), 297: frozenset({6788, 23973, 36613, 38309, 23402, 37233, 37234, 17876, 37593, 26646, 18231, 37592, 38393, 16028}), 298: frozenset({25865, 39181, 39182, 20510, 8478, 8480, 21280, 802, 8479, 20775, 12075, 8760, 25659, 25660, 33087, 33088, 33089, 2113, 33091, 33090, 8261, 13637, 10823, 10822, 13641, 13642, 13643, 15435, 15437, 15436, 15434, 34381, 2641, 30551, 11863, 32352, 12129, 5731, 32356, 32357, 20588, 17517, 14198, 5756, 5757, 16511, 21888, 21887, 16514, 22405, 22406, 32905, 22409, 22411, 32907, 16527, 22415, 1178, 39159, 19868, 20897, 16547, 11944, 11945, 15281, 15282, 960, 32972, 32973, 13297, 14839, 39163, 39164}), 299: frozenset({13952, 29632, 1413, 14023, 30026, 5072, 8021, 30037, 7287, 3037, 5150, 9247, 3550, 20190, 24669, 3235, 20515, 8550, 9128, 19438, 12272, 26737, 26738, 12273, 7284, 25075, 3510, 25079, 30584}), 300: frozenset({14282, 13912, 21337, 6812, 120, 121, 23205, 24038, 24678, 24040, 24039, 20202, 16239, 4976, 4019, 4020, 29623, 29624, 29625, 29626}), 301: frozenset({8896, 13953, 25920, 8893, 25096, 19469, 3342, 16397, 1623, 5020, 26398, 26399, 26400, 20191, 3168, 26406, 26407, 26409, 26414, 26419, 26995, 3517}), 302: frozenset({5760, 12097, 5762, 5763, 6404, 5765, 16011, 10829, 11534, 25871, 15439, 15440, 16530, 38487, 4888, 4889, 3418, 16034, 20904, 27050, 3946, 8171, 13486, 25391, 32948, 13627, 13628, 32318}), 303: frozenset({9986, 6019, 1796, 12549, 21257, 12555, 12556, 12557, 20107, 20112, 6674, 12562, 12563, 25754, 9980, 22696, 22697, 22706, 22707, 22708, 34870, 34871, 34872, 34489, 34874, 34379, 26961, 7508, 24549, 27496, 27498, 9972, 33269, 33142, 33270, 30456, 9977, 9973, 9979, 9974, 9981, 6015}), 304: frozenset({24192, 18816, 2050, 2051, 35846, 35847, 35848, 35849, 35850, 29195, 22535, 29833, 29832, 29831, 24850, 23317, 5654, 21271, 794, 30622, 11935, 11937, 30754, 34730, 34731, 34732, 33324, 33327, 26927, 33713, 33714, 28856, 28857, 28859, 16188, 28861, 28862, 15934, 28860, 1991, 9550, 19919, 28891, 28892, 28893, 15467, 26475, 34669, 19307, 34671, 15468, 15469, 34670, 17919}), 305: frozenset({20810, 38412, 38415, 37591, 17881, 39198, 1190, 1191, 1895, 25900, 7598, 37230, 37231, 37870, 37871, 18162, 4404, 37872, 12789, 12790, 9208, 29754}), 306: frozenset({12613, 9938, 7124, 9941, 9942, 9943, 30490, 2907, 29219, 9956, 16100, 18728, 23593, 19883, 19884, 5355, 21615, 2544, 16819, 6134, 31351, 32953, 29822}), 307: frozenset({35843, 28292, 14285, 19343, 29328, 39000, 21340, 21341, 39008, 39011, 39014, 17254, 17255, 17256, 17257, 23211, 38828, 23213, 9390, 38827, 29488, 29489, 17258, 25452, 29805, 17262, 17261, 29808, 17260, 18876}), 308: frozenset({10758, 33681, 5266, 18071, 8364, 36792, 36793, 36794, 36795, 36796, 27710, 27711, 27712, 6340, 20424, 8271, 10322, 9561, 14817, 5347, 23270, 34921, 34409, 34922, 34923, 15341, 15342, 30831, 15343, 29166, 15344, 10360}), 309: frozenset({18309, 21896, 18312, 21518, 19859, 19860, 1054, 1055, 12064, 12063, 1062, 1063, 12208, 32944, 7230, 3903, 21056, 14147, 7621, 5703, 6473, 5707, 5708, 14541, 5710, 5709, 3536, 5713, 5714, 6864, 5711, 5717, 5718, 5716, 4960, 10854, 30579, 16244, 16245, 16243, 2679, 30580}), 310: frozenset({6403, 25870, 13589, 13590, 38428, 7203, 13619, 23348, 33077, 33079, 33080, 25656, 2107, 12091, 12093, 8767, 12096, 12100, 2633, 17738, 30541, 10830, 38486, 16474, 13402, 13660, 13661, 13662, 13663, 13664, 13403, 25958, 3943, 3944, 3945, 13417, 3947, 5758, 5759, 5761, 5764, 16009, 16528, 16529, 13464, 13465, 15259, 13467, 23455, 23456, 23457, 15263, 15269, 15272, 15273, 15274, 15275, 25769, 15277, 15278, 15279, 22451, 22453, 16567, 28104, 17106, 14035, 19946, 14840}), 311: frozenset({29312, 11522, 905, 21004, 31758, 8853, 24217, 24218, 24219, 38568, 15147, 2739, 20533, 13752, 13753, 28092, 29757, 37181, 37182, 23876, 32583, 23624, 1482, 10832, 23633, 7252, 28123, 2531, 8419, 12137, 14187, 8172, 13676, 8432, 16886, 14842, 29311}), 312: frozenset({11535, 5650, 15443, 20051, 5850, 21277, 5854, 800, 25249, 33058, 22819, 5857, 5858, 5860, 13292, 16111, 8370, 35062, 15420, 15421}), 313: frozenset({20034, 32963, 14212, 32964, 32965, 32967, 24780, 8333, 6418, 39188, 30879, 5795, 5796, 27109, 5798, 5799, 25775, 20912, 20913, 15668, 13300}), 314: frozenset({28228, 19301, 8457, 13327, 5808, 5809, 24209, 24208, 725, 34967, 5819, 5821}), 315: frozenset({34434, 10890, 24716, 25102, 7632, 659, 26387, 19091, 34428, 20770, 19364, 34429, 17830, 34471, 34472, 3177, 8555, 14253, 8443, 6834, 4596, 17077, 20215, 8891, 23740, 23741, 23742}), 316: frozenset({8385, 33283, 33284, 5955, 34885, 7495, 20118, 26970, 6047, 6048, 33061, 6053, 20135, 21285, 33065, 33066, 21291, 22703, 20591, 22516, 22517, 22518, 20601}), 317: frozenset({12290, 12292, 12295, 24969, 16779, 4493, 19473, 36115, 16404, 16405, 16152, 36126, 17822, 36127, 20259, 1443, 35877, 35878, 1573, 36133, 36132, 1574, 26930, 37696, 37697, 5440, 10568, 3529, 3530, 10571, 10570, 37707, 3534, 4815, 1631, 1633, 3042, 8803, 23015, 31720, 31721, 2665, 2416, 10613, 13941, 20091}), 318: frozenset({17798, 8327, 8329, 656, 26388, 26389, 26390, 26392, 26393, 26394, 26401, 26402, 3041, 26404, 1572, 26405, 2664, 14252, 26996, 12286, 5439}), 319: frozenset({11843, 12805, 12806, 31626, 12811, 31627, 30991, 23952, 4948, 14555, 26914, 31651, 31655, 31656, 31657, 31658, 31659, 31660, 37038, 2095, 37040, 945, 26869, 37047, 23672, 32314, 11066}), 320: frozenset({3393, 25685, 12122, 13596, 23459, 25956, 34726, 34727, 34728, 34729, 15218, 15219, 15220, 15221, 15222, 2105}), 321: frozenset({30980, 11846, 3402, 3403, 9100, 3404, 3405, 86, 4953, 4954, 37280, 37281, 31010, 5666, 8739, 5669, 5671, 2093, 3252, 31797, 18935}), 322: frozenset({9344, 16102, 29835, 18095, 33041, 8274, 33042, 28215, 8382, 15935}), 323: frozenset({8321, 644, 645, 3526, 16395, 1614, 27024, 17431, 2711, 7320, 20192, 17442, 17443, 17444, 8802, 8874, 17455, 17456, 2415, 8816, 37495}), 324: frozenset({34929, 29745, 17155, 5349, 33624}), 325: frozenset({28035, 7300, 7299, 28036, 28037, 20630, 16791, 11685, 11686, 11687, 11688, 6954, 6958, 11696, 21562, 4156, 14653, 35773, 19775, 14654, 35781, 16074, 20299, 591, 25296, 20312, 9182, 25313, 28022, 36729, 37499, 18687}), 326: frozenset({34304, 23300, 30791, 27532, 27533, 18830, 18831, 22988, 22989, 788, 7521, 23333, 15397, 15399, 15400, 18731, 12846, 19124, 33529, 18106}), 327: frozenset({17345, 8514, 14213, 14214, 14215, 9593, 14730, 26893, 13133, 13134, 13135, 13136, 1938, 15826, 17557, 17558, 21143, 17559, 3029, 21530, 24614, 18799, 880, 178, 4660, 4661, 9141, 17334, 4665}), 328: frozenset({3588, 1925, 15366, 23368, 4045, 22543, 22544, 797, 11939, 11941, 8679, 25642, 23342, 27504, 9137, 6900, 4983}), 329: frozenset({6531, 6926, 9231, 6928, 6927, 17810, 19214, 1557, 6935, 19754, 4650, 3509, 4023, 15672, 26683, 16316, 6461, 26684, 26685, 14532, 13125, 19141, 13124, 26704, 21841, 20819, 21592, 7644, 17245, 21861, 3566, 114, 16242}), 330: frozenset({38624, 11970, 22531, 19300, 18090, 33586, 23123, 33651, 23255, 38623}), 331: frozenset({32384, 32385, 20129, 7172, 32389, 24868, 32390, 26958, 5657, 82, 31286, 31288, 1817, 33244, 8376}), 332: frozenset({23968, 26945, 7490, 20128, 24544, 101, 17657, 22030, 6031, 24568, 17689, 17690, 17691, 3199}), 333: frozenset({4353, 32386, 4355, 32387, 32388, 38662, 32391, 2569, 21133, 2573, 29843, 1815, 13335, 11805, 24863, 2079, 25250, 12709, 22825, 22826, 22827, 31280, 51, 35508, 5045, 31289, 36155, 1597, 5949, 33472, 9026, 20425, 15562, 23115, 15564, 34895, 19921, 20946, 20947, 9555, 17109, 17111, 19327, 29146, 29147, 29148, 29150, 8158, 34403, 30822, 14823, 30825, 17136, 30462, 30463}), 334: frozenset({22533, 11782, 16976, 21264, 27538, 789, 5912, 5913, 7769, 11934, 416, 5921, 29602, 11936, 29603, 33706, 25139}), 335: frozenset({39042, 10117, 10118, 15246, 15248, 15249, 21267, 15255, 15256, 5785, 5784, 15257, 9113, 5787, 5786, 15258, 25761, 21031, 38313, 13737, 11051, 30508, 17965, 38318, 5936, 12089, 39110, 16712, 39113, 20043, 17739, 24786, 12762, 32986, 13538, 12779, 13420, 12797, 12798, 3071}), 336: frozenset({3843, 13943, 17800, 14473, 3853, 28183, 34850, 19111, 19112, 19113, 28722, 21811, 21812, 28723, 15670, 28725, 28726, 21810, 28730, 28731, 28727, 28724, 11582, 11583, 11584, 11585, 11586, 28739, 28740, 28741, 5444, 11587, 11589, 11588, 18378, 18379, 21324, 2128, 5333, 26998, 2423}), 337: frozenset({32577, 25156, 14791, 36233, 32585, 23630, 39055, 23631, 23632, 18902, 18903, 7579, 35678, 35679, 35683, 35684, 13674, 3953, 3954, 16885, 20983, 29306}), 338: frozenset({14845, 11908, 15495, 15496, 15497, 15498, 37193, 37261, 37262, 8405, 16734, 37088, 30438, 3964, 15148, 10802, 3962, 3963, 38524, 36157, 22078}), 339: frozenset({2946, 18823, 38795, 15954, 23325, 10077, 10078, 7522, 10083, 10084, 10085, 10088, 10089, 38125, 38126, 2799, 25137, 21297, 2801, 3262}), 340: frozenset({18081, 5859, 5861, 22474, 22475, 22476, 22477, 8371, 35064, 12701}), 341: frozenset({7552, 8195, 22019, 18958, 22035, 22036, 38807, 27034, 38812, 38813, 38814, 39199, 26656, 5407, 14754, 9381, 7462, 26665, 14762, 14763, 14764, 14765, 23597, 29357, 29359, 29360, 29361, 26667, 26669, 11334, 24902, 11337, 26838, 26841, 17885, 18147, 29423, 31860, 37877, 37878, 37879, 37880, 12788, 25978, 37876, 37881}), 342: frozenset({3585, 3586, 21505, 11777, 20230, 9994, 9995, 15372, 15373, 15375, 22547, 4885, 4886, 9495, 4644, 19759, 7216, 27187, 7219, 27189, 3894, 25143, 27192, 27188, 3901, 4670, 15176, 15177, 15179, 15180, 17741, 15181, 15184, 17497, 23899, 7262, 10850, 16740, 16485, 34407, 2408, 25449, 27499, 10105, 10106, 15230, 15231, 15232, 14465, 27012, 30597, 16518, 18569, 18570, 16522, 18572, 22416, 1939, 25757, 26018, 19877, 32939, 32940, 179, 35513, 18628, 18629, 4043, 8680, 27116, 27117, 27118, 21750, 4605}), 343: frozenset({18560, 18561, 18562, 18563, 18564, 20229, 18565, 15625, 21770, 17555, 17556, 18583, 18584, 1570, 14243, 5156, 21798, 6952, 20777, 6826, 6828, 19757, 177, 20786, 3893, 4662, 17335, 17336, 17337, 26298, 2491, 4664, 26303, 17344, 26305, 4034, 17347, 17348, 17349, 8516, 3913, 21069, 9426, 12243, 12375, 16859, 21851, 21852, 19166, 14559, 12128, 21854, 16737, 16738, 6886, 881, 21749, 31736, 18556, 18557, 18558, 18559}), 344: frozenset({25988, 1797, 33287, 33288, 22665, 22666, 8840, 33290, 22667, 12558, 12559, 12560, 12561, 23570, 1812, 36757, 5754, 6044, 6045, 20125, 18086, 37800, 22698, 22702, 25134, 22704, 22705, 34491, 34877, 34878, 26943, 9536, 16963, 7494, 23367, 26951, 7498, 26955, 15438, 34897, 7505, 28372, 23897, 23133, 20062, 23136, 22500, 22501, 22502, 22503, 27495, 22505, 22506, 22507, 22514, 5747, 11507, 5749, 9978, 33275}), 345: frozenset({37442, 38534, 38535, 38536, 32327, 20814, 12239, 846, 25426, 30869, 918, 38435, 38436, 1891, 551, 37035, 108, 14189, 38383, 38386, 37883, 37884, 37885}), 346: frozenset({33281, 1798, 2568, 1814, 34713, 21274, 34714, 12700, 24862, 34719, 18080, 6051, 6052, 22820, 33062, 33063, 31272, 33064, 22821, 22701, 22575, 18099, 10164, 10165, 10166, 8372, 10169, 10170, 10171, 5953, 9539, 34884, 26950, 34887, 22473, 20429, 22478, 28240, 28370, 28371, 7506, 23134, 14821, 34280, 32365, 32367, 32368, 30458, 30459, 33279}), 347: frozenset({775, 21263, 34710, 8471, 18841, 2083, 25381, 36775, 15401, 15402, 12845, 12847, 32049, 18104, 33601, 33602, 33603, 8389, 12870, 10055, 10056, 10057, 10058, 30793, 33610, 33613, 33611, 13269, 5342, 21470, 19296, 17122, 7790, 2932, 34293, 34294, 2814}), 348: frozenset({36995, 36996, 9477, 37510, 9478, 17544, 6435, 6438, 25895, 3752, 3753, 2987, 21037, 18990, 18991, 17842, 17843, 17845, 37558, 37559, 37560, 37561, 37562, 37565, 37566, 37567, 37568, 37569, 38471, 13778, 38356, 1115, 13788, 9445, 36469, 37118}), 349: frozenset({19201, 24578, 6022, 6665, 6668, 20751, 19727, 10257, 24342, 24344, 11032, 24346, 37275, 29084, 29083, 24348, 29085, 29086, 29087, 13608, 13609, 7467, 18860, 3116, 7473, 26939, 17862, 19016, 31437, 31438, 36955, 36956, 25950, 21738, 12531, 18932, 12532, 10236, 10237, 10238, 8447}), 350: frozenset({17346, 20228, 20740, 17350, 15623, 17351, 14153, 17994, 17354, 21068, 17356, 1937, 12242, 20509, 37535, 15522, 15523, 6949, 6950, 6951, 4904, 1965, 11438, 20783, 882, 26293, 18552, 18553}), 351: frozenset({8705, 2628, 25418, 34383, 25680, 34385, 34386, 4887, 20888, 13401, 38426, 13405, 23454, 13470, 13406, 7263, 13731, 25955, 5732, 5733, 19947, 8496, 23921, 13618, 23359}), 352: frozenset({38600, 14284, 29329, 21338, 17251, 17252, 17253, 23207, 31784, 31783, 23210, 24044, 38829, 24045, 17263, 32560, 17264, 29490, 32563, 18869, 38837, 38839, 38840, 1909}), 353: frozenset({24266, 8012, 26703, 20821, 3034, 19168, 19425, 19426, 12261, 21160, 21290, 24431, 19442, 117, 25017, 25531}), 354: frozenset({2689, 259, 4755, 4756, 4757, 4116, 19231, 680, 21943, 21944, 21945, 35775, 20307, 35796, 35797, 35798, 10585, 10588, 989, 10589, 10594, 20708, 20709, 3686, 3687, 1640, 3688, 16616, 18025, 14441, 16365, 20078, 20080, 18676, 35445, 35446, 35447, 35444, 35451, 35452}), 355: frozenset({17222, 711, 31756, 12686, 22806, 31257, 25049, 37338, 9954, 34786, 35045, 34405, 29164, 29296, 20980, 10429, 10430}), 356: frozenset({24131, 6755, 1323, 1324, 26381, 26382, 23825, 23826, 1620, 8854, 2038, 16344, 23834, 23835, 16059}), 357: frozenset({14210, 24963, 23940, 29634, 23942, 23943, 24392, 24393, 24394, 23947, 3213, 9230, 3214, 24403, 11486, 21535, 2660, 17061, 25000, 24432, 26994, 24436, 24437, 19451, 19453, 19454, 19455}), 358: frozenset({22017, 22018, 26231, 26897, 26233, 26899, 2837, 2838, 5402, 9380, 1958, 25519, 25520, 32569, 13370, 7097, 1862, 36295, 36296, 14409, 1863, 20172, 14291, 17753, 17754, 29408, 29409, 29410, 29411, 29796, 29797, 29798, 29412, 29800, 29414, 29419, 29420, 35951, 21365, 23158, 21367, 26232, 35961, 10492}), 359: frozenset({38529, 902, 31435, 28110, 28111, 35667, 29081, 17116, 32606, 32608, 19939, 13604, 13605, 8421, 35687, 23979, 20459, 10804, 18236}), 360: frozenset({19968, 30979, 13449, 13450, 30985, 16012, 19350, 9111, 540, 16544, 25121, 8738, 31011, 31012, 39077, 37282, 7207, 16674, 5674, 23467, 1068, 39085, 23468, 7983, 31792, 31793, 31794, 24627, 31795, 15157, 13490, 15159, 11064, 15161, 15162, 15163, 11836, 13629, 10812, 13631, 20032, 15166, 13630, 26563, 13632, 2632, 1101, 14544, 39130, 23900, 3421, 8412, 8424, 19949, 31798, 15160, 18931, 14068, 11516}), 361: frozenset({4880, 23314, 34323, 34709, 30747, 6110, 36777, 36778, 36779, 5931, 5933, 30765, 5932, 2796, 10353, 10354, 32051, 10356, 10357}), 362: frozenset({10501, 38411, 39193, 9498, 26651, 39195, 26652, 2845, 26657, 26660, 26661, 26662, 26663, 26664, 23596, 16687, 32308, 4406, 32570, 16700, 10816, 16707, 16725, 37227, 31861, 31862, 31863, 24951, 12151, 21371, 36488, 19338, 38806, 16027, 6557, 6561, 14755, 14756, 30884, 14757, 14758, 14759, 29356, 35246, 35247, 35249, 7093, 7094, 36297, 36298, 17879, 17880, 36315, 36316, 36317, 14302, 36319, 35563, 21484, 12783, 20463, 12785, 13817, 23546, 19196}), 363: frozenset({3840, 17153, 34818, 15490, 31748, 31749, 34821, 21507, 34820, 33217, 33218, 10691, 24525, 3533, 10574, 14482, 14483, 14485, 19093, 10717, 10847, 36128, 36129, 14256, 26997, 183, 14017, 23999}), 364: frozenset({21506, 23610, 3847, 18381, 18382, 1551, 18383, 3896, 4708, 4709, 10726, 27623, 27624, 27625, 27626, 36459, 27627, 36460, 20475, 15669, 16117, 23607, 23608, 23609, 21818, 23611, 23612, 15671}), 365: frozenset({20438, 17912, 15337, 27706, 7804, 33677, 27705}), 366: frozenset({21769, 9232, 9234, 1559, 21527, 4655, 4656, 4657, 21046, 26686, 26687, 26688, 26689, 2370, 13123, 26690, 26691, 26692, 2371, 17992, 11593, 13131, 13132, 26700, 26699, 14159, 29774, 23634, 21844, 21845, 21846, 20823, 20824, 20825, 20827, 20828, 2413, 4978, 2163, 18548, 18549, 18550, 18302, 30590, 1936, 3218, 18579, 18580, 6820, 9134, 18611, 3508, 4027, 4028, 14531, 19142, 3023, 9424, 11495, 35568, 3573, 3574, 20214, 14586}), 367: frozenset({12289, 12291, 16387, 19459, 19461, 3849, 34442, 36107, 34444, 27023, 19472, 16401, 36114, 36116, 36117, 36120, 36121, 5151, 20517, 17062, 35889, 35890, 35891, 18357, 182, 35407, 10576, 8912, 35411, 3156, 35413, 35412, 5461, 5460, 8025, 8026, 24534, 3038, 1630, 5091, 5092, 20072, 12278, 12279, 3835}), 368: frozenset({9216, 37056, 3208, 31630, 31631, 31632, 31633, 31634, 31636, 23957, 16538, 38490, 38491, 4585, 4587, 4588, 19957, 30518, 37053, 37054}), 369: frozenset({24583, 3337, 4108, 4112, 4113, 4114, 4115, 9260, 9261, 10541, 24117, 24118, 24119, 4152, 4153, 2358, 20303, 8787, 30547, 21081, 4187, 14684, 30046, 30047, 3680, 3681, 3682, 10590, 14688, 3685, 18021, 14687, 30048, 30049, 37486, 8823, 37497, 6522, 24701, 24702, 24703, 127, 24707, 24708, 10884, 10883, 4490, 7308, 10900, 5012, 11672, 11673, 11674, 11675, 20636, 11676, 11679, 11680, 11681, 4516, 3237, 3240, 8616, 20651, 30637, 11697, 11698, 11699, 11700, 11701, 11702, 11703, 12478, 35790, 9172, 35799, 11999, 5628}), 370: frozenset({13440, 36992, 37117, 37508, 17545, 17546, 2316, 10509, 7564, 21015, 18968, 18969, 18970, 25879, 7199, 35618, 25893, 7589, 35626, 35627, 35628, 35629, 35630, 6442, 35639, 35640, 35641, 38458, 38459, 38460, 7099, 7102, 7103, 13504, 7098, 23100, 35644, 8772, 15813, 197, 15821, 13777, 38353, 13779, 13787, 23387, 14011, 36984, 27063, 9444, 30950, 27064, 749, 12142, 12143, 8182, 23928, 36985, 19197, 36991}), 371: frozenset({17281, 17027, 17283, 23173, 17285, 23175, 23176, 17284, 18893, 6734, 11343, 23184, 4369, 26258, 14287, 38867, 38872, 38873, 38874, 26843, 38880, 21344, 38883, 10490, 19179, 23215, 23218, 23219, 17273, 506, 17274}), 372: frozenset({24768, 9540, 32040, 32041, 16970, 2254, 1519, 25360, 28788, 28789, 409, 22526}), 373: frozenset({9157, 10694, 19080, 37512, 21514, 33226, 33227, 33228, 33229, 3599, 5135, 33230, 33231, 33232, 30614, 6809, 6810, 4636, 2144, 16232, 20201, 16233, 24622, 4016, 35761, 35762, 3891, 31734, 3831, 17087}), 374: frozenset({32007, 32008, 32010, 32011, 15243, 29202, 29203, 30815, 33017, 33319, 33320, 33321, 18091, 6705, 33015, 33016, 5945, 17917, 33021}), 375: frozenset({8075, 1550, 3856, 21780, 20760, 4892, 31776, 31777, 31778, 24992, 21540, 23595, 36014, 8498, 8499, 8500, 8501, 17082, 14266, 7614, 18369, 14018, 18371, 18372, 18373, 18374, 11591, 11590, 14273, 11592, 28748, 28749, 28750, 28751, 2769, 36433, 13142, 5334, 13143, 13144, 24537, 13145, 37467, 37468, 37469, 4701, 4702, 37466, 13153, 20194, 1124, 6887, 1006, 26224, 19193, 15482, 10876, 895}), 376: frozenset({4865, 25889, 2982, 37417, 27053, 28118, 36247, 28120, 28121, 13750, 7580}), 377: frozenset({20483, 34819, 12294, 34822, 12296, 3344, 25106, 25107, 20759, 36122, 671, 672, 673, 21538, 20260, 36134, 36135, 14254, 14255, 35888, 23985, 23986, 26931, 14260, 14259, 822, 18362, 7611, 11070, 37441, 5441, 35394, 35396, 35397, 35398, 35399, 35395, 35401, 7882, 35404, 10573, 35406, 10575, 1999, 35405, 10572, 24526, 4694, 8024, 4696, 2138, 10715, 1123, 3556, 20071, 19178, 8554, 3183, 16766}), 378: frozenset({37507, 26630, 13689, 2828, 7574, 6552, 18971, 16029, 37025, 37026, 37155, 37032, 37033, 1194, 30890, 30889, 37034, 18988, 37550, 35632, 1195, 2994, 2866, 8763, 25533, 13503, 13505, 13506, 18251, 20048, 2771, 16723, 1108, 35927, 35928, 35929, 35930, 35934, 35935, 35936, 4472, 30948, 30949, 37220, 24934, 37480, 37481, 18792, 37477, 37479, 37478, 30961, 9587, 6260, 6259, 13688, 18169, 1018, 3067}), 379: frozenset({28864, 15936, 34696, 34697, 30282, 33043, 32020, 33044, 33046, 33047, 33045, 33051, 30761, 18096, 16496, 18097, 33329, 30195}), 380: frozenset({18566, 6923, 19215, 19348, 3226, 9242, 9245, 4641, 4651, 7084, 19758, 27183, 27182, 3505, 3506, 4666, 4667, 13121, 4036, 4037, 4038, 13127, 13896, 13895, 13898, 13899, 13900, 13901, 13902, 13903, 13911, 21856, 24674, 24677, 27110, 23654, 26225, 4981, 26229, 7286, 10494}), 381: frozenset({27650, 20482, 19460, 16391, 19464, 19465, 19466, 26383, 26384, 26385, 5152, 21536, 7456, 20518, 20519, 20520, 20521, 1589, 26423, 25151, 35408, 35409, 35410, 10577, 10580, 35417, 35419, 35420, 8027, 35422, 35423, 35424, 35425, 8028, 1635, 1636, 8029, 12282, 20074, 2414, 8558, 34430, 34431, 34432, 34433, 24966, 34439, 34440, 34441, 34443, 24986, 17821, 19107, 676, 9130, 6835, 8890, 3516, 3519, 3521, 3522, 10700, 10701, 10702, 10704, 3557, 5094, 3559, 5096, 25065, 5097, 3834, 12283, 12284, 12285, 27646, 27647}), 382: frozenset({16896, 38405, 38406, 36615, 38408, 36616, 36617, 38409, 37132, 13579, 37133, 37131, 13582, 13583, 9500, 32293, 25897, 842, 850, 16724, 37224, 37225, 18801, 18802, 18803, 4474, 36481, 9093, 9094, 37258, 37259, 17294, 17298, 17301, 38299, 1184, 18850, 7843, 38308, 7842, 3754, 7595, 7597, 23221, 3000, 11453, 35521, 28098, 28099, 14289, 8409, 13800, 13804, 3054, 3055, 13809, 13810, 13811, 36598, 13815, 36600, 36601, 36602, 18173}), 383: frozenset({39041, 39043, 10119, 15242, 7180, 15247, 5778, 15251, 15252, 8341, 15253, 15254, 15250, 5781, 25760, 13736, 17966, 25786, 12090, 39039, 32984, 32985, 12761, 32987, 13276, 1887, 38495, 34404, 38500, 18156, 20461, 11506, 18164, 12153, 39038, 12799}), 384: frozenset({6912, 6790, 6791, 27018, 2767, 37456, 2768, 34834, 37458, 37457, 37455, 37459, 19095, 37460, 37463, 36506, 37464, 37461, 37462, 37465, 19163, 37470, 18275, 18276, 18277, 18279, 18280, 9585}), 385: frozenset({4608, 9988, 3589, 3590, 27017, 16521, 22539, 22540, 22541, 22542, 22546, 796, 11938, 11940, 20714, 2156, 7220, 6901}), 386: frozenset({16258, 18315, 18321, 30995, 17176, 1561, 22648, 17181, 4897, 4899, 4901, 26918, 18345, 2351, 24635, 7997, 3904, 7232, 8003, 24004, 21455, 2133, 14558, 7265, 36577, 36579, 36580, 36581, 36582, 26471, 4714, 20205, 22646, 16760, 2682, 16253, 16254}), 387: frozenset({24705, 10882, 24706, 35779, 35782, 24584, 21577, 20300, 20301, 24590, 20627, 9173, 11684, 3238, 3239, 11689, 11690, 11691, 11692, 11693, 11694, 6512, 21557, 18677, 37498, 16826, 4155}), 388: frozenset({23628, 9102, 37071, 25425, 3410, 35666, 6427, 4955, 7197, 4571, 29790, 3423, 545, 546, 3299, 18852, 37287, 31015, 31017, 37290, 23469, 37613, 8752, 26097, 38582}), 389: frozenset({26882, 11909, 8839, 8335, 7575, 16035, 3747, 7204, 13735, 5800, 37161, 37162, 13483, 13484, 25776, 1200, 12210, 30898, 1203, 20914, 1201, 22454, 22452, 10810, 955, 18879, 25798, 2118, 24775, 34255, 24015, 13656, 25048, 21606, 8169, 26218, 26219, 13418, 11501, 11503, 19967}), 390: frozenset({26953, 26957, 36239, 36240, 7250, 20115, 7464, 7469, 12528, 17523, 6004, 6005, 6006, 824, 4409, 14782, 9535}), 391: frozenset({707, 24839, 30471, 28103, 35529, 11019, 13326, 10318, 31868, 34965, 18074, 8155, 9564, 34333, 6178, 28261, 425, 33455, 33657, 5818, 5820, 31869}), 392: frozenset({21137, 2075, 22812, 22814, 29215, 31264, 34337, 31265, 31266, 31267, 25766, 13351, 22824, 34349, 16179, 12213, 8374, 15287, 15288, 34875, 20428, 6352, 7123, 27736, 22493, 22494, 22495, 22497, 22498, 14818, 9957, 9960, 34284, 34285, 35053, 5743, 5744, 5745, 5746, 5748, 16116, 5752}), 393: frozenset({18432, 6986, 10897, 24724, 24725, 18198, 4893, 23776, 23777, 23778, 23779, 23780, 23781, 12454, 10215, 30632, 25005, 25007, 3248, 20274, 8826, 8058}), 394: frozenset({2373, 27020, 29773, 6478, 6479, 17365, 21849, 21850, 21503, 1568, 5154, 6819, 5155, 19755, 19756, 20781, 4654, 26287, 26285, 26288, 4658, 18547, 884, 16319, 4031}), 395: frozenset({24128, 24132, 1354, 26827, 3664, 3665, 19989, 7001, 8858, 2041, 18207, 23846, 5481, 11434, 8809, 17198, 17199, 17200, 8889, 1981}), 396: frozenset({34816, 27648, 27016, 24535, 24536, 24985, 19106, 8552, 9129, 3560, 8559, 18543, 34813, 4594, 1588, 6837, 6836, 4598, 27643, 27644, 27645, 3839}), 397: frozenset({13700, 26628, 15814, 26631, 26632, 26633, 13515, 8076, 15820, 2830, 13519, 17872, 38351, 17871, 26907, 35931, 35932, 24937, 30891, 6445, 5421, 30958, 30959, 30960, 4402, 23537, 14193, 23539, 23538}), 398: frozenset({32897, 21766, 18942, 20744, 6858, 6859, 5712, 7958, 7959, 32895, 1053, 21793, 3173, 5734, 5735, 22441, 12074, 19881, 32941, 32943, 22450, 20219, 21054, 3007}), 399: frozenset({8560, 4601, 6839, 29640, 24681, 24987, 4600}), 400: frozenset({8384, 8135, 713, 28217, 22481, 22482, 6355, 12696, 30492, 5852, 12702, 5853, 18082, 33060, 35065, 22822, 16103, 10745, 1532, 31359}), 401: frozenset({18817, 18818, 23435, 22801, 8467, 5652, 11541, 34680, 11799, 12698, 12699, 27676, 27677, 24478, 22816, 22817, 30755, 18732, 33325, 33326, 25136, 1716, 28858, 18113, 30278, 5959, 5960, 12617, 9544, 21581, 28368, 28369, 9691, 17756, 30817, 14820, 29160, 9323, 16495, 30191, 34672, 30193, 34673, 38643, 1524, 35061, 34679, 34677, 34674, 31354, 31355, 34675, 34676, 6143}), 402: frozenset({15361, 4610, 15363, 4609, 4613, 25641, 2807, 13140, 7221, 2646, 6903, 19766, 15358, 15359}), 403: frozenset({20290, 11718, 11719, 10888, 24713, 4745, 4747, 1034, 4746, 24715, 3343, 11722, 11723, 1361, 1362, 1363, 11730, 2774, 1366, 5084, 16352, 16353, 7009, 7008, 16615, 37496, 23738}), 404: frozenset({4576, 4577, 4578, 18725, 37323, 37324, 24493, 9105, 19956, 1501}), 405: frozenset({30739, 30740, 2076, 22813, 34334, 34338, 31269, 22823, 10744, 31273, 34352, 9012, 34360, 23354, 34364, 5951, 5952, 25666, 12612, 34372, 12621, 28239, 24657, 33130, 28273, 32369, 18811, 18812, 23431, 29839, 12693, 12703, 25248, 18083, 11949, 33458, 8373, 15286, 18102, 19899, 8387, 10437, 8134, 712, 714, 20943, 22479, 6353, 22480, 22483, 25814, 22488, 22489, 22490, 34278, 34281, 34282, 34283, 35054, 35055, 35056, 8436, 11512, 35066}), 406: frozenset({12098, 39111, 5769, 11913, 39115, 25046, 10008, 24792, 17818, 16729, 14184, 20906, 25771, 24940, 3757, 33007, 33008, 33009, 37241, 37243, 25790}), 407: frozenset({12544, 23552, 23554, 12546, 12545, 17669, 17670, 35589, 23553, 17675, 17676, 6672, 22033, 22034, 24351, 31781, 4394, 4395, 32299, 32300, 16686, 7478, 16706, 19011, 19012, 19010, 24901, 14921, 14922, 25428, 25429, 36448, 31597, 31598, 31599, 31600, 31601, 31602, 9073, 31604, 31605, 31606, 31607, 31608, 31609, 31610, 31611, 31612, 36479, 36480, 38804, 9383, 29353, 938, 15788, 15806, 2762, 14798, 9439, 18145, 26866, 13555, 13556, 24571, 23550, 23551}), 408: frozenset({20672, 2754, 16070, 1292, 1293, 28175, 18641, 3644, 5476, 23079, 11881, 300, 13165, 6766, 16052, 30074, 17212, 30078, 11903}), 409: frozenset({17796, 24390, 24391, 9224, 24395, 24396, 7630, 24401, 5458, 24402, 24679, 19436, 19437, 12271, 29629, 29630, 29631}), 410: frozenset({18114, 18819, 18820, 33038, 33040, 5651, 21717, 23318, 30491, 7774, 15907, 17127, 17773, 5357, 21684, 34678, 34681, 34682, 31356, 31357}), 411: frozenset({34307, 34308, 34309, 34312, 34313, 34314, 38794, 22991, 22992, 30741, 6230, 30743, 30744, 20718, 14386, 18105, 34299}), 412: frozenset({20293, 11717, 7304, 24714, 4107, 11724, 11725, 11726, 36091, 1364, 664, 5081, 5082, 20065, 24996, 18022, 9192, 8810, 28076, 36077, 1330, 36090, 23739, 36092}), 413: frozenset({23810, 18439, 2312, 23818, 1035, 23819, 23820, 23821, 1295, 1296, 28177, 28178, 19218, 19225, 8222, 8231, 21553, 6962, 6963, 5178, 35899, 17210, 17213, 23870, 19776, 19777, 585, 14666, 14671, 18768, 4181, 5466, 6495, 30053, 30054, 30055, 30056, 8297, 8298, 30057, 13162, 13161, 30058, 30061, 30059, 30060, 30064, 36467, 30065, 8309, 30066, 30062, 11896, 12410, 12411, 30082, 13964, 21901, 21902, 10895, 10896, 21906, 21907, 1690, 16794, 30619, 16795, 25501, 35487, 35488, 5029, 7334, 5031, 12457, 12458, 30633, 12460, 7346, 5051, 5055, 1983, 2760, 25302, 23768, 23770, 23771, 10461, 23774, 23775, 12000, 23773, 2016, 2017, 36324, 23782, 23783, 23784, 3302, 2036, 3321, 4091, 18431}), 414: frozenset({1509, 33485, 33486, 25231, 33487, 23888, 29844, 22525}), 415: frozenset({2816, 9098, 38419, 38420, 19349, 535, 7194, 31005, 16541, 13601, 23464, 3388, 25662, 38337, 38338, 834, 3399, 11850, 30795, 844, 14540, 33102, 33103, 11084, 8411, 13410, 13411, 5865, 14185, 5867, 5866, 23918, 15226, 30973, 30974, 30975}), 416: frozenset({18178, 37062, 3209, 907, 37067, 530, 31640, 38488, 38489, 31645, 27048, 4586, 35692, 35703, 35697, 35700, 35701, 31670, 35702, 24379, 24380, 11837}), 417: frozenset({22656, 22657, 8000, 3011, 4677, 14025, 7625, 16203, 16204, 19351, 14557, 19744, 5986, 5987, 5990, 5991, 5992, 3822, 1071, 3825, 18356, 16761, 22650, 7996, 22655}), 418: frozenset({9217, 3207, 11849, 12812, 23953, 23954, 23955, 16537, 31641, 31643, 31642, 9215, 14174, 25439, 39137, 19953, 19954, 37307, 37310, 37311}), 419: frozenset({20808, 20042, 37580, 37581, 25358, 37582, 915, 7572, 27095, 13721, 13722, 9499, 13723, 13534, 13535, 23971, 38307, 1579, 6449, 5427, 38392, 25405}), 420: frozenset({6144, 18117, 11980, 25229, 34702, 34704, 34705, 31956, 23320, 27675, 2909, 27682, 15906, 30762, 2794, 16112, 30196, 28218, 29823}), 421: frozenset({1030, 1032, 9225, 9228, 3604, 21525, 9241, 23069, 5149, 21789, 20514, 23599, 21043, 1587, 568, 2361, 13886, 13887, 13888, 13889, 13890, 13891, 23872, 21579, 11085, 24397, 11087, 24400, 18772, 11092, 24404, 8022, 2137, 24667, 24412, 24671, 18528, 25445, 8549, 24680, 26729, 26730, 18539, 1643, 14703, 3441, 26739, 7285, 6778, 3455, 18304, 14721, 36482, 36483, 4994, 23936, 23937, 1921, 23938, 23941, 3223, 3224, 3228, 8608, 19362, 35495, 35496, 35497, 9395, 29622, 18615, 18616, 29628, 7869, 3775, 29633, 9158, 5071, 3549, 37864, 7659, 3578, 3579, 3580, 3581}), 422: frozenset({9096, 19853, 3537, 38421, 39063, 39064, 13466, 38427, 8410, 13404, 4963, 4964, 9125, 5864, 5047, 15224, 15225, 2106, 15227, 3389}), 423: frozenset({20739, 1028, 1031, 6936, 6937, 6938, 6939, 6940, 6941, 6942, 21796, 20780, 4653, 27181, 20525, 20782, 20784, 4659, 8757, 4663, 6458, 2110, 23873, 10818, 10819, 37444, 37445, 36164, 36165, 17993, 13129, 21067, 13130, 23884, 32334, 845, 32336, 11856, 26705, 6480, 26708, 4188, 20829, 94, 17247, 2148, 9573, 2407, 15470, 21870, 21871, 2162, 21875, 15476, 883, 885, 17527, 18551, 18554, 18555, 1159, 1160, 1161, 30600, 1163, 30601, 1167, 1935, 18581, 16022, 18582, 37791, 6824, 26284, 11436, 11437, 175, 176, 3761, 26292, 11449, 2752, 9413, 17353, 14028, 17358, 17359, 9425, 17361, 17363, 17364, 16854, 16855, 16858, 37596, 36327, 39145}), 424: frozenset({10116, 32004, 32005, 30539, 9359, 24787, 24788, 29209, 29210, 32988, 32989, 18846, 32990, 32991, 32993, 32994, 32996, 30629, 5353, 29868, 6132, 17782, 11514, 25787, 12605, 8254}), 425: frozenset({292, 35580, 294, 12524, 35581, 21598}), 426: frozenset({33024, 33025, 33026, 33027, 27729, 27730, 27731, 34997, 14108, 15933, 33023}), 427: frozenset({25794, 6277, 11911, 22985, 2954, 2955, 6221, 15952, 32977, 32978, 8340, 32980, 24791, 8346, 33765, 33003, 38123, 33517, 33516, 19122, 25785, 5114}), 428: frozenset({17024, 26241, 26242, 26243, 36484, 26247, 2825, 2826, 26251, 2827, 26252, 27035, 23217, 7090, 7089, 23228, 1858, 38853, 38854, 36935, 36934, 38857, 38858, 29387, 38860, 6732, 38859, 38863, 35918, 38862, 35920, 35919, 35922, 14293, 35921, 14292, 14298, 35915, 1884, 35916, 39007, 35917, 21352, 29801, 26857, 29802, 36937, 6731, 26234, 17275, 26236, 17277}), 429: frozenset({12614, 12615, 15240, 21607, 16108, 29871, 35057, 11956, 12695, 31352, 31353, 27674, 5790}), 430: frozenset({17793, 2309, 5517, 5518, 9237, 19222, 21531, 27294, 27295, 17824, 3232, 5028, 18213, 9255, 16433, 16434, 30005, 30006, 30007, 25016, 14139, 18492, 30018, 30019, 30020, 30021, 25546, 30028, 30029, 30030, 30031, 30034, 12246, 12247, 2135, 12249, 12250, 12251, 8796, 24670, 26719, 20832, 24689, 7282, 26740, 24693, 2295, 1919}), 431: frozenset({12225, 12226, 37189, 37190, 37191, 967, 12138, 38573, 38574, 6424, 37084}), 432: frozenset({35456, 35457, 14085, 35851, 24844, 8077, 33678, 33679, 33680, 8852, 32405, 33686, 32406, 18070, 23602, 23861, 23862, 35510, 9018, 27707, 27708, 27709, 19903, 36420, 3155, 33623, 23127, 26460, 17504, 17760, 17506, 34408, 1513, 15336, 17913}), 433: frozenset({17280, 26248, 26249, 26250, 38856, 38861, 26255, 29391, 38864, 20248, 27037, 39015, 38247, 29804, 10491, 24049, 29812, 20661, 503, 504, 505, 23227, 17276, 17278, 17023}), 434: frozenset({513, 17026, 7555, 31750, 23174, 17288, 36231, 26256, 14742, 14743, 14744, 14745, 14746, 18874, 35453, 20166, 6733, 14286, 11344, 38865, 38866, 11345, 14288, 38875, 26844, 38876, 38878, 38879, 21345, 28274, 29813, 17270, 1014, 508, 509, 35454}), 435: frozenset({23808, 18437, 3334, 18438, 18440, 18441, 2311, 18954, 3340, 1042, 20256, 23842, 23843, 23844, 26925, 10549, 20285, 1609, 3657, 24651, 14668, 14669, 14670, 24139, 593, 14674, 21079, 26208, 30063, 8308, 27253, 18040, 8313, 13185, 13186, 13187, 17551, 17552, 21142, 12455, 12456, 30634, 12461, 12462, 12463, 3249, 1458, 8629, 20669, 20670, 25542, 9168, 9169, 23769, 11995, 23772, 996, 23785, 23786, 2032, 2033, 23795, 23796, 23797, 23798, 23799, 23800, 5625, 23805, 18430, 23807}), 436: frozenset({9488, 8728, 5400, 5416, 32559, 13364, 14399, 14400, 35906, 35909, 35910, 35911, 14408, 35912, 35914, 13393, 29778, 35923, 35924, 29779, 39001, 21339, 39005, 21343, 17248, 17249, 17250, 39010, 39009, 38245, 39013, 39012, 38246, 17259, 29806, 29807, 21361, 29810, 29811, 15986, 15989, 15988, 17271, 17272, 28275, 17025, 21499, 26000, 17050, 17051, 17052, 1959, 23209, 23212, 25522, 7091, 11443, 11442, 18875, 18877, 38598, 38855, 23238, 14283, 38868, 38869, 38870, 38871, 10463, 24046, 24048, 24050, 24051, 507}), 437: frozenset({10755, 33671, 33672, 3154, 15507, 32403, 23126, 34912, 5345, 17507, 14116, 1512, 37865, 36785, 434, 7803, 5183}), 438: frozenset({18176, 18243, 13447, 37321, 37903, 527, 852, 36215, 39128, 4573}), 439: frozenset({23424, 18115, 18821, 1990, 30535, 3146, 27727, 27728, 6675, 28884, 28885, 28886, 28890, 16187, 34683, 20452, 29606, 17918, 33322, 33708, 8687, 19891, 25140, 10740, 25654, 33014, 8379, 34684, 22589, 22590, 22591}), 440: frozenset({15360, 15362, 4611, 15364, 4612, 21510, 15367, 15370, 21774, 784, 785, 22545, 787, 9493, 24989, 24990, 23329, 23330, 30782, 30783, 30785, 13137, 13138, 13139, 15448, 19161, 18265, 15450, 4698, 15451, 14172, 18266, 15454, 15456, 18267, 18268, 5209, 15455, 15458, 8567, 2808, 8569, 4607}), 441: frozenset({13128, 21066, 23948, 26706, 26707, 9235, 2772, 6934, 6933, 4902, 21866, 21867, 21868, 17649, 13495, 13496, 13497, 13498}), 442: frozenset({3456, 3457, 23939, 3910, 28302, 3024, 18771, 9239, 9240, 15642, 24668, 9246, 4894, 3551, 31782, 26728, 26731, 18540, 26733, 26734, 26735, 26736, 18541, 7283, 17076, 18303, 18617, 18618, 29627, 2364, 15807}), 443: frozenset({1153, 1154, 1155, 1156, 25862, 1158, 26887, 7561, 23949, 23950, 23951, 20754, 16018, 38552, 38553, 31780, 6439, 17707, 25387, 17708, 17709, 17710, 38447, 17712, 17713, 38446, 17711, 8756, 2102, 19000, 38335, 16575, 38602, 38604, 38605, 38479, 38481, 17624, 17625, 17626, 17627, 860, 17628, 23386, 34401, 98, 25442, 21858, 9443, 17640, 17641, 17642, 17643, 17644, 13678, 13679, 12144, 14834, 9590, 118, 14200, 31614}), 444: frozenset({1794, 34509, 34510, 35023, 26064, 10129, 1809, 24857, 24858, 31258, 16477, 20133, 33261, 33263, 23345, 34353, 34483, 9969, 16114, 23356}), 445: frozenset({29376, 12803, 25412, 8837, 11916, 11917, 39116, 25045, 8351, 7845, 29864, 24809, 39212, 29869, 20908, 39215, 29870, 38317, 33006, 7603, 4476, 29372, 37245, 831}), 446: frozenset({8257, 33037, 33039, 6354, 25813, 5848, 6681, 5849, 5851, 18079, 27681, 33059, 30757, 24489, 428, 18098, 35059, 35063, 15417, 31358}), 447: frozenset({18337, 18338, 22436, 20901, 15237, 16519, 5448, 12073, 32936, 32942, 22447, 12079, 22426, 27134, 21823}), 448: frozenset({20130, 23475, 16470, 32394, 2574, 29151}), 449: frozenset({30624, 34913, 34914, 35459, 17096, 33673, 33674, 18809, 23436, 27699, 35539, 33013, 30422, 35545, 26461}), 450: frozenset({22528, 28227, 31942, 30421, 9049, 410, 32029, 1502, 14112, 26976, 28265, 32042, 6699, 32043, 32044, 28790, 18807, 28792, 22527}), 451: frozenset({24385, 37448, 37065, 906, 13451, 13452, 37066, 37450, 37449, 37064, 37325, 37199, 37328, 37327, 23421, 35670, 35671, 9112, 35673, 35675, 35676, 35553, 4584, 110, 38579, 24630, 26876, 28093}), 452: frozenset({34962, 11011, 31950}), 453: frozenset({32404, 36246, 727, 31900, 8270, 17503}), 454: frozenset({14208, 12037, 12039, 4238, 8590, 18515, 7700, 18516, 18517, 23064, 18520, 35354, 35355, 18521, 3426, 36644, 15589, 15590, 3432, 27177, 2152, 21042, 36659, 21754, 2365, 13884, 13885, 20734}), 455: frozenset({8835, 2579, 13338, 13340, 36769, 33703, 24872, 33710, 15919, 33073, 33075, 23347, 25655, 28855, 22587, 22588, 31803, 22592, 6336, 34244, 25796, 34247, 34252, 34253, 34254, 28879, 8273, 28888, 28889, 6105, 24802, 24803, 17916, 9342}), 456: frozenset({19009, 1602, 10245, 6666, 14796, 22029, 6671, 8083, 24567, 5405, 8287, 17699, 31592, 104, 21740, 3117, 31596, 10222, 10223, 7475, 7479, 26940, 12542, 12543}), 457: frozenset({14464, 15233, 15234, 15235, 9993, 21771, 16524, 21070, 9427, 10849, 27114, 22443, 27115, 22445, 6832, 27185, 27186, 6833, 3003}), 458: frozenset({6787, 14609, 26770, 26771, 14610, 12437, 26773, 26769, 19224, 8220, 7328, 8230, 28200, 6953, 28202, 30636, 24115, 20276, 8244, 8634, 8635, 19772, 27204, 27205, 3654, 3652, 3655, 3653, 35528, 19789, 5464, 18649, 2008, 18651, 18652, 25569, 8293, 6759, 6760, 3049, 24170, 14700, 28013, 14701, 18416, 18417, 18418, 18419}), 459: frozenset({4547, 25860, 4548, 4549, 38540, 38541, 38542, 38543, 36300, 32274, 32275, 32276, 7189, 32278, 32277, 16536, 24918, 36891, 36892, 7850, 35946, 35947, 30445, 4401, 35954, 14196, 20469, 17974, 35960, 30969}), 460: frozenset({2572, 21138, 13334, 2078, 32030, 24864, 22522, 24867, 18089, 5802, 31274, 32045, 19888, 31284, 16181, 28214, 16183, 8377, 1596, 9025, 34888, 22729, 34889, 15567, 33489, 19922, 15571, 15570, 29144, 9050, 30556, 30824, 32371, 32372, 32373, 32374, 32375, 18805, 28793, 21626, 22523, 22524, 32383}), 461: frozenset({35584, 35585, 35586, 35587, 10241, 35590, 35591, 35592, 35593, 14795, 14797, 24349, 24350, 17662, 293, 295, 9576, 12526, 7474, 7477, 12539, 12540, 12541, 9534, 17663}), 462: frozenset({7680, 7681, 7682, 3459, 3458, 1285, 3597, 3091, 30613, 20502, 35353, 23067, 36660, 36661, 36662, 21047, 566, 18619, 2363, 36671, 7875, 25550, 25551, 4049, 25554, 3427, 1253, 3433, 3434, 26732, 18542, 36472}), 463: frozenset({14880, 29377, 34269, 24808, 18217, 11914, 16692, 34264, 34265, 34266, 34267, 14877}), 464: frozenset({31360, 6145, 6146, 17784, 34698, 34699, 12619, 12620, 29842, 22484, 22485, 22487, 27680, 34336, 29217, 33122, 34276, 34277, 34279, 13352, 13353, 33069, 33070, 10991, 6707, 16115, 17783, 28216}), 465: frozenset({37504, 7297, 7298, 640, 24704, 24582, 1033, 639, 12177, 1691, 11677, 11678, 3106, 8867, 8612, 8868, 8614, 8613, 8615, 11682, 20650, 11683, 8617, 23091, 21556, 4149, 4150, 4151, 24116, 12474, 12475, 4154, 12477, 12479, 12480, 5058, 27203, 5059, 21576, 587, 1612, 588, 20302, 14685, 24169, 24171, 24172, 24173, 24174, 24175, 11884, 10867, 10868, 8318, 37503}), 466: frozenset({13184, 18434, 18436, 18442, 12174, 18447, 1299, 10899, 18199, 24993, 35493, 35494, 13993, 30635, 24113, 21554, 24114, 5053, 26825, 26826, 14672, 14673, 36178, 24160, 997, 2030, 2031, 23791, 23792, 10864, 122, 23804}), 467: frozenset({33152, 11906, 33159, 37896, 37897, 6422, 6423, 24473, 24474, 38569, 38570, 26033, 15540, 13749, 12219, 37183, 37185, 966, 32326, 30544, 10833, 37074, 37077, 1494, 37079, 20706, 14179, 16487, 8173, 3955, 3956, 11518}), 468: frozenset({24896, 22987, 24790, 24798, 24799, 10082, 18149, 32999, 33000, 12841, 12842, 25131, 33001, 34226, 34227, 34228, 33524, 34230, 20724, 34232, 34233, 34234, 34235, 18108, 33525, 33526}), 469: frozenset({33696, 6689, 29153, 15555, 28932, 28933, 28934, 15559, 26977, 16972, 22735, 22736, 7261, 15549, 33695}), 470: frozenset({18435, 24581, 2313, 18444, 18445, 18446, 4109, 4110, 18449, 18450, 18451, 18452, 3341, 26895, 4143, 4145, 4146, 4147, 4148, 17203, 17204, 17205, 20282, 6481, 14676, 24152, 24153, 24154, 24155, 24156, 24157, 24158, 14681, 14682, 24161, 24162, 24163, 24164, 24165, 24166, 24167, 24168, 24176, 24177, 13174, 13177, 8316, 637, 8317, 638, 6524, 14683, 20634, 20635, 16797, 35486, 8865, 8866, 14686, 35492, 5030, 7336, 20649, 13994, 13995, 7347, 12469, 12470, 12471, 12472, 4281, 4282, 12473, 12476, 1982, 5056, 5057, 974, 11733, 11734, 11735, 17241, 11736, 11738, 17242, 8670, 8671, 11749, 998}), 471: frozenset({34240, 34241, 34658, 24804, 25380, 29092, 21131, 10348, 28365, 6095, 21713, 33074, 2900, 8347, 34238}), 472: frozenset({35907, 35908, 36485, 13391, 13392, 35455, 13394, 35925, 16534, 35926, 857, 37018, 14297, 26280, 7849, 6573, 7086, 6256, 7092, 30965, 4541, 4542, 11839}), 473: frozenset({10881, 11779, 33161, 9104, 33169, 20887, 37277, 5662, 542, 5664, 26017, 5665, 37279, 8740, 5661, 37278, 5663, 5672, 37293, 37294, 37295, 37296, 37297, 39090, 39089, 37300, 37299, 37298, 5947, 5948, 36158, 22079, 22080, 22081, 11842, 7246, 4559, 4560, 20050, 3414, 39127, 39126, 4956, 8414, 19951, 19952, 1913, 33148, 18174}), 474: frozenset({19331, 10115, 28361, 8458, 28362, 29201, 11797, 23574, 23575, 23577, 14875, 14876, 27551, 27552, 33317, 16690, 3253, 29371, 8253}), 475: frozenset({20485, 3845, 3846, 14474, 14475, 14476, 14477, 14478, 14479, 20761, 19482, 19100, 19102, 17823, 34849, 34851, 34861, 34862, 34863, 34864, 21808, 34867, 21813, 21814, 21815, 3895, 11575, 11576, 11579, 11580, 11581, 24639, 14271, 7617, 11073, 14019, 196, 5445, 18384, 5330, 27611, 10721, 10722, 27620, 10725, 27622, 27621, 10724, 1125, 4711, 27629, 35575, 13944, 7289, 8444}), 476: frozenset({10756, 20934, 24202, 24203, 8142, 5199, 21136, 37907, 8084, 33173, 35543, 35544, 35547, 34915, 23269, 37414, 34917, 36787, 36788, 36789, 27702, 17911, 14781}), 477: frozenset({10885, 10886, 7303, 13191, 4750, 4111, 23822, 23827, 23828, 23829, 19991, 18200, 8859, 8860, 8864, 24994, 8232, 17201, 8885, 25918, 17473, 24130, 19397, 5062, 11720, 11721, 24138, 5066, 24140, 25933, 24141, 1360, 19792, 9171, 24148, 24149, 1365, 1619, 1369, 7002, 3675, 7003, 24159, 3683, 16356, 18023, 14440, 37866, 37867, 16375, 16376, 2042, 5629}), 478: frozenset({17920, 23427, 33028, 33029, 33030, 33031, 33032, 2058, 34700, 32013, 34701, 14095, 29840, 29841, 29204, 11798, 23319, 33048, 33049, 33050, 11800, 33052, 14109, 33053, 33054, 33055, 27673, 418, 23585, 23586, 34332, 29608, 6696, 24874, 18092, 18093, 33709, 25135, 33331, 34996, 21685, 8381, 28863, 8383, 17225, 30794, 30283, 3147, 25810, 21716, 31957, 22486, 28887, 28894, 7775, 30818, 23139, 33018, 28268, 30189, 2547, 6138, 33020}), 479: frozenset({11016, 4879, 34322, 23315, 34324, 2580, 4883, 34327, 34328, 34326, 34329, 34330, 30748, 30749, 30750, 2084, 5929, 30764, 32050, 3126, 31804, 33604, 28231, 30536, 30537, 9051, 34653, 34654, 34655, 34656, 34657, 34659, 10355, 10358, 23429, 34708, 34325, 18843, 8348, 417, 29091, 29605, 29094, 29095, 29093, 29862, 33704, 33707, 29607, 8125, 18110, 18109, 26051, 8388, 34245, 34246, 34778, 6111, 23264, 17120, 2795, 14829, 4882, 14077}), 480: frozenset({14787, 6534, 38413, 8206, 38414, 37454, 26643, 24659, 3284, 37589, 16726, 37590, 17878, 6556, 26653, 26654, 26655, 18720, 14760, 14761, 35564, 7855, 14195, 4405, 24952, 20030}), 481: frozenset({18182, 14534, 14536, 9097, 17496, 537, 38425, 19866, 12059, 39069, 39070, 20893, 7974, 15206, 7976, 15208, 15209, 15210, 15211, 15212, 15213, 12207, 5742, 5741, 22390, 22393}), 482: frozenset({4545, 23235, 10495, 38537, 38539, 19195, 7188, 26263, 14296, 36890, 26275, 26276, 26277, 26278, 26281, 9581, 21357, 21493, 30966, 1016, 23417, 20026, 18171, 10814, 23231}), 483: frozenset({4480, 10496, 3074, 37122, 13572, 13573, 13574, 9092, 38544, 2833, 2839, 36893, 18847, 35248, 17975, 1977, 10815, 14913, 15809, 4550, 4551, 36299, 36301, 36304, 36305, 4184, 14303, 14305, 29795, 19943, 35562, 35953, 1910, 35959, 21368}), 484: frozenset({528, 9108, 31644, 31646, 35704, 37304, 37049, 37050, 37051, 37309, 37055, 11848, 35674, 4957, 39135, 39136, 8417, 37471, 4579, 4580, 24033, 107, 18925, 16889, 35698, 16888, 35705, 35707, 35708, 35709, 35710, 35711}), 485: frozenset({23105, 6339, 36228, 9354, 24843, 24204, 33676, 33675, 10381, 5200, 20885, 16987, 6684, 31901, 5346, 34916, 34918, 34919, 19304, 15338, 15339, 36790, 27703, 27704}), 486: frozenset({8451, 22791, 34503, 22793, 34504, 35016, 34505, 26959, 1808, 26960, 25234, 24660, 9557, 24856, 23131, 37411, 29158, 20142, 1791}), 487: frozenset({19969, 38532, 38533, 3212, 25874, 23956, 23960, 25116, 550, 38580, 948, 15811, 12228, 12229, 12230, 37063, 12233, 12235, 4940, 12237, 37068, 4941, 3291, 37475, 14186, 35691, 35693, 35694, 35695, 8176, 35696, 9586, 35699, 16887}), 489: frozenset({38404, 17860, 37255, 37256, 37257, 24331, 37134, 13584, 37137, 16733, 38302, 37795, 18851, 32295, 32296, 7594, 13802, 36588, 7853, 13807, 24947, 2999}), 490: frozenset({7881, 17069, 21840, 34835, 34836, 34453, 34838, 34454, 19096, 14166, 27642, 18941, 1215}), 491: frozenset({37123, 10500, 3079, 38410, 38546, 2840, 9497, 2841, 2842, 2843, 2844, 26658, 26659, 32305, 32306, 32307, 17976, 20027, 36302, 36303, 36306, 36307, 36308, 36309, 36310, 17751, 3287, 36313, 36314, 17752, 36318, 14304, 8425, 12784, 18934, 21369, 21370, 18172}), 492: frozenset({37636, 37637, 37638, 26886, 7560, 13706, 13707, 13709, 13710, 37651, 25877, 534, 31766, 7705, 7709, 3744, 23974, 26923, 16559, 39100, 14013, 31678, 31679, 31680, 31681, 36679, 8775, 27081, 36423, 27083, 38476, 38477, 27084, 26191, 26192, 38603, 20813, 27082, 26189, 25559, 38360, 38361, 1116, 23389, 37600, 97, 32329, 37603, 99, 37605, 37606, 13677, 8184}), 493: frozenset({36672, 36673, 37057, 37315, 37313, 31688, 31635, 15827, 31637, 31638, 23959, 31639, 23958, 9109, 25555, 39138, 4589, 947, 18166, 31672, 32315, 23422}), 494: frozenset({21248, 6177, 23106, 14117, 28232, 34920, 2541, 24205, 24206, 25242, 17789, 25822}), 495: frozenset({12035, 1289, 3595, 3596, 5136, 7697, 36626, 36627, 36628, 35348, 7702, 7703, 7704, 35347, 3092, 17827, 27173, 27174, 27175, 27178, 35756, 35757, 36654, 36655, 19376, 36657, 18610, 36658, 4013, 4014, 36663, 2362, 7868, 36669, 14142, 36670, 9154, 3907, 3017, 4050, 26195, 15588, 3429, 3430, 3431, 15592, 16231, 20211, 37494, 12024, 4991}), 496: frozenset({20288, 641, 5060, 12171, 8332, 18706, 18707, 1618, 17470, 35484, 35485, 13981, 17441, 5484, 28077, 28080, 1333, 1334, 35895, 24181, 3774, 17471}), 497: frozenset({15168, 15169, 31429, 31430, 10248, 10253, 32600, 32601, 18141, 21737, 7470, 6395, 5680, 5681, 8115, 8120, 33146, 18235, 38526}), 498: frozenset({643, 16393, 658, 660, 17432, 35482, 35483, 19229, 17438, 1571, 20261, 20262, 8870, 17063, 8875, 28078, 26417, 28081, 26932, 1336, 16062, 26430, 26432, 26433, 8898, 3523, 3524, 26434, 4804, 25921, 20296, 20297, 8906, 4803, 5073, 1626, 26210, 26211, 2663, 16359, 21097, 7657, 36085, 36086, 8319}), 499: frozenset({34242, 6338, 35078, 33606, 19923, 18842, 30751, 30752, 14815, 24800, 34660, 5925, 34661, 34662, 36776, 5930, 8124, 10349, 30446, 18111, 9015, 34236, 34237, 34239}), 500: frozenset({12040, 12813, 12814, 12050, 21011, 30998, 31000, 31001, 7717, 8745, 37418, 6447, 3376, 3377, 17718, 2618, 18242, 36678, 36680, 4681, 36681, 2635, 36682, 26190, 4942, 4950, 38492, 38493, 4958, 14180, 24938, 12147, 25972, 23413, 14197, 4473, 9087, 13455, 7568, 13456, 13457, 16019, 30868, 13458, 2996, 31671, 31673, 16828, 39101, 25536, 37314, 37058, 31685, 31686, 31687, 12234, 12236, 12238, 13520, 12240, 13521, 13523, 13524, 25560, 39139, 19940, 3301, 4590, 4591, 26870, 8183, 8188, 3069}), 501: frozenset({1922, 29635, 30596, 29636, 10891, 3019, 3603, 29652, 3230, 9248, 9251, 17828, 4599, 29991, 29995, 29996, 19763, 6838, 1590, 7288, 7290, 9403}), 502: frozenset({3395, 22391, 38423, 38424, 19865, 12121, 39067, 39068, 20891, 20892, 12067, 26026, 22383, 22384, 7857, 22385, 22387, 22388, 22389, 35511, 22392, 22394}), 503: frozenset({17664, 8193, 14920, 17679, 272, 17687, 36248, 17688, 17693, 9501, 18723, 102, 24553, 36907, 9072, 6002, 6003, 21628}), 504: frozenset({24897, 33605, 12872, 8460, 33615, 34320, 34321, 33616, 30742, 5922, 5923, 15461, 5926, 34663, 34666, 34667, 34295, 33532, 34301, 34302, 34303}), 505: frozenset({18112, 34243, 35077, 33608, 34318, 34319, 23316, 30746, 5341, 17119, 30753, 36770, 21281, 931, 24801, 34664, 33385, 33387, 34668, 32052, 34229, 34231, 33598, 33599}), 506: frozenset({19330, 10244, 35525, 10247, 7241, 273, 14738, 14867, 14868, 14869, 30418, 3929, 38816, 14182, 31594, 3118, 12529, 17524, 4920, 31865, 12538}), 507: frozenset({7168, 28355, 3140, 21701, 25990, 24838, 31947, 31948, 31949, 34961, 5202, 34963, 37332, 20436, 23257, 6176, 11425, 33451, 33452, 33453, 5803, 27885, 27886, 17904, 15922, 33655, 7799, 33656, 5049, 33658}), 508: frozenset({17794, 12036, 17162, 20490, 1038, 17426, 17427, 7699, 36633, 36634, 36635, 36636, 21532, 36637, 36638, 36639, 8880, 2353, 18493, 18494, 14021, 8006, 5454, 3032, 19419, 19421, 19422, 12253, 12254, 21857, 25443, 26992, 2296}), 509: frozenset({9575, 6025, 12523, 7468, 12525, 7481, 35578, 35579, 826}), 510: frozenset({12162, 13827, 25410, 39114, 8208, 849, 5783, 16728, 5791, 5792, 21282, 25129, 23403, 5934, 20910, 5935, 36213, 37239, 37240}), 511: frozenset({8892, 8902, 14024, 25097, 25098, 19467, 24717, 654, 29645, 14161, 4691, 5086, 3040, 21537, 23743, 26403, 9252, 20522, 14251, 24683, 30001, 26431, 18358, 24183, 7291, 24636, 8895}), 512: frozenset({7169, 11971, 18884, 16971, 22731, 8267, 31951, 31952, 31953, 17170, 17171, 34964, 23124, 14102, 37333, 28795, 8365, 5805, 5806, 33456, 33454, 5810, 17905, 28796, 33659, 33660}), 513: frozenset({15552, 11012, 3141, 34970, 36168, 15560, 14732, 21293, 15923, 13299, 7800, 20057, 19706, 28799}), 514: frozenset({30464, 28930, 22732, 22734, 5811, 5812, 5813, 9335, 19705, 411, 28797, 28798}), 515: frozenset({35937, 35938, 27075, 13699, 26629, 35945, 13514, 971, 21362, 14299, 14012, 35933}), 516: frozenset({26624, 14912, 15808, 9091, 13571, 4546, 36486, 9487, 32273, 26898, 38545, 4945, 13396, 32279, 32280, 16535, 8408, 30967, 3293, 36894, 30968, 26279, 26874, 23408, 25521, 6257, 24375, 24376, 30970, 4479}), 517: frozenset({34817, 16771, 19092, 28184, 28185, 19103, 30626, 23984, 14257, 14258, 11058, 34865, 34814, 24631, 21817, 18363, 18364, 7612, 18365, 2759, 35400, 35402, 35403, 24524, 11088, 18257, 4051, 2644, 4052, 4695, 10713, 1634, 13154, 4710, 4967, 4712, 8553, 10731, 27628, 14705, 8563, 4595, 16119, 34812, 14718, 34815}), 518: frozenset({27649, 27651, 21508, 21509, 26391, 9249, 20523, 19762, 821, 21048, 26424, 3897, 14145, 18255, 18256, 18258, 2643, 4692, 4693, 1621, 1625, 2154, 24682, 8556, 36461, 36462, 8557, 18287, 8561, 8562, 4602, 2420, 34427, 16765, 1923, 7306, 19083, 8331, 24718, 19090, 19353, 3229, 34465, 34473, 34474, 34475, 3244, 17078, 18360, 18361, 7610, 18620, 18623, 23744, 23745, 23746, 29637, 29638, 29639, 29641, 29642, 29643, 29644, 19159, 10712, 10714, 27612, 5085, 5087, 3558, 25067, 21743, 4597, 34809, 34810, 34811, 3838}), 519: frozenset({6016, 6017, 6664, 10252, 15441, 6041, 22428, 22429, 22430, 22431, 22432, 24547, 25126, 7465, 3115, 5679, 5682, 10231, 825}), 520: frozenset({5127, 5128, 5129, 25099, 25100, 7696, 3600, 7698, 21777, 21524, 20501, 25109, 25111, 25110, 36629, 36630, 36631, 36632, 36640, 36641, 9250, 27176, 29992, 29993, 29994, 27180, 24621, 29997, 29998, 29999, 30000, 30004, 567, 4690, 18519, 18522, 14171, 18525, 18526, 18527, 25444, 26473, 16234, 24686, 36463, 24688, 24687, 18288, 18289, 18290, 13949, 4993, 19078, 4239, 8593, 8595, 8596, 8597, 8598, 8599, 3231, 35763, 35764, 35765, 35766, 35767, 35768, 35769, 7866, 35770, 24000, 24001, 33219, 9156, 9155, 25543, 10695, 201, 3018, 19420, 12252, 7658, 21755, 20735}), 521: frozenset({28800, 28929, 22786, 22787, 28931, 7173, 8453, 32392, 8845, 25233, 18072, 18073, 5804, 5807, 4530, 3123, 31290, 31291, 19907, 18887, 22728, 15561, 22730, 29767, 17489, 81, 33491, 33490, 29149, 29152, 17505, 30437, 1510, 17765, 30826}), 522: frozenset({29952, 29953, 26626, 29954, 8203, 10507, 10508, 35596, 35597, 7184, 18966, 25881, 35615, 35616, 35617, 6437, 2858, 2859, 4906, 302, 37425, 24377, 14914, 37443, 4418, 3066, 16722, 37210, 37216, 37218, 37219, 24932, 6258, 15475, 21363, 6261, 13684, 21624, 21625, 36474, 36475, 36476, 23930, 23931, 12159, 13697, 21122, 13698, 26500, 13702, 2440, 2441, 7566, 37020, 37021, 37022, 3743, 37024, 7839, 37029, 37031, 2993, 1970, 1976, 27072, 27074, 200, 13517, 3282, 3283, 10462, 30943, 30944, 30945, 30946, 30947, 14309, 30951, 14311, 8187, 15603, 15604, 15605, 29946, 29947, 1022}), 523: frozenset({20480, 16256, 18946, 27657, 27658, 18318, 18320, 20497, 30997, 5143, 30999, 21786, 6811, 25115, 18346, 3507, 23603, 16310, 16311, 16312, 13113, 11067, 32316, 25277, 10683, 28347, 8001, 14530, 19139, 11844, 10693, 4679, 21832, 21450, 16202, 16842, 21839, 2134, 5977, 5978, 7643, 5979, 5982, 5983, 5984, 3297, 2146, 10856, 3821, 21616, 25080, 4733}), 524: frozenset({12161, 36489, 36490, 8207, 7185, 26641, 32281, 3746, 1186, 15524, 1187, 30887, 1193, 11440, 1971, 39099, 25409, 39106, 14915, 13518, 36311, 36312, 25433, 13536, 25446, 24944, 15477, 23542, 23543, 13816, 36221, 13822, 13823}), 525: frozenset({22402, 34380, 25679, 6416, 6417, 24785, 19862, 19863, 37335, 20889, 32983, 1118, 5727, 27104, 12065, 5729, 5730, 5728, 12131, 38380, 38381, 38382, 25966}), 526: frozenset({16768, 17152, 6914, 6913, 6792, 6793, 27144, 6794, 17808, 28180, 9494, 19354, 15387, 15388, 24988, 15390, 24991, 15391, 15393, 19748, 21797, 36007, 36008, 36009, 36010, 28713, 36012, 28712, 36013, 28719, 28720, 28721, 36011, 17080, 14265, 19768, 12346, 7613, 11071, 18368, 18370, 25419, 18259, 2648, 4697, 19162, 4699, 4700, 30428, 18274, 18278, 16871, 8564, 2422, 8566, 8568, 36219, 6910}), 527: frozenset({19202, 26889, 26634, 36623, 26640, 26642, 26644, 26645, 18468, 21029, 21030, 17964, 7225, 8762, 2116, 20809, 1107, 37226, 37228, 37229, 12160, 26502, 914, 13818, 916, 13819, 6554, 16026, 13724, 30881, 1185, 30883, 1188, 30885, 23972, 1189, 30886, 1192, 38311, 38310, 962, 37574, 37575, 37578, 3274, 37579, 37584, 37585, 37586, 17875, 37588, 37587, 17873, 17874, 30882, 23540, 23541, 9206, 9207, 9204, 38394, 38395, 13820, 38397, 38398}), 528: frozenset({36608, 16897, 4481, 36610, 36612, 3076, 36609, 36611, 25986, 36618, 36619, 36620, 8205, 36622, 36621, 912, 38396, 32290, 32291, 32292, 7844, 23970, 38312, 25899, 7596, 29741, 27057, 32311, 7224, 39105, 37576, 37577, 20049, 17749, 17750, 3070, 3290, 36603, 25953, 37222, 37223, 18163, 13813, 13814, 36599, 12795, 36604, 36605, 36606}), 529: frozenset({9984, 9985, 4867, 4868, 32393, 33291, 7499, 33293, 33294, 33295, 31763, 7507, 26964, 598, 20120, 33242, 33243, 33245, 20126, 33247, 33249, 24869, 20137, 1790, 23472, 8689, 31796, 35509, 16184, 1789, 5950}), 530: frozenset({7141, 32966, 5797, 32969, 8428, 39184, 39186, 39187, 25395, 20757, 25047, 11864, 16025, 30877, 30878}), 531: frozenset({1924, 19367, 4042, 782, 786, 15379, 15380, 15381, 15382, 3159, 4603, 6905, 30779, 10111}), 532: frozenset({22721, 20989, 17861, 10216, 26505, 35689, 24333, 24334, 20113, 24337, 8851, 24338, 7480, 29082, 6043, 1021, 18142}), 533: frozenset({7491, 10246, 6663, 24906, 23548, 4557, 20110, 26965, 14870, 1499, 22684, 1500, 39209, 13546, 23915, 13547, 10224, 10225, 12794, 20470, 29367, 31866, 25979, 7484, 23549}), 534: frozenset({27136, 21515, 12056, 7961, 12314, 12058, 7962, 1052, 1057, 12066, 4642, 4643, 7973, 7975, 12071, 24634, 30778, 21053, 3902, 21055, 21824, 21825, 6471, 16199, 36426, 15182, 15187, 14164, 3160, 17498, 3169, 4962, 10852, 10853, 15207, 5736, 5737, 5740, 2165, 10107, 21884, 10108, 14207, 10112, 15238, 21895, 19850, 16525, 22417, 22419, 22420, 20890, 19870, 20903, 22446, 22448, 25777, 9138, 20915, 20916, 16832, 16578, 14535, 6856, 6860, 6861, 6862, 14562, 2804, 27127}), 535: frozenset({6018, 9987, 6020, 12550, 12551, 12552, 12553, 1801, 33292, 22669, 22670, 22671, 22672, 6673, 22668, 12564, 12565, 12566, 6039, 20116, 6040, 22427, 20124, 25756, 19872, 22692, 22693, 12072, 22709, 22710, 22711, 23864, 33081, 33082, 23865, 26942, 7487, 7488, 9537, 26946, 26947, 7492, 7504, 15442, 21623, 2540, 14067, 6007, 6008}), 536: frozenset({3201, 3202, 24325, 3077, 22025, 19339, 24332, 17677, 17678, 37135, 37136, 13585, 17682, 37139, 17683, 37138, 17684, 17685, 6032, 6033, 17686, 37140, 9502, 32294, 36902, 4393, 17979, 17980, 20029, 18240, 16705, 14918, 1892, 100, 1893, 19945, 18160, 24948, 17680, 9211, 17681}), 537: frozenset({38400, 24321, 24322, 38403, 24323, 24324, 37254, 24326, 9095, 24320, 37253, 19013, 14924, 31613, 8404, 37141, 38551, 3289, 16732, 19936, 13805, 32302, 32303, 24368, 7854, 13806, 12781, 23409, 24949, 18234, 17981}), 538: frozenset({38401, 8194, 38402, 22020, 22021, 28936, 28937, 6670, 10258, 24339, 10259, 24341, 12569, 11289, 5403, 4397, 32301, 24367, 16176, 4398, 7476, 26941, 26944, 3142, 19014, 19015, 14923, 25424, 35583, 25430, 26967, 1894, 35688, 35690, 25451, 30828, 25977, 21627, 6023, 6024, 6026, 8843, 20108, 14734, 38805, 920, 20121, 22685, 15521, 29354, 22722, 34500, 34501, 34502, 17863, 14794, 31434, 31436, 17869, 14799, 35549, 24550, 10217, 24557, 37617, 12533, 13558, 12535, 13557, 12534, 1786, 8446, 10239}), 539: frozenset({26112, 29313, 29314, 1793, 4361, 21003, 17163, 25888, 20132, 3749, 23978, 20141, 11053, 23598, 13744, 13745, 13746, 13747, 13748, 37178, 37179, 32578, 32579, 32580, 837, 32581, 31302, 36424, 36425, 74, 32586, 32584, 2121, 37072, 37073, 23507, 35677, 35680, 35681, 35682, 35685, 29157, 38504, 38505, 38506, 38507, 33259, 13673, 37614, 37615, 16883, 14709, 6390, 19959, 37882, 29307, 29308}), 540: frozenset({7618, 7619, 24196, 3394, 20486, 28295, 14533, 21897, 21898, 19851, 21517, 28431, 2129, 3538, 5715, 38422, 19864, 39066, 37340, 17501, 16542, 1056, 26464, 21609, 5739, 941, 35505, 22386, 35512, 33145, 15677}), 541: frozenset({11521, 6405, 17161, 526, 37904, 25872, 543, 8742, 31016, 2096, 2099, 24628, 37180, 32324, 32325, 1094, 11847, 23626, 23627, 76, 23629, 4939, 18767, 3412, 3415, 90, 91, 12125, 3422, 4959, 3424, 13672, 13675, 23420, 33155, 39053, 39054, 35469, 35470, 14733, 33171, 33175, 18853, 37288, 15530, 15531, 15532, 15533, 15534, 15535, 39088, 15536, 15537, 15538, 13491, 15539, 946, 37048, 12218, 956, 37059, 37318, 37319, 37320, 37075, 37076, 39124, 39125, 28119, 39129, 28122, 4572, 8413, 39140, 39141, 3300, 37862, 37292, 19950, 14833, 23028, 11519}), 542: frozenset({37891, 20750, 21007, 6425, 547, 8743, 37426, 37427, 37428, 24629, 36150, 25397, 36156, 2620, 833, 1095, 37194, 37195, 30543, 10834, 32602, 32603, 32604, 32605, 22366, 16735, 22368, 22367, 26467, 36214, 33150, 38527, 33151, 19844, 13448, 33165, 33166, 37263, 37264, 33167, 16014, 37265, 15518, 37794, 24227, 24228, 18854, 16039, 18857, 26029, 38575, 38576, 38577, 6396, 12223, 12224, 37060, 37061, 37322, 23506, 4574, 37087, 4575, 37089, 19938, 37860, 8174, 8433, 17139, 26100, 26099, 26102, 26101, 20988, 7678}), 543: frozenset({3392, 1091, 18181, 11852, 26878, 13591, 536, 39065, 13592, 13593, 12060, 13594, 13468, 31806, 37343, 4961, 25954, 25765, 3391, 15223, 15280, 8688, 15217, 33076, 33078, 3767, 29814, 953, 3390, 25657}), 544: frozenset({34387, 34388, 13622, 13595, 1100, 13469, 8750}), 545: frozenset({38528, 10498, 17156, 17158, 1805, 22687, 22688, 18726, 36136, 36137, 36138, 36139, 36140, 21294, 3120, 34868, 34869, 34484, 34485, 34486, 12086, 26938, 8118, 26554, 7485, 7486, 15167, 26949, 31431, 17224, 35022, 17108, 7260, 20445, 17502, 24546, 2538, 24558, 33262, 33264, 33265, 10226, 10227, 10228, 33266, 6012, 9983}), 546: frozenset({2049, 34947, 32009, 32015, 22803, 22804, 29206, 25753, 29212, 25247, 24224, 11427, 14119, 25127, 5038, 6706, 20926, 30276, 20940, 25811, 14812, 34783, 9568, 32992, 34402, 26468, 35046, 35047, 30185, 30186, 30830, 28272, 9971, 10739, 6133, 11513}), 547: frozenset({8838, 20998, 25866, 39185, 8339, 39189, 30876, 13734, 29866, 8747, 29867, 37163, 29872, 15667, 10809, 14016, 26879, 26440, 10825, 32971, 34382, 34384, 32981, 32982, 37336, 37337, 5726, 23392, 27107, 27108, 32359, 32362, 11500, 7669, 14203, 39166, 10879}), 548: frozenset({2048, 36227, 24197, 32006, 4364, 32014, 5653, 29205, 36768, 33701, 33702, 33323, 16186, 31805, 29375, 25795, 25797, 34248, 34249, 34250, 34251, 27723, 6093, 6094, 27724, 8272, 6103, 6104, 13274, 25050, 17758, 30816, 23138, 19306, 10347, 17781}), 549: frozenset({10497, 12802, 13826, 13828, 13829, 13830, 26891, 14879, 25130, 30507, 25901, 39214, 37166, 37165, 39213, 16691, 18232, 8764, 25411, 12099, 16711, 26441, 26443, 17228, 24908, 6224, 1109, 1898, 1899, 14201, 37242, 11912, 5779, 5780, 5782, 8343, 5793, 5794, 5801, 22442, 38314, 38315, 25773, 38316, 20909, 30899, 30902, 30903, 29373, 29374, 25789, 39112, 17097, 3285, 34268, 34781, 34784, 34785, 34787, 7142, 33002, 33004, 33005, 11502, 21487, 11504, 20465, 33527})}


In [2]:
mapping = {22721: 532, 20989: 532, 17861: 532, 10216: 532, 26505: 532, 35689: 532, 24333: 532, 24334: 532, 20113: 532, 24337: 532, 8851: 532, 24338: 532, 7480: 532, 29082: 532, 6043: 532, 1021: 532, 18142: 532, 19201: 349, 24578: 349, 6022: 349, 6665: 349, 6668: 349, 20751: 349, 19727: 349, 10257: 349, 24342: 349, 24344: 349, 11032: 349, 24346: 349, 37275: 349, 29084: 349, 29083: 349, 24348: 349, 29085: 349, 29086: 349, 29087: 349, 13608: 349, 13609: 349, 7467: 349, 18860: 349, 3116: 349, 7473: 349, 26939: 349, 17862: 349, 19016: 349, 31437: 349, 31438: 349, 36955: 349, 36956: 349, 25950: 349, 21738: 349, 12531: 349, 18932: 349, 12532: 349, 10236: 349, 10237: 349, 10238: 349, 8447: 349}


In [5]:
len(mapping)

58

In [6]:
# node_flows[target]["in"].add(node)
node_flows = {1: {"in":{4,5,6}, "out":{1,2,3}}}

In [ ]:
for part, flow in node_flows.items():
    for node in flow["out"]:
        self.mapping.pop(node)

1
{'in': {4, 5, 6}, 'out': {1, 2, 3}}


In [8]:
test = {}
test[1]=2
test[1]=4
test

{1: 4}

In [ ]:
for node in flow["out"]:
    self.mapping.pop(node)
for node in flow["in"]:
    self.mapping

In [3]:
part_ids = {part for part in mapping.values()}
districts = {part: set() for part in part_ids}

for key in mapping:
    value = mapping[key]
    districts[value].add(value)

In [4]:
len(districts)

2

In [1]:
neigh = {(8816, 17441, 24181, 20296, 8332, 4803, 26432), (16395, 17444, 37495, 2415), (28077, 17456, 18707, 17455), (3196, 5060, 17471, 1333), (8816, 17442, 17444, 8802), (1334, 1618, 28080, 7320), (17443, 2711, 644, 3526), (2415, 17444, 8802, 1614), (8816, 17441, 17443, 2711), (12287, 1614, 17431, 27024), (17473, 20262, 5484, 17470), (2415, 8321, 1614, 17437, 12287, 17433), (18706, 4807, 3774, 18707, 35895), (10858, 35485, 18706, 28080, 35484), (20192, 645, 644, 8321), (16395, 645, 8877, 8321), (20296, 13981, 36086, 4804), (16355, 17471, 28077, 20288), (10858, 16358, 20288, 3774), (35893, 18707, 35484, 17455, 8816), (17456, 7320, 24181, 35895, 8816), (3526, 643, 8874, 27024), (8802, 17063, 1614, 17431), (35893, 8816, 24967, 645, 37495), (8816, 13981, 17442, 17438), (17442, 17438, 8802, 17063), (17474, 1334, 28080, 35485), (17471, 1333, 28077, 17456), (5060, 26932, 1333, 641), (17455, 35895, 13981, 17441, 16395, 17442, 17443, 17444), (17472, 5484, 1618, 1334), (17470, 5484, 1618, 12171), (17455, 641, 12171, 35483, 13981), (20192, 644, 3526, 8874), (1618, 12171, 7320, 17455), (28080, 7320, 35484, 35895), (17430, 17431, 8874, 2663, 8871), (20288, 28077, 3774, 18707), (20192, 37495, 2415, 17439), (10858, 3774, 35896, 35484, 4807), (8816, 645, 17443, 644)}
for tup in neigh:
    if 496 in tup:
        print(tup)

In [ ]:
a = {231: {'in': {12697, 37978, 10428}, 'out': set()}, 212: {'in': set(), 'out': {12697, 37978, 10428}}}

In [ ]:
a={37161, 16035, 1203, 11501}

b = {11909, 8839, 8335, 16035, 5800, 37161, 13483, 1200, 1201, 30898, 25776, 22452, 20914, 22454, 1203, 955, 
              24775, 25048, 13418, 26219, 26218, 11501, 11503}
for node in b:
    if graphhh.nodes[node]['candidate']==1:
        print(node)

16035
37161
11501
1203


In [43]:
node_flows ={549: {'in': {11909, 8839, 8335, 16035, 5800, 37161, 13483, 1200, 1201, 30898, 25776, 22452, 20914, 22454, 1203, 955, 
                 24775, 25048, 13418, 26219, 26218, 11501, 11503}, 
          'out': set()}, 
    389: {'in': set(), 
          'out': {11909, 8839, 8335, 16035, 5800, 37161, 13483, 1200, 1201, 30898, 25776, 22452, 20914, 
                  22454, 1203, 955, 24775, 25048, 13418, 26219, 26218, 11501, 11503}}}



In [44]:

part_549 = frozenset({10497, 12802, 13826, 13828, 13829, 13830, 26891, 14879, 25130, 30507, 25901, 39214, 37166, 37165, 
           39213, 16691, 18232, 8764, 25411, 12099, 16711, 26441, 26443, 17228, 24908, 6224, 1109, 1898, 1899, 
           14201, 37242, 11912, 5779, 5780, 5782, 8343, 5793, 5794, 5801, 22442, 38314, 38315, 25773, 38316, 
           20909, 30899, 30902, 30903, 29373, 29374, 25789, 39112, 17097, 3285, 34268, 34781, 34784, 34785, 
           34787, 7142, 33002, 33004, 33005, 11502, 21487, 11504, 20465, 33527})
len(part_549)

68

In [45]:
part_389 = frozenset({26882, 11909, 8839, 8335, 7575, 16035, 3747, 7204, 13735, 5800, 37161, 37162, 13483, 13484, 25776, 1200, 12210, 30898, 1203, 20914, 1201, 22454, 22452, 10810, 955, 18879, 25798, 2118, 24775, 34255, 24015, 13656, 25048, 21606, 8169, 26218, 26219, 13418, 11501, 11503, 19967})
len(part_389)

41

In [54]:
parts = {"389":part_389, "549":part_549}

In [50]:
candidates = {}
for part in parts:
    candidates[part] = {node for node in parts[part] if graphhh.nodes[node]['candidate']==1}
candidates

{'389': {1203, 8169, 10810, 11501, 13735, 16035, 25798, 37161},
 '549': {11912, 16691, 16711, 25411, 25789, 29373, 33527, 37242, 39112, 39213}}

In [51]:
centers = {}
radius = {}

for part in parts:
    
    save_candidates_radius = {}

    for candidate in candidates[part]:
        candidate_radius = max(real_travel_times[(candidate, node)] for node in parts[part])
        save_candidates_radius[candidate] = candidate_radius
    
    best_candidate = min(save_candidates_radius, key=save_candidates_radius.get)  # center of the part
    centers[part] = best_candidate
    radius[part] = save_candidates_radius[best_candidate]

In [52]:
print(centers)
print(radius)

{'389': 25798, '549': 25411}
{'389': 16, '549': 24}


In [53]:
teams = {}
for part in parts:
    teams[part] = 2
teams

{'389': 2, '549': 2}

In [46]:
import collections

def create_flow():
    return {"in": set(), "out": set()}

candidate_flows = collections.defaultdict(create_flow)


candidate_flows = collections.defaultdict(create_flow)
for part in node_flows:
    candidate_flows[part] ={
    "in" : {node for node in node_flows[part]["in"] if graphhh.nodes[node]['candidate']==1},
    "out": {node for node in node_flows[part]["out"] if graphhh.nodes[node]['candidate']==1}
    }


In [47]:
candidate_flows

defaultdict(<function __main__.create_flow()>,
            {549: {'in': {1203, 11501, 16035, 37161}, 'out': set()},
             389: {'in': set(), 'out': {1203, 11501, 16035, 37161}}})

In [48]:
part_flows = {'in': {549, 389}, 'out': set()}

In [ ]:
def compute_part_flows(merged_parts, new_ids):
    old_ids = merged_parts.difference(new_ids)  # keys that will be removed from partition.parts.keys()
    new_ids = new_ids.difference(merged_parts)  # will be added to partition.parts.keys()
    part_flows = {"in": set(new_ids), "out": set(old_ids)}
    return part_flows

merged_parts=set()
flip.new_ids={549, 389}

In [ ]:
for part in part_flows["in"]:
    parts[part] = set()
    candidates[part] = set()
    centers[part] = None
    radius[part] = None
    teams[part] = None

for part in part_flows["out"]:
    parts.pop(part, None)
    candidates.pop(part, None)
    centers.pop(part, None)
    radius.pop(part, None)
    teams.pop(part, None)






def update_flows(self, node_flows, part_flows, team_flips, candidate_flows):

    self.update_part_flows(part_flows)

    for part, flow in node_flows.items():
        if part not in part_flows["out"]:
        
            # updating mapping
            for node in flow["in"]:
                self.mapping[node] = part

            # update part ndoes
            self.parts[part] = frozenset((self.parts[part] - flow["out"]) | flow["in"])
            
            self.teams[part] = team_flips[part]
            
            # update candidates
            self.candidates[part] = frozenset((self.candidates[part] - candidate_flows[part]["out"]
                ) | candidate_flows[part]["in"])
            
            self.centers[part], self.radius[part] = self.facility_assignment(part)
                

In [ ]:
from falcomchain.tree import Cut, Flip
Flip(flips={}, team_flips={}, new_ids=set(), merged_ids=set(), super_cut_object=Cut(node=389, subnodes=frozenset({389}), assigned_teams=2, pop=2956))

In [56]:
from falcomchain.tree import Cut, Flip
cut_object = Cut(node=389, subnodes=frozenset({389}), assigned_teams=2, pop=2956)
merge = set(cut_object.subnodes)
print(merge)


{389}


In [57]:
superflip = Flip(merged_ids=merge, super_cut_object=cut_object)
print(superflip)

Flip(flips={}, team_flips={}, new_ids=set(), merged_ids={389}, super_cut_object=Cut(node=389, subnodes=frozenset({389}), assigned_teams=2, pop=2956))


In [ ]:
merge = set(cut_object.subnodes)
    superflip = Flip(merged_ids=merge, super_cut_object=cut_object)
    subgraph = partition.graph.graph.subgraph(
        set.union(*(set(partition.parts[part]) for part in merge)))
    
    max_id = max(district for district in partition.parts)
    sub_assignments = {node: partition.assignment.mapping[node] for node in subgraph.nodes}

In [33]:
import collections

def create_flow():
    return {"in": set(), "out": set()}

node_flows = {231: {'in': {12697, 37978, 10428}, 'out': set()}, 212: {'in': set(), 'out': {12697, 37978, 10428}}}

candidate_flows = collections.defaultdict(create_flow)

for part in node_flows:
    candidate_flows[part] ={
    "in" : {node for node in node_flows[part]["in"] if graphhh.nodes[node]['candidate']==1},
    "out": {node for node in node_flows[part]["out"] if graphhh.nodes[node]['candidate']==1}
    }
candidate_flows

defaultdict(<function __main__.create_flow()>,
            {231: {'in': {10428}, 'out': set()},
             212: {'in': set(), 'out': {10428}}})

In [2]:
Assignment.travel_times = MappingProxyType(real_travel_times)
initial_solution = Partition.load_partition(assignment_class=Assignment)

In [20]:
#Cut = namedtuple("Cut", "node subnodes assigned_teams pop")
district_nodes = frozenset({8450, 2568, 11536, 1814, 24471, 34714, 12700, 24862, 18080, 22820, 22821, 33062, 31271, 31270, 33064, 31272, 
                          33063, 22575, 8372, 16180, 18101, 5953, 16961, 34883, 34881, 23877, 34884, 9539, 22473, 34892, 20429, 22478, 
                          28240, 20945, 28370, 28371, 15444, 15445, 5847, 30430, 20447, 5856, 5855, 23134, 14821, 34280, 32364, 32365, 
                          32366, 32367, 32368, 30458, 6779, 19325, 33279})
current_new_ids = {346} # already assigned one
remaining_ids= {254}
max_id = 549
assigned_teams = 2

supergraph_subnodes = frozenset({346, 254})
total_teams = 3
assignments = {33281: 346, 8450: 254, 1798: 346, 2568: 346, 11536: 254, 1814: 346, 24471: 254, 34713: 346, 21274: 346, 34714: 346, 
                  12700: 346, 24862: 346, 34719: 346, 18080: 346, 30459: 346, 6051: 346, 6052: 346, 22820: 346, 33062: 346, 33063: 346, 
                  31272: 346, 33064: 346, 22821: 346, 31270: 254, 31271: 254, 22701: 346, 22575: 346, 18099: 346, 10164: 346, 10165: 346, 
                  10166: 346, 8372: 346, 16180: 254, 10169: 346, 10170: 346, 10171: 346, 18101: 254, 5953: 346, 16961: 254, 9539: 346, 
                  34884: 346, 34881: 254, 26950: 346, 34887: 346, 34883: 254, 22473: 346, 23877: 254, 34892: 254, 20429: 346, 22478: 346, 
                  28240: 346, 20945: 254, 28370: 346, 28371: 346, 7506: 346, 15444: 254, 15445: 254, 5847: 254, 23134: 346, 30430: 254, 
                  20447: 254, 5856: 254, 5855: 254, 14821: 346, 34280: 346, 32364: 254, 32365: 346, 32366: 254, 32367: 346, 32368: 346, 
                  30458: 346, 6779: 254, 19325: 254, 33279: 346}

In [ ]:
if len(remaining_ids)>0:
    remarkable_district_nodes = {node for node in district_nodes if assignments[node] in remaining_ids}
    
    if len(remarkable_district_nodes)>0:
        assignment_counts = Counter([assignments[node] for node in remarkable_district_nodes])
        district = max(assignment_counts, key=assignment_counts.get)
        remaining_ids.remove(district)
    else: 
        district = random.choice(list(remaining_ids))
        remaining_ids.remove(district)
else:
    district = max_id
    max_id = max_id + 1
        


set()

In [ ]:
if len(remaining_ids) > 0:
    remarkable_district_nodes = {node for node in district_nodes if assignments[node] in remaining_ids}

    if len(remarkable_district_nodes) > 0:
        assignment_counts = Counter([assignments[node] for node in remarkable_district_nodes])
        # Find the most frequent *available* district
        district = max(assignment_counts, key=assignment_counts.get)
        remaining_ids.remove(district)
    else:
        # Get the ID, convert remaining_ids to list only once
        remaining_list = list(remaining_ids) 
        district = random.choice(remaining_list)
        # We can use the set's remove method on the result
        remaining_ids.remove(district)
else:
    # Create a new district ID
    district = max_id
    max_id += 1 # Use += 1 for cleaner code

In [ ]:
from collections import Counter

assignment_counts = Counter([assignments[node] for node in district_nodes])
if len(remaining_ids)>0:
    district = max(assignment_counts, key=assignment_counts.get)
    remaining_ids.remove(district)
else:
    district = max_id
    max_id = max_id + 1
print(district)


ValueError: max() iterable argument is empty

In [10]:
from collections import Counter

assignment_counts = Counter([assignments[node] for node in district_nodes if assignments[node] in remaining_ids])
if len(remaining_ids)>0:
    district = max(assignment_counts, key=assignment_counts.get)
    remaining_ids.remove(district)
else:
    district = max_id
    max_id = max_id + 1

In [11]:
assignment_counts

Counter({254: 24})

In [16]:
counts = Counter([])
counts

Counter()

In [12]:
district

254

In [24]:
district

549

In [14]:
district

346